## Загрузка библиотек

In [1]:
#basic
import pandas as pd
from typing import Dict
import numpy as np
import re

#error
import warnings
warnings.filterwarnings('ignore')

## Загрузка данных

#### Напишем функцию, с помощью которой будем загружать данные

> Так как у нас таблицы из разных источников, то и форматирование у них разное, в том числе начало строк с информацией по блогерам можем быть на 10 строчки, а название столбцов у разных источников отличаться. Чтобы привести к единому формату, сделаем функции и в укажет ней ключи, по которым обрежем таблицу снизу и сверху

In [2]:
def read_csv_from_dict(data_dict: Dict[str, str], column_number: int, header_keyword: str, footer_keyword: str) -> pd.DataFrame:
    
    
    """
    Функция принимает как параметр словарь data_dict, в котором ключ - название DataFrame, а объект - ссылка на источник,
    и в дальнейшем мы обрезаем DataFrame с помощью параметров column_number, header_keyword, footer_keyword.
    
    ПАРАМЕТРЫ:
    - data_dict (DataFrame) - словарь, в котором ключ - это будущее название датафрейма, объект - 
    ссылка на страницу таблицы, из которой берём данные.
    - column_number (int) - номер столбца, по которому определяем границы DataFrame.
    - header_keyword (str) - ключ, по которому мы обрезаем таблице сверху.
    - footer_keyword - ключ, по которому мы обрезаем таблице снизу.
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df: DataFrame, название и источник которого указан в словаре data_dict и который обрезан по нашим параметрам header_keyword и footer_keyword
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    didenok_df = read_csv_from_dict(didenok_data_dict, 0, didenok_header_keyword, didenok_footer_keyword)
    """
    
    for name, url in data_dict.items():
        url = url.replace('/edit#gid=', '/export?format=csv&gid=')
        df = pd.read_csv(url)

# В pandas метод .iloc используется для доступа к элементам в DataFrame или Series по их целочисленным позициям.
# Индексация начинается с нуля, поэтому .iloc[n] обращается к элементу с позицией n-1.        
    
        header_index = df.loc[df.iloc[:, column_number].str.strip().isin(header_keyword)].index[0]
        
#Используем globals() для создания новой переменной с именем, которое хранится в переменной name,
# и присваиваем этой переменной значение, полученное в результате операции df.set_axis(df.iloc[header_index], axis='columns').

        globals()[name] = df.set_axis(df.iloc[header_index], axis='columns')

        if footer_keyword:
            footer_index = df.loc[df.iloc[:, column_number].str.strip().isin(footer_keyword)].index[0] - 2
            globals()[name] = globals()[name].iloc[header_index + 1:footer_index]
        else:
            globals()[name] = globals()[name].iloc[header_index + 1:]
            
    return df

In [3]:
print(read_csv_from_dict.__doc__)


    Функция принимает как параметр словарь data_dict, в котором ключ - название DataFrame, а объект - ссылка на источник,
    и в дальнейшем мы обрезаем DataFrame с помощью параметров column_number, header_keyword, footer_keyword.
    
    ПАРАМЕТРЫ:
    - data_dict (DataFrame) - словарь, в котором ключ - это будущее название датафрейма, объект - 
    ссылка на страницу таблицы, из которой берём данные.
    - column_number (int) - номер столбца, по которому определяем границы DataFrame.
    - header_keyword (str) - ключ, по которому мы обрезаем таблице сверху.
    - footer_keyword - ключ, по которому мы обрезаем таблице снизу.
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df: DataFrame, название и источник которого указан в словаре data_dict и который обрезан по нашим параметрам header_keyword и footer_keyword
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    didenok_df = read_csv_from_dict(didenok_data_dict, 0, didenok_header_keyword, didenok_footer_keyword)
    


> Стоит отметить, что так как исходные данные у нас лежат на Google-таблицах, то помимо разного формата, мы ещё столнулись с тем, что часть ссылок сделаны гиперссылками в виде текста, которые Pandas и специальные сторонние библиотеки переводят текст. Поэтому чтобы не разводить лишний код, мы сделали определённые преобразования в Google-таблицах, а именно:
>* Через функцию IMPORTRANGE копируем данные из разных Google-таблиц в единую.
>* Напишем функцию для App Script на специальном языке google script (gs), которая преобразовывает гиперссылки в ссылки и выводит их в отдельный столбец (пример функции ниже):
>
><em>function GetURL(input) {  
  var myFormula = SpreadsheetApp.getActiveRange().getFormula();  
  var myAddress = myFormula.replace('=GetURL(','').replace(')','');   
  var myRange = SpreadsheetApp.getActiveSheet().getRange(myAddress);  
  return myRange.getRichTextValue().getLinkUrl();  
 };</em>  
> 
>**За счёт этого мы**:
>* Получим единую Google-таблицу, в которой видим все предварительные данные, в том числе если будут какие-то изменения в первоначальных данным, то мы это увидим, что позволит нам избежать ошибки.
>* Решим проблему с гиперссылками.

### Блогеры от Didenok Team

In [4]:
didenok_data_dict = {'didenok_instagram':'HREF',
                     'didenok_youtube':'HREF',
                     'didenok_telegram':'HREF',
                     'didenok_vk_zen_show':'HREF',
                     'didenok_vk_group':'HREF',
#                      'didenok_vk_ls':'HREF',
                     'didenok_twitch':'HREF',
                    }

In [5]:
didenok_header_keyword = ['Блогер', 'Канал']
didenok_footer_keyword = ['Примечание']

In [6]:
didenok_df = read_csv_from_dict(didenok_data_dict,
                                0,
                                didenok_header_keyword,
                                didenok_footer_keyword)

> Проверим результат

In [7]:
didenok_instagram.head()

7,Блогер,Ссылка,Категория/\nТематика блогера,Вид сотрудничества**,Количество подписчиков,Средний охват фотопоста*,Средний охват 1 сторис*,Среднее кол-во просмотров Reels*,Среднее кол-во лайков*,Среднее кол-во комментариев*,"ERR, %",Стоимость за фотопост (руб.) до НДС,Стоимость за 1 сторис до 15 сек (руб.) до НДС,Стоимость за серию из 3х сториc до 45 сек (руб.) до НДС,Стоимость за Reels (руб.) до НДС,CPU (фото),CPU (сторис),CPV \n(Reels),Статистика,Статистика
8,Катя Клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,П,10 100 000,250 000,150 000,200 000,50 000,100,"20,04%",р.375 000,р.335 000,р.530 000,р.820 000,"р.1,5","р.2,2","р.4,1",Статистика,https://drive.google.com/drive/folders/1J26W2R...
9,Влад Бумага А4,https://www.instagram.com/a4omg/,челленджи,П,7 100 000,900 000,400 000,1 000 000,60 000,700,"6,74%",р.875 000,р.440 000,р.625 000,р.1 875 000,"р.1,0","р.1,1","р.1,9",-,NaN
10,Катя Адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,Э,6 300 000,700 000,260 000,600 000,30 000,100,"4,30%",р.445 000,р.305 000,р.625 000,р.685 000,"р.0,6","р.1,2","р.1,1",Статистика,https://drive.google.com/drive/u/1/folders/11S...
11,Ваша Маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,Э,3 300 000,185 000,115 000,350 000,30 000,100,"16,27%",р.240 000,р.160 000,р.315 000,р.315 000,"р.1,3","р.1,4","р.0,9",Статистика,https://drive.google.com/drive/folders/1xNqiOI...
12,Амина Тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,Э,3 900 000,600 000,300 000,650 000,110 000,150,"18,36%",р.695 000,р.265 000,р.625 000,р.835 000,"р.1,2","р.0,9","р.1,3",Статистика,https://drive.google.com/drive/u/1/folders/1M3...


### Блогеры от VK

In [8]:
vk_data_dict = {'vk_bloggers':\
                'HREF',
               }

In [9]:
vk_header_keyword = ['Сообщество']
vk_footer_keyword = []

In [10]:
vk_df = read_csv_from_dict(vk_data_dict,
                           0,
                           vk_header_keyword,
                           vk_footer_keyword
                          )

> Проверим результат

In [11]:
vk_bloggers.head()

,Сообщество,Шоу,KPI (стартов просмотра на 1 эпизод на 30-е сутки после релиза),Стоимость интеграции 1-й слот до НДС,Стоимость интеграции 2-й слот до НДС,Продакт плейсмент (30 сек) до НДС,Стоимость рекламной заставки (пре-ролл до 20 сек) до НДС,Стоимость рекламной заставки (мид-ролл до 20 сек) до НДС,Статистика,Контакты менеджера по продажам,Ссылка,Статистика ссылка
1,Азамат Мусагалиев,Большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,Статистика,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,Это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,NaN,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,Я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,NaN,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,Шоу памяти Азамата Мусагалиева NEW,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,NaN,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,Пикабу,Чебатков читает Пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


### Блогеры от BlogoSphere

In [12]:
blogosphere_data_dict = {'blogosphere_youtube':'HREF',
                        'blogosphere_tg_vk_zen':'HREF'
                        }

In [13]:
blogosphere_header_keyword = ['Название канала', 'Имя блогера']
blogosphere_footer_keyword = []

In [14]:
blogosphere_df = read_csv_from_dict(blogosphere_data_dict,
                                    1,
                                    blogosphere_header_keyword,
                                    blogosphere_footer_keyword)

> Проверим результат

In [15]:
blogosphere_youtube.head()

5,NaN,Название канала,Шоу,Подписчики,Среднее кол-во просмотров на видео,Статистика аудитории,Стандартные форматы,NaN,NaN,NaN,...,NaN,NaN,NaN,Пример рекламы 1,Пример рекламы 2,Пример рекламы 3,Среднее CPV,NaN,Ссылка,Статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,...,NaN,Формат 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,...,Цена* от,Описание,Цена* от,NaN,NaN,NaN,Стандартный Формат 1,Стандартный Формат 2,NaN,NaN
8,Э,NaN,"Осторожно, Собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,...,1 310 000 ₽,Product Placement,1 470 000 ₽,Eduson,Экоплант,Травелата,"1,31₽","0,96₽",NaN,NaN
9,NaN,Ксения Собчак,Осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,...,NaN,NaN,NaN,Astons,CGPods,SkillFactory,"1,38₽","0,93₽",https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,NaN,Красота требует КЭШ,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,...,NaN,NaN,NaN,Золотое яблоко,4FORMS,FOREO,"1,20₽","1,08₽",NaN,NaN


### Блогеры от Insight People

In [16]:
insight_data_dict = {'insight_instagram':'HREF',
                     'insight_threads':'HREF',
                     'insight_telegram':'HREF',
                     'insight_vk_ls':'HREF',
                     'insight_vk_group':'HREF',
                     'insight_youtube':'HREF',
                     'insight_zen':'HREF',
                     'insight_likee':'HREF',
                     'insight_yappy':'HREF',
                     'insight_rutube':'HREF',
#                      'insight_ok':'HREF',
                    }

In [17]:
insight_header_keyword = ['Город']
insight_footer_keyword = []

In [18]:
insight_df = read_csv_from_dict(insight_data_dict,
                                0,
                                insight_header_keyword,
                                insight_footer_keyword)

> Проверим результат

In [19]:
insight_rutube.head()

3,Город,ФИО,Ссылка,Количество подписчиков,Средние просмотры за месяц,Стоимость преролла,Стоимость интеграции (до 1 минуты),"CPV, преролл","CPV, интеграция"
4,Тверь,Аркатов Станислав Александрович,https://rutube.ru/channel/23684374/,Фудблогер,118 300,25 030,"р.50 060,00","р.0,75","р.2,00"
5,Краснодар,Солодов Юрий Юрьевич,https://rutube.ru/channel/23722477/,"Лайфстайл, влог",24 500,47 000,"р.82 250,00","р.0,50","р.1,75"
6,Казань,Ахмадуллин Наиль Радикович,https://rutube.ru/channel/23694775/,"Путешествия, лайфстайл",18 500,27 524,под запрос,-,-
7,Воронеж,Рязанова Татьяна Алексеевна,https://rutube.ru/channel/19363597/,"Бьюти, лайфстайл",16 500,50 066,"р.87 615,00","р.0,75","р.2,00"
8,Воронеж,Лаврова Валерия Владимировна,https://rutube.ru/channel/26070201/,Художник,15 600,40 063,"р.70 115,00","р.0,75","р.2,00"


### Блогеры от NextUp

In [20]:
nextup_data_dict = {'nextup_instagram':'HREF',
                    'nextup_telegram':'HREF',
                    'nextup_youtube':'HREF',
                    'nextup_twitch':'HREF',
                    'nextup_vk':'HREF',
                   }

In [21]:
nextup_header_keyword = ['Имя блогера']
nextup_footer_keyword = []

In [22]:
nextup_df = read_csv_from_dict(nextup_data_dict,
                               2,
                               nextup_header_keyword,
                               nextup_footer_keyword
                              )

> Проверим результат

In [23]:
nextup_vk.head()

5,NaN,NaN,Имя блогера,Ссылка,Кол-во подписчиков,Статистика,ЦА по городам,ЦА по возрасту,ЦА по полу,Охват поста за 30 дней,Охват VK клип за 30 дней,Стоимость поста,CPV пост,Стоимость VK клип,CPV Клип,Статистика ссылка
6,NaN,П,Илья Exile,https://vk.com/exile_music,507 872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,П,Paradeev1ch,https://vk.com/paradeev1ch,414 000,NaN,NaN,NaN,NaN,45 000,-,78 500 ₽,NaN,71 500 ₽,NaN,NaN
8,NaN,П,Frametamer,https://vk.com/frametamer1337,262 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Э,ЛУНАКОМ,https://vk.com/lynacom2000,253 000,статистика,NaN,NaN,NaN,1 200 000,550 000,NaN,"0,00 ₽",NaN,"0,00 ₽",https://drive.google.com/drive/folders/1JEkYq2...
10,NaN,Э,Bad Barbie,https://vk.com/badbarbieeeeeee,167 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Блогеры от AvtorMedia

In [24]:
avtormedia_data_dict = {'avtormedia_youtube':'HREF',
                        'avtormedia_instagram':'HREF',
                        'avtormedia_telegram':'HREF',
                        #'avtormedia_tiktok':'HREF',
                        'avtormedia_vk':'HREF',
                        'avtormedia_zen':'HREF',
                        'avtormedia_podcast':'HREF',
                       }

In [25]:
avtormedia_header_keyword = ['Блогер']
avtormedia_footer_keyword = []

In [26]:
avtormedia_df = read_csv_from_dict(avtormedia_data_dict,
                                   0,
                                   avtormedia_header_keyword,
                                   avtormedia_footer_keyword
                                  )

> Проверим результат

In [27]:
avtormedia_telegram.head()

,Блогер,Тематика,Подписчики,Материалы\nи статистика,Средние просмотры,Короткое видео,Пост,Спешл,Цена,Спешл,...,Пакетное предложение,Цена,Охват,CPV,Пакетное предложение,Цена,Охват,CPV,Ссылка,Статистика ссылка
1,Илья Варламов @varlamov_news,новости,1 200 000,Ссылка,470 000,833 333 ₽,450 000 ₽,Сторис,500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov_news,https://drive.google.com/drive/u/1/folders/165...
2,Mamix,эксперименты,337 000,Ссылка,261 000,NaN,425 531 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/its_mamix,https://drive.google.com/drive/u/1/folders/12S...
3,Илья Варламов @varlamov,урбанистика\nобщество\nполитика,329 000,Ссылка,212 000,888 888 ₽,600 000 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov,https://drive.google.com/drive/u/1/folders/165...
4,Венедиктов,общество\nполитика,223 000,Ссылка,131 000,NaN,250 000 ₽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/aavst2022,https://drive.google.com/drive/folders/1UtGUA_...
5,Даша Козловская,творчество\nбьюти\nлайфстайл,186 400,Ссылка,81 000,430 000 ₽,350 000 ₽,NaN,NaN,NaN,...,Фото-пост в Инст + пост в Telegram,1 402 009 ₽,362 000,"3,87",NaN,NaN,NaN,NaN,https://t.me/dashakozlouskayatg,https://drive.google.com/drive/u/1/folders/1cW...


### Блогеры от PLATFORMA.COMMUNITY

In [28]:
sheet_url = 'HREF',
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_youtube = pd.read_csv(csv_export_url, header=1)

sheet_url = 'HREF',
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_instagram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'HREF',
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
platformacommunity_telegram = pd.read_csv(csv_export_url, header=0)

In [29]:
platformacommunity_youtube.head()

,Вид контракта,Канал,Проект\nВедущий\nФормат,Тематика,Подписчики,Средние просмотры для рекламных интеграций,Статистика,"Pre-roll / OLV, от, руб., до учета налогов","CPV, pre-roll, от, руб., до учета налогов","MID-roll / OLV, от, руб., до учета налогов","Интеграция 1-ый слот, от, руб., до учета налогов","CPV, 1 слот, от, руб., до учета налогов","Интеграция 2-ой слот, от, руб., до учета налогов","CPV, 2 слот, от, руб., до учета налогов","Продакт плейсмент, от, руб., до учета налогов","Размещение в сообществе, от, руб., до учета налогов",Пример,Ссылка,Статистика ссылка
0,Э,Уральские Пельмени,Официальный канал шоу «Уральские Пельмени»,Специальные сборки номеров под рекламодателя,"2 550 000,00","2 000 000,00",Статистика,"1 200 000,00","0,60",NaN,"2 500 000,00","1,25",NaN,NaN,NaN,NaN,Тинькофф,https://www.youtube.com/c/ProPelmeny/featured,https://drive.google.com/drive/folders/1vQIyBd...
1,NaN,NaN,NaN,Эксклюзивные номера,NaN,"1 100 000,00",NaN,"850 000,00","0,77",NaN,NaN,"0,00",NaN,NaN,NaN,NaN,Пятерочка,NaN,NaN
2,NaN,NaN,NaN,Регулярные выкладки,NaN,"700 000,00",NaN,"600 000,00","0,86",NaN,NaN,NaN,NaN,NaN,NaN,NaN,HAVAL,NaN,NaN
3,NaN,NaN,NaN,"""ЕДЕМ С ПЕЛЬМЕНЯМИ"" travel шоу про гастроли и ...",NaN,"300 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"800 000,00",NaN,Орматек,NaN,NaN
4,NaN,ВПИСКА,Коля Редькин,Интервью,"1 910 000,00","2 500 000,00",Статистика,"1 500 000,00",NaN,по запросу,по запросу,#VALUE!,NaN,"0,00",по запросу,NaN,NaN,https://www.youtube.com/@VPISKA/featured,https://drive.google.com/drive/folders/1KxM-Mc...


### Блогер ТОПЛЕС

In [30]:
sheet_url = 'HREF',
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_telegram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'HREF',
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_instagram = pd.read_csv(csv_export_url, header=0)

sheet_url = 'HREF',
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
toples_vk = pd.read_csv(csv_export_url, header=0)

In [31]:
toples_telegram.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Telegram-канал Топлес,NaN,NaN,NaN,NaN
2,NaN,https://t.me/toplesofficial,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Подписчики: 500К,NaN,NaN,NaN,NaN
5,NaN,Охват поста: 120-130К,NaN,NaN,NaN,Условия актуальны до 15.12.2023
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Формат публикации,NaN,NaN,Подробности формата,Стоимость (от)
8,NaN,1,Текстовый пост\n\nМатериалы клиента,- текст от клиента\n- визуал от клиента (по не...,"Материалы клиента, при необходимости адаптируе...",р.85 000
9,NaN,2,Текстовый пост\n\nПишем сами по брифу клиента,- текст по брифу\n- визуал от клиента (по необ...,Текст пишем в стиле канала по брифу клиента. Т...,р.100 000


## Подготовка данных


#### Функция, с помощью которой мы предварительно подготовим DataFrame для дальнейшей работы

In [32]:
def pretraining_df(df: pd.DataFrame,
                   drop_keywords: str,
                   drop_word: str,
                   upd_columns: str) -> pd.DataFrame:

    """
    Функция принимает DataFrame - df, с которым мы делаем преобразования:
    • Редактируем названия столбцов: приводим к нижнему регистру, убираем лишние пробелы до и после текста, удаляем перенос \n и сноски *.
    • Удаляем столбцы по ключевым словам drop_keywords. В коде учтено, что их может и не быть.
    • Удаляем ключевые слова drop_word из названия столбцов. В коде учтено, что их может и не быть.
    • Столбцы upd_columns, в которых содержится текстовая информация для дальнейшего объединения разных DataFrame,
    приводим к единому формату по примеру первого пункта, а также заполняем NaN с помощью 

    
    ПАРАМЕТРЫ:
    df (DataFrame) - DataFrame, в котором мы будем вносить изменения
    drop_keywords (str) - список ключевых слов для удаления столбцов
    drop_word (str) - слово для удаления из названий столбцов
    upd_columns (str) - список имен столбцов для приведения к нижнему регистру и удаления символов
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df - DataFrame, в котором мы делали преобразования
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    key_word = [] # Список ключевых слов для поиска столбцов и их удаления
    drop_word = ' до ндс' # Список ключевых слов для их удаления из названия столбцов
    upd_columns = ['сообщество', 'шоу'] # Список столбцов с именами блогеров для приведения к единому формату
    upd_columns = [column.lower() for column in upd_columns]

    vk_bloggers = pretraining_df(vk_bloggers, key_word, drop_word, upd_columns)
    
    """
    
    drop_keywords = [keywords.lower() for keywords in drop_keywords] #
    drop_keywords = [keyword.strip() for keyword in drop_keywords]
    drop_word = drop_word.lower()
    upd_columns = [column.lower() for column in upd_columns] #
    
    
    df.columns = df.columns.str.strip()\
                            .str.lower()\
                            .str.replace('/\n', ', ')\
                            .str.replace('\n', ', ')\
                            .str.replace('*', '')
    
# Найдём столбцы по ключевым словам и их удалим их, так как в DataFrame у нас встречаются столбцы,
# которые не несут уникальной информации, например cpm, cpv и тд,
# и которые мы можем вычислить точнее через математические операции.


    if drop_keywords:
        columns_to_drop = [column for column in df.columns if isinstance(column, str) and any(keyword in column for keyword in drop_keywords)]
        df.drop(columns = columns_to_drop, axis=1, inplace=True)

        
    
# Удалим ключевые слова из названия столбцов
    if drop_word:
        df.columns = df.columns.str.replace(drop_word, '', regex=False)


# Столбцы, в которых содержится информация для дальнейшего объединения, приведём к единому формату
    for upd_column in upd_columns:
        df[upd_column] = df[upd_column].fillna('').apply(lambda x: str(x).strip().lower()\
                                                                         .replace('/\n', ', ')\
                                                                         .replace('\n', ', ')\
                                                                         .replace('*', '')\
                                                             if isinstance(x, str) else x)
        # Заменим пустую строку на NaN
        df[upd_column] = df[upd_column].replace('', np.nan)
    
    return df

## Подготовка данных по блогерам от Didenok Team

#### Имеющиеся датафреймы:  

* didenok_instagram  
* didenok_youtube  
* didenok_telegram  
* didenok_vk_zen_show  
* didenok_vk_group  
<!-- * didenok_vk_ls   -->
* didenok_twitch  

> Сделаем список DataFrame, чтобы в дальнейшем работать с ним

In [33]:
didenok_df = [didenok_instagram,
              didenok_youtube,
              didenok_telegram,
              didenok_vk_zen_show,
              didenok_vk_group,
#               didenok_vk_ls,
              didenok_twitch]

> Так как до предобработки названий столбцов DataFrame название столбца с именем блогера может быть как "Блогер", так и "Канал", учтём это в цикле ниже для запуска функции pretraining_df

In [34]:
key_word = ['вид сотрудничества', 'cpv', 'cpu', 'err']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ' (руб.) до ндс'  # Список ключевых слов для их удаления из названия столбцов

for i, df in enumerate(didenok_df):
    if 'Блогер' in df.columns:
        upd_columns = ['блогер']  # Столбец с именами блогеров для приведения к единому формату
        pretraining_df(df, key_word, drop_word, upd_columns)
    elif 'Канал' in df.columns:
        upd_columns = ['канал']  # Столбец с именами блогеров для приведения к единому формату
        pretraining_df(df, key_word, drop_word, upd_columns)
    else:
        print(f'Не удалось найти столбец "Блогер" или "Канал" для преобразования DataFrame {i}') 

> Проверим результат

In [35]:
didenok_youtube.head()

7,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,стоимость за shorts до 1 мин,статистика,статистика ссылка
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,Статистика,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,Статистика,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,Статистика,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,Статистика,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,Статистика,https://drive.google.com/drive/folders/1VTEv65...


### Подготовка таблицы didenok_instagram

In [36]:
[col for col in didenok_instagram.columns]

['блогер',
 'ссылка',
 'категория, тематика блогера',
 'количество подписчиков',
 'средний охват фотопоста',
 'средний охват 1 сторис',
 'среднее кол-во просмотров reels',
 'среднее кол-во лайков',
 'среднее кол-во комментариев',
 'стоимость за фотопост',
 'стоимость за 1 сторис до 15 сек',
 'стоимость за серию из 3х сториc до 45 сек',
 'стоимость за reels',
 'статистика',
 'статистика']

In [37]:
bufer_columns = ['блогер',
                 'ссылка',
                 'тематика',
                 'количество подписчиков',
                 'среднее кол-во просмотров фотопоста',
                 'среднее кол-во просмотров 1 сторис',
                 'среднее кол-во просмотров reels / видеопоста',
                 'среднее кол-во лайков',
                 'среднее кол-во комментариев',
                 'стоимость фотопост',
                 'стоимость 1 сторис до 15 сек',
                 'стоимость серия из 3х сторис до 45 сек',
                 'стоимость reels / видеопост',
                 'статистика удалить',
                 'статистика']

In [38]:
didenok_instagram.columns = bufer_columns

In [39]:
didenok_instagram.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров reels / видеопоста,среднее кол-во лайков,среднее кол-во комментариев,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия из 3х сторис до 45 сек,стоимость reels / видеопост,статистика удалить,статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,Статистика,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,-,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,Статистика,https://drive.google.com/drive/u/1/folders/11S...
11,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,100,р.240 000,р.160 000,р.315 000,р.315 000,Статистика,https://drive.google.com/drive/folders/1xNqiOI...
12,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,Статистика,https://drive.google.com/drive/u/1/folders/1M3...


> Удалим столбец со статистикой, в котором не высветились гиперссылки

In [40]:
didenok_instagram.drop(columns = ['статистика удалить'], axis=1, inplace=True)

> Удалим первую строку, которая не показывает что-то по блогерам, а являестся побочной строкой названия столбцом.  
> <i>[Она то появляется, то исчезает, поэтому оставим как удаление пропуска]</i>

In [41]:
didenok_instagram.drop(didenok_instagram[didenok_instagram['блогер'] == ''].index, inplace=True)

In [42]:
didenok_instagram.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров reels / видеопоста,среднее кол-во лайков,среднее кол-во комментариев,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия из 3х сторис до 45 сек,стоимость reels / видеопост,статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
11,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,100,р.240 000,р.160 000,р.315 000,р.315 000,https://drive.google.com/drive/folders/1xNqiOI...
12,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...


> Добавим указание на название соц.сети в название столбцов,чтобы в дальнейшем правильно объединять и искать нужные столбцы, в том числе в боте

##### Функция для указания названия соц.сети в названии столбцов

In [43]:
def type_social_network(df: pd.DataFrame, social_network: str) -> pd.DataFrame:
    
    
    """
    Функция принимает DataFrame - df, в название столбцов которого мы добавим название соц сети social_network
    
    ПАРАМЕТРЫ:
    df - DataFrame, в котором делаем преобразования.
    social_network - название соц. сети, которое мы добавим в название столбцов
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (DataFrame) с обновлёнными названиями столбцов
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    type_social_network(didenok_instagram, 'Instagram')
    """
    
    bufer_lst = []
    
    for element in df.columns.tolist():
        bufer_lst.append(f'{social_network} ' + element)
        
    df.columns = bufer_lst
    
    return df

In [44]:
type_social_network(didenok_instagram, 'Instagram').head()

,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
8,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
9,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
10,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
11,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,100,р.240 000,р.160 000,р.315 000,р.315 000,https://drive.google.com/drive/folders/1xNqiOI...
12,амина тендерлибае,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...


### Подготовка таблицы didenok_youtube

In [45]:
didenok_youtube.head()

7,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,стоимость за shorts до 1 мин,статистика,статистика ссылка
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,Статистика,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,Статистика,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,Статистика,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,Статистика,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,Статистика,https://drive.google.com/drive/folders/1VTEv65...


> Переименуем столбцы

In [46]:
[col for col in didenok_youtube.columns]

['канал',
 'ссылка',
 'категория, тематика канала',
 'количество подписчиков',
 'среднее кол-во просмотров',
 'среднее кол-во просмотров shorts',
 'среднее кол-во лайков',
 'среднее кол-во комментариев',
 'стоимость за интеграцию до 1 мин',
 'стоимость за продакт плейсмент',
 'стоимость за преролл до 30 сек',
 'стоимость за мидролл до 30 сек',
 'стоимость за shorts до 1 мин',
 'статистика',
 'статистика ссылка']

> У нас есть столбцы с возрастом и полу в процентном соотношении - объединим их в единый столбец.  
> <i>[Данные столбцы то появляются, то исчезают, поэтому на будущее оставим код]</i>

In [47]:
try:
    bufer_columns = ['блогер',
                     'ссылка',
                     'тематика',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'среднее кол-во просмотров shorts',
                     'среднее кол-во лайков',
                     'среднее кол-во комментариев',
                     'стоимость интеграция 1 слот',
                     'стоимость продакт плейсмент',
                     'стоимость преролл до 30 сек',
                     'стоимость мидролл до 30 сек',
                     'стоимость shorts до 1 мин',
                     'статистика удалить',
                     'м',
                     'ж',
                     '13-17',
                     '18-24',
                     '25-34',
                     '35-44',
                     '45-54',
                     '55+',
                     'ца гео рф',
                     'статистика ссылка']
    
    didenok_youtube.columns = bufer_columns
    
    didenok_youtube['ца по возрасту'] = didenok_youtube.apply(lambda row:
                                                              f"13-17 - {row['13-17']}, " +
                                                              f"18-24 - {row['18-24']}, " +
                                                              f"25-34 - {row['25-34']}, " +
                                                              f"35-44 - {row['35-44']}, " +
                                                              f"45-54 - {row['45-54']}" +
                                                              f"55+ - {row['55+']}", axis=1)
    
    didenok_youtube.drop(columns=['13-17',
                              '18-24',
                              '25-34',
                              '35-44',
                              '45-54',
                              '55+'], axis=1, inplace=True)
    didenok_youtube['ца по полу'] = didenok_youtube.apply(lambda row:
    f"Мужской - {row['м']}, " +
    f"Женской - {row['ж']}", axis=1)
    didenok_youtube.drop(columns = ['м', 'ж'], axis=1, inplace=True)
    
    
except:
    bufer_columns = ['блогер',
                     'ссылка',
                     'тематика',
                     'количество подписчиков',
                     'среднее кол-во просмотров видео',
                     'среднее кол-во просмотров shorts',
                     'среднее кол-во лайков',
                     'среднее кол-во комментариев',
                     'стоимость интеграция 1 слот',
                     'стоимость продакт плейсмент',
                     'стоимость преролл до 30 сек',
                     'стоимость мидролл до 30 сек',
                     'стоимость shorts до 1 мин',
                     'статистика удалить',
                     'статистика ссылка']
    
    didenok_youtube.columns = bufer_columns

In [48]:
didenok_youtube.drop(columns = ['статистика удалить'], axis=1, inplace=True)
didenok_youtube.rename(columns = {'статистика ссылка':'статистика'}, inplace=True)

In [49]:
didenok_youtube.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,https://drive.google.com/drive/folders/1VTEv65...


> Удалим пустые строки

In [50]:
didenok_youtube.drop(didenok_youtube[didenok_youtube['блогер'] == ''].index, inplace=True)

In [51]:
didenok_youtube.head()

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
8,влад бумага а4,https://www.youtube.com/@A4a4a4a4,юмор/челленджи/развлечения,47 100 000,8 000 000,-,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...
9,влад глент,https://www.youtube.com/@GLENTYOUTUBE,юмор/челленджи/гейминг,16 300 000,1 800 000,-,70 000,4 000,р.1 250 000,-,р.690 000,р.690 000,-,https://drive.google.com/drive/folders/1N165vD...
10,watchme,https://www.youtube.com/@WatchMeOff,дети/подростки,11 900 000,90 000,-,1 000,-,р.200 000,р.145 000,р.125 000,р.125 000,-,https://yadi.sk/d/BYF3WbphVjtT-Q
11,vikishow,https://www.youtube.com/@VikiShow,дети/подростки,11 200 000,1 500 000,200 000,50 000,4 000,р.700 000,р.440 000,р.440 000,р.350 000,р.440 000,https://drive.google.com/drive/folders/1_CHbok...
12,fixeye,https://www.youtube.com/@FixEye,гейминг,10 300 000,400 000,-,30 000,500,р.350 000,р.240 000,р.240 000,р.210 000,-,https://drive.google.com/drive/folders/1VTEv65...


> У нас есть проблема в том, что часть по одному и тому же блогеру выведены в отдельные строки как 1 и 2 слот

In [52]:
didenok_youtube.loc[(didenok_youtube['блогер'].str.contains('1 слот')) | (didenok_youtube['блогер'].str.contains('2 слот'))]

,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика
14,катя адушкина (1 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.640 000,р.400 000,р.440 000,р.370 000,р.250 000,https://drive.google.com/drive/folders/1aSmonK...
15,катя адушкина (2 слот),https://www.youtube.com/@KatyaAdushkinaa,лайфстайл,5 390 000,500 000,4 000,20 000,500,р.490 000,-,-,-,-,https://drive.google.com/drive/folders/1aSmonK...
16,supercrastan (1 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.980 000,-,р.735 000,р.615 000,р.735 000,https://drive.google.com/drive/folders/1dVEwSt...
17,supercrastan (2 слот),https://www.youtube.com/user/SuperCrastan,научпоп/DIY/шоу,3 430 000,750 000,40 000,7 000,500,р.860 000,-,-,-,-,https://drive.google.com/drive/folders/1dVEwSt...
21,контакты (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,450 000,-,20 000,1 000,р.950 000,-,р.500 000,р.400 000,-,https://drive.google.com/drive/u/1/folders/1dz...
22,контакты (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,450 000,-,20 000,1 000,р.800 000,-,р.500 000,р.400 000,-,NaN
23,цитаты (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,100 000,-,7 000,500,р.650 000,-,р.400 000,р.300 000,-,NaN
24,цитаты (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,100 000,-,7 000,500,р.500 000,-,р.400 000,р.300 000,-,NaN
25,не игры (1 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,500 000,-,30 000,1 000,р.950 000,-,р.500 000,р.400 000,-,NaN
26,не игры (2 слот),https://www.youtube.com/@shastoon.channel,шоу/юмор/развлечения,2 560 000,500 000,-,30 000,1 000,р.800 000,-,р.500 000,р.400 000,-,NaN


> Исправим данную проблему, переведя данные цены в отдельный столбец и объединив строки

In [53]:
def new_position_in_columns(df: pd.DataFrame,
                            columns_with_name: str,
                            columns_with_position: str) -> pd.DataFrame:
    
    """
    ОПИСАНИЕ РАБОТЫ:
        Функция выполняет определенные преобразования в DataFrame, связанные с именами блогеров,
        их позициями и стоимостью за интеграцию, а именно объединяем 1 и 2 слот
        по одному и тому же блогеры в единую строку

    ПАРАМЕТРЫ:
        - df (pd.DataFrame): DataFrame, в котором выполняются преобразования.
        - columns_with_name (str): Название столбца, содержащего имена блогеров.
        - columns_with_position (str): Название столбца, содержащего информацию о позиции блогера и стоимости за интеграцию.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
        - pd.DataFrame: Измененный DataFrame после выполнения преобразований.

    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
        didenok_youtube = new_position_in_columns(didenok_youtube, 'блогер', 'стоимость за интеграцию до 1 мин')
    """
    
    df[['блогер upd', 'слот']] = df[columns_with_name].str.extract(r'(.+)\s?\((\d+ слот)?\)')
    df['блогер upd'] = df['блогер upd'].fillna(df[columns_with_name])
    
    rows_to_delete = []

# Итерируемся по строкам DataFrame
    for index, row in df.iterrows():
        if '2 слот' in row['блогер']:
            blogger_name = row['блогер'].split(' (')[0]
        # Проверяем, есть ли строка с '1 слот' для этого блогера
            if any(df[(df['блогер'] != row['блогер']) & (df['блогер'].str.contains(blogger_name))]['блогер'].str.contains('1 слот')):
            # Находим индексы строк с совпадающим именем блогера
                matching_indices = df[df['блогер'].str.contains(blogger_name)].index
            # Обновляем данные в столбце 'стоимость за интеграцию 2 слот до 1 мин'
                df.loc[matching_indices, f'{columns_with_position} 2 слот'] = row[columns_with_position]
            # Добавляем индекс строки для удаления
                rows_to_delete.append(index)

# Удаляем строки с 2 слотом
    df.drop(rows_to_delete, inplace=True)
    
    df.drop(columns=['слот', 'блогер'], inplace=True)

# Переименовываем столбец 'блогер upd' в 'блогер'
    df.rename(columns={'блогер upd': 'блогер'}, inplace=True)

# Отчисти имена блогеров от лишних пробелов по бокам имени
    df['блогер'] = df['блогер'].str.strip()

    df.reset_index(drop=True, inplace=True)
    
    return df

In [54]:
didenok_youtube = new_position_in_columns(didenok_youtube, 'блогер', 'стоимость интеграция 1 слот')

In [55]:
# Проверим результат

didenok_youtube.loc[(didenok_youtube['блогер'].str.contains('1 слот')) | (didenok_youtube['блогер'].str.contains('2 слот'))]

,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,среднее кол-во лайков,среднее кол-во комментариев,стоимость интеграция 1 слот,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,стоимость shorts до 1 мин,статистика,блогер,стоимость интеграция 1 слот 2 слот


In [56]:
didenok_youtube.rename(columns = {'стоимость интеграция 1 слот 2 слот':'стоимость интеграция 2 слот'
                                  }, inplace=True)

### Нулевое объединение

> Сейчас мы объединим не два DataFrame из источников выше, а сделаем первый DataFrame, который станет основой для будущих объединений.  
> Перед объединением исправим неточность в наименовании блогера.

In [57]:
didenok_instagram.loc\
    [didenok_instagram['Instagram блогер'] == 'амина тендерлибае', 'Instagram блогер']  = 'амина tenderlybae'

didenok_instagram.loc\
    [didenok_instagram['Instagram блогер'] == 'helenavi', 'Instagram блогер']  = 'helenavi и gopronin'

> У нас есть два столбца, которые нам отдельно важны - это Никнейм блогера и его Тематика. Создадим единый DataFrame с такими столбцами, которые будем заполнять по мере объединения столбцов

In [58]:
bufer = pd.DataFrame(columns=['блогер','тематика'])
bufer

,блогер,тематика


> Объединим два DataFrame по вертикали с помощью метода concat (добавляет строки из didenok_instagram в bufer, пропуски в bufer в дальнейшем заполним)

In [59]:
bufer = pd.concat([bufer, didenok_instagram], ignore_index=True)

In [60]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
0,NaN,NaN,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
1,NaN,NaN,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
2,NaN,NaN,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
3,NaN,NaN,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,100,р.240 000,р.160 000,р.315 000,р.315 000,https://drive.google.com/drive/folders/1xNqiOI...
4,NaN,NaN,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...


> Те пропуски в данных двух столбцах, что мы имеем сейчас, мы будем заполнять с помощью похожих столбцов из присоединённых датафреймов.  
  
> Сделаем для этого отдельную функцию

#### Функция, с помощью которой мы заполним Имя и Тематику блогера в едином DataFrame

In [61]:
def bufer_name_category(bufer: pd.DataFrame, social_network: str) -> pd.DataFrame:
    
    """
    Функция принимает DataFrame - bufer, столбцы 'блогер' и 'тематика' которого мы заполняем с помощью данных из новых колонок
    '{social_network} блогер' и '{social_network} тематика'.
    Учитываем, что столбец 'тематика' не всегда есть в присоединяемом столбце

    
    ПАРАМЕТРЫ:
    bufer (DataFrame) - единый DataFrame, к которому мы присоединяем другие DataFrame
    social_network (str) - название соц сети, по которому мы будем искать вновь присоединённые колонки
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    DataFrame - bufer, в котором мы заполнили столбцы 'блогер' и 'тематика'
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    bufer = bufer_name_category(bufer, 'Instagram')
    """
    
    bufer['блогер'] = bufer['блогер'].fillna(bufer[f'{social_network} блогер'])
    
    # Учтём, что столбец 'тематика' не всегда есть в присоединяемом столбце

    if f'{social_network} тематика' in bufer.columns:
        bufer['тематика'] = bufer['тематика'].fillna(bufer[f'{social_network} тематика'])
    
    return bufer

In [62]:
bufer = bufer_name_category(bufer, 'Instagram')

In [63]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,Instagram среднее кол-во комментариев,Instagram стоимость фотопост,Instagram стоимость 1 сторис до 15 сек,Instagram стоимость серия из 3х сторис до 45 сек,Instagram стоимость reels / видеопост,Instagram статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,100,р.375 000,р.335 000,р.530 000,р.820 000,https://drive.google.com/drive/folders/1J26W2R...
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,700,р.875 000,р.440 000,р.625 000,р.1 875 000,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,100,р.445 000,р.305 000,р.625 000,р.685 000,https://drive.google.com/drive/u/1/folders/11S...
3,ваша маруся,лайфстайл/музыка,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,100,р.240 000,р.160 000,р.315 000,р.315 000,https://drive.google.com/drive/folders/1xNqiOI...
4,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,150,р.695 000,р.265 000,р.625 000,р.835 000,https://drive.google.com/drive/u/1/folders/1M3...


### Первое объединение

#### Функция для объединения DataFrame

> Так как мы будем объединять DataFrame, в которых как могут быть общие столбцы, так и нет, то для корректной работы merge мы напишем специальную функцию, которая будет как указывать в названии новых DataFrame название соц. сети, объединять по общим столбцам, а если их нет, то по имени блогеров. Также отдельно мы будем выводить количество дубликатов в объединённом DataFrame

In [64]:
def merge_bufer_df(bufer: pd.DataFrame,
                   df: pd.DataFrame,
                   social_network: str) -> pd.DataFrame:
    
    """
    Функция добавляет название соц сети в название столбцов с помощью ключа social_network,
    далее делает объединение по двум сценариям:
    1) У нас есть общие столбцы с учётом добавления названия соц сети - тогда объединяет по ним с помощью merge.
    2) У нас нет общих столбов - тогда объединяет по имени блогера из bufer['блогер'] и нового DataFrame.
    
    Далее функция заполняет столбцы 'блогер' и 'тематика' в едином DataFrame с учётом того,
    что 'тематика' не всегда есть в присоединяемом DataFrame.
    
    ПАРАМЕТРЫ:
    - bufer (DataFrame): наш единый DataFrame, к которому мы присоединяем новый DataFrame (df)
    - df (DataFrame): DataFrame, который мы будем присоединять
    - social_network (str): Ключ-название соц сети, по которому мы будем искать столбцы с именем и тематикой блогеров, кототорые присоединили

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    merged_bufer: единый DataFrame, в который мы объединили два DataFrame
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    bufer = merge_bufer_df(bufer, didenok_youtube, 'YouTube')

    """
    
    list_columns_no_social_network = []
    
    for element in df.columns.tolist():
        list_columns_no_social_network.append(f'{social_network} ' + element)
        
    df.columns = list_columns_no_social_network
    
    common_columns = bufer.columns.intersection(df.columns)
    
# В Pandas нельзя использовать if для проверки истинности объекта Index, поскольку он считается неоднозначным.
# Вместо этого мы будем использовать if not common_columns.empty: для проверки, является ли common_columns пустым или нет.
    
    if not common_columns.empty:
        merged_bufer = bufer.merge(df,
                                   left_on = common_columns.tolist(),
                                   right_on = common_columns.tolist(),
                                   how = 'outer')
    
    else:
        merged_bufer = bufer.merge(df, left_on='блогер', right_on=f'{social_network} блогер', how='outer')

    
    merged_bufer['блогер'] = merged_bufer['блогер'].fillna(merged_bufer[f'{social_network} блогер'])

# Учтём, что столбец 'тематика' не всегда есть в присоединяемом столбце
    if f'{social_network} тематика' in merged_bufer.columns:
        merged_bufer['тематика'] = merged_bufer['тематика'].fillna(merged_bufer[f'{social_network} тематика'])
        
        
    duplicates_merged_bufer = merged_bufer[merged_bufer.duplicated('блогер', keep=False)]

    # Чтобы посчитать количество дубликатов, можно использовать len() или метод .shape
    count_duplicates = len(duplicates_merged_bufer)  # или duplicates.shape[0]

    print("Количество дубликатов:", count_duplicates)
    
    return merged_bufer

In [65]:
bufer = merge_bufer_df(bufer, didenok_youtube, 'YouTube')

Количество дубликатов: 0


In [66]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube среднее кол-во лайков,YouTube среднее кол-во комментариев,YouTube стоимость интеграция 1 слот,YouTube стоимость продакт плейсмент,YouTube стоимость преролл до 30 сек,YouTube стоимость мидролл до 30 сек,YouTube стоимость shorts до 1 мин,YouTube статистика,YouTube блогер,YouTube стоимость интеграция 2 слот
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,60 000,1 000,р.1 330 000,-,-,-,-,https://drive.google.com/drive/folders/1LU4q-1...,катя клэп,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,350 000,30 000,р.4 375 000,-,р.1 625 000,р.1 625 000,-,https://drive.google.com/drive/u/1/folders/1Z-...,влад бумага а4,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,20 000,500,р.640 000,р.400 000,р.440 000,р.370 000,р.250 000,https://drive.google.com/drive/folders/1aSmonK...,катя адушкина,р.490 000
3,ваша маруся,лайфстайл/музыка,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,...,20 000,500,р.240 000,р.160 000,р.120 000,р.115 000,р.115 000,https://drive.google.com/drive/folders/1o0bh4e...,ваша маруся,NaN
4,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,26 000,500,р.560 000,р.280 000,р.250 000,р.225 000,-,https://drive.google.com/drive/folders/13YIWAD...,амина tenderlybae,NaN


> На данный момент мы не будем удалять дублирующие столбцы с именем и категорией блогеров из просоединённых столбцов, так как они нам понадобятся в дальнейшем для бота и для исследования

### Подготовка таблицы didenok_telegram

In [67]:
didenok_telegram.head()

7,блогер,ссылка,"категория, тематика блогера",количество подписчиков,среднее кол-во просмотров поста,среднее кол-во реакций и комментариев,стоимость за фотопост + текст,стоимость за видеопост + текст,стоимость за кружочек до 60 сек,стоимость за текстовое сообщение без фото,статистика,статистика ссылка
8,влад бумага а4,https://t.me/A4omg,лайфстайл/гейминг,753 000,500 000,100 000,р.1 000 000,р.1 500 000,р.875 000,р.625 000,Статистика,https://tgstat.ru/channel/@A4omg/stat
9,влад глент,https://t.me/vladglenttg,техно/лайфстайл,515 000,270 000,15 000,р.440 000,р.565 000,р.565 000,р.440 000,Статистика,https://tgstat.ru/channel/@vladglenttg/stat
10,антон шастун,https://t.me/shastoonchannel,лайфстайл/юмор,273 000,200 000,15 000,р.645 000,р.645 000,р.500 000,-,Статистика,https://drive.google.com/drive/folders/14xxWvL...
11,амина тендерлибае,https://t.me/plsbetenderly,твич/лайфстайл,256 000,80 000,3 000,р.240 000,р.390 000,р.310 000,р.170 000,Статистика,https://drive.google.com/drive/folders/1Gg5LC6...
12,катя клэп,https://t.me/KateClappTelegram,лайфстайл/бьюти,151 000,95 000,5 000,р.285 000,р.650 000,р.395 000,-,-,NaN


In [68]:
didenok_telegram.drop(columns=['статистика'], axis=1, inplace=True)

In [69]:
didenok_telegram.columns

Index(['блогер', 'ссылка', 'категория, тематика блогера',
       'количество подписчиков', 'среднее кол-во просмотров поста',
       'среднее кол-во реакций и комментариев',
       'стоимость за фотопост + текст', 'стоимость за видеопост + текст',
       'стоимость за кружочек до 60 сек',
       'стоимость за текстовое сообщение без фото', 'статистика ссылка'],
      dtype='object', name=7)

In [70]:
didenok_telegram.rename(columns = {'категория, тематика блогера':'тематика',
                                   'статистика ссылка':'статистика',
                                   'среднее кол-во реакций и комментариев':'средняя вовлечённость',
                                   'стоимость за фотопост + текст':'стоимость фотопост',
                                   'стоимость за видеопост + текст':'стоимость видеопост',
                                   'стоимость за кружочек до 60 сек':'стоимость кружочек до 60 сек',
                                   'стоимость за текстовое сообщение без фото':'стоимость текстовое сообщение без фото'
                                  }, inplace=True)

### Второе объединение

> Исправим неправильное наименование блогеров

In [71]:
didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'амина тендерлибае', 'блогер'] = 'амина tenderlybae'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'володя ххл', 'блогер'] = 'володя xxl'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'маша омг', 'блогер'] = 'мария омг'

didenok_telegram.loc\
    [didenok_telegram['блогер'] == 'илья бёрн', 'блогер'] = 'илья берн'

> Объединяем

In [72]:
bufer = merge_bufer_df(bufer, didenok_telegram, 'Telegram')

Количество дубликатов: 0


In [73]:
bufer.tail()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Telegram ссылка,Telegram тематика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram средняя вовлечённость,Telegram стоимость фотопост,Telegram стоимость видеопост,Telegram стоимость кружочек до 60 сек,Telegram стоимость текстовое сообщение без фото,Telegram статистика
71,позавчерашние новости,новости/интервью,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,чай с жасмин,религия/психология,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,dimixer live,новости/события,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://t.me/notmasha1,лайфстайл/бьюти,37 000,16 000,500,р.35 000,р.50 000,р.35 000,р.20 000,https://tgstat.ru/channel/@notmasha1/stat


### Подготовка таблицы didenok_vk_zen_show

> Мы видим, что помимо рекламы в VK, в таблице также есть строки с рекламой в Дзен Шоу.  
> Сделаем отдельный датафрейм для них, а из основного датафрейма удалим их.

In [74]:
didenok_vk_zen_show.head()

6,канал,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость за интеграцию до 1 мин,стоимость за продакт плейсмент,стоимость за преролл до 30 сек,стоимость за мидролл до 30 сек,статистика,статистика ссылка
7,громкий вопрос (1 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,Статистика,https://drive.google.com/drive/u/1/folders/1nx...
8,громкий вопрос (2 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 600 000,NaN,NaN,NaN,NaN,NaN
9,шоу истории (1 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,NaN,NaN
10,шоу истории (2 слот),https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 200 000,NaN,NaN,NaN,NaN,NaN
11,что делаем? куда идём? (1 слот),https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,NaN,NaN


> Заполним столбец "Статистика"

In [75]:
didenok_vk_zen_show[['статистика ссылка']] = \
            didenok_vk_zen_show[['статистика ссылка']].fillna(method='ffill')

In [76]:
didenok_vk_zen_show.drop(columns = ['статистика'], axis=1, inplace=True)
didenok_vk_zen_show.rename(columns = {'статистика ссылка':'статистика'}, inplace=True)

In [77]:
didenok_vk_zen_show.rename(columns = {'канал':'блогер',
                                      'категория, тематика блогера':'тематика',
                                      'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
                                      'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
                                      'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
                                      'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
                                     }, inplace=True)

> Переведём позиции со строк "2 слот" в отдельный столбец

In [78]:
didenok_vk_zen_show = new_position_in_columns(didenok_vk_zen_show, 'блогер', 'стоимость интеграция до 1 мин')
didenok_vk_zen_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
                                  }, inplace = True)

> Используем функцию .str.contains() для поиска подстроки в строках столбца DataFrame, а именно 'https://dzen.ru' в ссылке

In [79]:
# didenok_zen_show = didenok_vk_zen_show[didenok_vk_zen_show['ссылка'].str.contains('https://dzen.ru', na=False)]

In [80]:
# didenok_zen_show

> Используем ту же логику, что и выше, но теперь исключим строки, где в ссылке встречается 'https://dzen.ru', то есть оставим только 'https://vk.com/'

In [81]:
#Инверсия условия ~ здесь означает "не содержит", то есть выбираются строки,
#которые не содержат 'https://dzen.ru' в столбце 'ссылка'.
#Таким образом, didenok_vk_show будет содержать строки, которые не соответствуют заданному условию.

# didenok_vk_show = didenok_vk_zen_show[~didenok_vk_zen_show['ссылка'].str.contains('https://dzen.ru', na=False)]

> Переименуем столбцы, чтобы подвести их под единый принятый формат

In [82]:
# didenok_vk_show.columns

In [83]:
# didenok_vk_show.rename(columns = {'канал':'блогер',
#                                   'категория, тематика блогера':'тематика',
#                                   'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
#                                   'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
#                                   'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
#                                   'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
#                                  }, inplace = True)

# didenok_zen_show.rename(columns = {'канал':'блогер',
#                                   'категория, тематика блогера':'тематика',
#                                   'стоимость за интеграцию до 1 мин':'стоимость интеграция до 1 мин',
#                                   'стоимость за продакт плейсмент':'стоимость продакт плейсмент',
#                                   'стоимость за преролл до 30 сек':'стоимость преролл до 30 сек',
#                                   'стоимость за мидролл до 30 сек':'стоимость мидролл до 30 сек'
#                                  }, inplace = True)

In [84]:
# didenok_zen_show = new_position_in_columns(didenok_zen_show, 'блогер', 'стоимость интеграция до 1 мин')

In [85]:
# didenok_zen_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
#                                   }, inplace = True)

In [86]:
# Проверим, что получилось
# didenok_zen_show.head()

In [87]:
# didenok_vk_show = new_position_in_columns(didenok_vk_show, 'блогер', 'стоимость интеграция до 1 мин')

In [88]:
# didenok_vk_show.rename(columns = {'стоимость интеграция до 1 мин 2 слот':'стоимость интеграция 2 слот'
#                                   }, inplace = True)


In [89]:
#Проверим, что получилось
didenok_vk_zen_show.head()

6,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость интеграция до 1 мин,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,статистика,блогер,стоимость интеграция 2 слот
0,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1nx...,громкий вопрос,р.1 600 000
1,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,https://drive.google.com/drive/u/1/folders/1nx...,шоу истории,р.1 200 000
2,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,NaN
3,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.2 700 000,NaN,NaN,NaN,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,NaN
4,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000


> У нас получилась интересная ситуация: у блогера "что делаем? куда идём?" не сработала функция.  
> Сделаем проверку:

In [90]:
string1 = didenok_vk_zen_show.loc[2, 'блогер']
string2 = didenok_vk_zen_show.loc[3, 'блогер']

print(repr(string1))
print(repr(string2))

'что делаем? куда идём?'
'что делаем? куда идём?'


In [91]:
# Затем вы можете сравнить их
if repr(string1) == repr(string2):
    print("Строки идентичны")
else:
    print("Строки различаются")

Строки идентичны


> Объединим данные строки:

In [92]:
indexes_to_remember = []

# Проверяем условие и сохраняем индексы в список
for index, row in didenok_vk_zen_show.iterrows():
    if row['блогер'] == 'что делаем? куда идём?':
        indexes_to_remember.append(index)

# Получаем последний и первый индексы
last_index = indexes_to_remember[-1]
first_index = indexes_to_remember[0]

# Заменяем значение в столбце 'стоимость за интеграцию до 1 мин 2 слот' в строке с первым индексом
didenok_vk_zen_show.at[first_index, 'стоимость интеграция 2 слот'] = \
            didenok_vk_zen_show.at[last_index, 'стоимость интеграция до 1 мин']

didenok_vk_zen_show = didenok_vk_zen_show.drop(last_index)

In [93]:
didenok_vk_zen_show.head()

6,ссылка,"категория, тематика канала",количество подписчиков,среднее кол-во просмотров ролика,стоимость интеграция до 1 мин,стоимость продакт плейсмент,стоимость преролл до 30 сек,стоимость мидролл до 30 сек,статистика,блогер,стоимость интеграция 2 слот
0,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1nx...,громкий вопрос,р.1 600 000
1,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 200 000,р.1 500 000,р.850 000,р.850 000,р.650 000,https://drive.google.com/drive/u/1/folders/1nx...,шоу истории,р.1 200 000
2,https://vk.com/improcom,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,р.2 700 000
4,https://vk.com/improcom,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000
5,https://vk.com/shastoon.channel,шоу/юмор/развлечения,358 000,1 600 000,р.1 900 000,р.1 000 000,р.1 000 000,р.800 000,https://drive.google.com/drive/u/1/folders/1US...,контакты,р.1 600 000


### Третье объединение

In [94]:
# didenok_zen_show.rename(columns = {'категория, тематика канала':'тематика'}, inplace=True)

In [95]:
# bufer = merge_bufer_df(bufer, didenok_zen_show, 'Дзен Шоу')

In [96]:
# bufer.tail()

### Четвёртое объединение

In [97]:
didenok_vk_zen_show.rename(columns = {'категория, тематика канала':'тематика'}, inplace=True)

In [98]:
bufer = merge_bufer_df(bufer, didenok_vk_zen_show, 'VK Видео')

Количество дубликатов: 0


In [99]:
bufer.tail()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK Видео тематика,VK Видео количество подписчиков,VK Видео среднее кол-во просмотров ролика,VK Видео стоимость интеграция до 1 мин,VK Видео стоимость продакт плейсмент,VK Видео стоимость преролл до 30 сек,VK Видео стоимость мидролл до 30 сек,VK Видео статистика,VK Видео блогер,VK Видео стоимость интеграция 2 слот
74,несусвет,интервью/шоу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,маша нарциссизм,лайфстайл/бьюти,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,что делаем? куда идём?,шоу/юмор/тревел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/тревел,460 000,3 000 000,р.3 000 000,р.2 500 000,р.1 700 000,р.1 300 000,https://drive.google.com/drive/u/1/folders/1nx...,что делаем? куда идём?,р.2 700 000
77,тейблтайм,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/развлечения,460 000,1 000 000,р.1 600 000,р.900 000,р.900 000,р.700 000,https://drive.google.com/drive/u/1/folders/1nx...,тейблтайм,р.1 300 000
78,jam,шоу/юмор/развлечения,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,шоу/юмор/развлечения,61 000,700 000,р.900 000,р.500 000,р.500 000,р.400 000,https://drive.google.com/drive/u/1/folders/1sk...,jam,р.700 000


### Подготовка таблицы didenok_vk_group

In [100]:
didenok_vk_group.head()

7,блогер,ссылка,"категория, тематика блогера",количество подписчиков,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,среднее кол-во лайков,среднее кол-во комментариев,стоимость за фотопост,стоимость за видеопост,стоимость за вк клип,статистика,статистика ссылка
8,влад бумага а4,https://vk.com/a4,лайфстайл/гейминг,1 540 000,200 000,250 000,6 000,180,р.750 000,-,р.1 625 000,обновляем,NaN
9,амина тендерлибае,https://vk.com/tenderlybae,твич/лайфстайл,739 000,250 000,80 000,10 000,50,р.210 000,р.300 000,р.300 000,Статистика,https://drive.google.com/drive/folders/12PNvkW...
10,катя адушкина,https://vk.com/katyaadushkina,музыка/лайфстайл/танцы,1 000 000,250 000,150 000,4 000,10,р.155 000,р.185 000,р.185 000,Статистика,https://drive.google.com/drive/folders/1MnkkeU...
11,vikishow,https://vk.com/vikishow,лайфстайл/дети/подростки,478 000,70 000,50 000,2 000,20,р.70 000,р.115 000,р.115 000,Статистика,https://drive.google.com/drive/folders/14MQXd3...
12,ваша маруся,https://vk.com/vashamarousia_tut,лайфстайл/музыка,478 000,70 000,60 000,3 500,30,р.160 000,р.190 000,р.190 000,Статистика,https://drive.google.com/drive/folders/1Daes55...


In [101]:
didenok_vk_group.drop(columns=['статистика'], axis=1, inplace=True)

In [102]:
didenok_vk_group.columns

Index(['блогер', 'ссылка', 'категория, тематика блогера',
       'количество подписчиков', 'среднее кол-во просмотров поста',
       'среднее кол-во просмотров вк клипа', 'среднее кол-во лайков',
       'среднее кол-во комментариев', 'стоимость за фотопост',
       'стоимость за видеопост', 'стоимость за вк клип', 'статистика ссылка'],
      dtype='object', name=7)

In [103]:
didenok_vk_group.rename(columns = {'категория, тематика блогера':'тематика',
                                   'статистика ссылка':'статистика',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост',
                                   'стоимость за вк клип':'стоимость вк клип'
                                  }, inplace=True)

### Пятое объединение

> Исправим одну неточность в имени блогера

In [104]:
didenok_vk_group.loc\
    [didenok_vk_group['блогер'] == 'амина тендерлибае', 'блогер'] = 'амина tenderlybae'

In [105]:
bufer = merge_bufer_df(bufer, didenok_vk_group, 'VK группа')

Количество дубликатов: 0


In [106]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK группа тематика,VK группа количество подписчиков,VK группа среднее кол-во просмотров поста,VK группа среднее кол-во просмотров вк клипа,VK группа среднее кол-во лайков,VK группа среднее кол-во комментариев,VK группа стоимость фотопост,VK группа стоимость видеопост,VK группа стоимость вк клип,VK группа статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,лайфстайл/гейминг,1 540 000,200 000,250 000,6 000,180,р.750 000,-,р.1 625 000,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,музыка/лайфстайл/танцы,1 000 000,250 000,150 000,4 000,10,р.155 000,р.185 000,р.185 000,https://drive.google.com/drive/folders/1MnkkeU...
3,ваша маруся,лайфстайл/музыка,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,...,лайфстайл/музыка,478 000,70 000,60 000,3 500,30,р.160 000,р.190 000,р.190 000,https://drive.google.com/drive/folders/1Daes55...
4,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,твич/лайфстайл,739 000,250 000,80 000,10 000,50,р.210 000,р.300 000,р.300 000,https://drive.google.com/drive/folders/12PNvkW...


### Подготовка таблицы didenok_vk_ls

In [107]:
# didenok_vk_ls
# didenok_vk_ls.drop(columns=['статистика'], axis=1, inplace= True)
# didenok_vk_ls.rename(columns = {'категория, тематика блогера':'тематика',
#                                'статистика ссылка':'статистика'}, inplace = True)

# ### Шестое объединение
# bufer = merge_bufer_df(bufer, didenok_vk_ls, 'VK личная страница')
# bufer.tail()

### Подготовка таблицы didenok_twitch

In [108]:
didenok_twitch.head()

7,блогер,сcылка,категория,количество подписчиков,среднее кол-во зрителей,"уникальные зрители , (за 30 дней)",статистика,баннер + ссылка в описании 1 мес,оверлей использование логотипа товара/бренда в процессе трансляции,нативное голосовое упоминание рекламируемого товара в процессе трансляции,ссылка в чат бот 1 стрим,демонстрация-обзор товара на трансляции,статистика ссылка
8,амина tenderlybae,twitch.tv/tenderlybae,Just Chatting / IRL / Multigaming,1 300 000,6 500,546 000,Статистика,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA


In [109]:
didenok_twitch.drop(columns=['статистика'], axis=1, inplace=True)

In [110]:
didenok_twitch.rename(columns = {'сcылка':'ссылка',
                                 'уникальные зрители , (за 30 дней)':'уникальные зрители (30 дней)',
                                 'баннер + ссылка в описании 1 мес':'стоимость за баннер + ссылка в описании 1 мес',
                                 'оверлей использование логотипа товара/бренда в процессе трансляции':'стоимость оверлей логотипа товара/бренда в процессе трансляции',
                                 'нативное голосовое упоминание рекламируемого товара в процессе трансляции':'стоимость нативное голосовое упоминание товара в трансляции',
                                 'ссылка в чат бот 1 стрим ':'стоимость ссылка в чат-боте на 1 стрим',
                                 'демонстрация-обзор товара на трансляции':'стоимость демонстрация/обзор товара на трансляции',
                                 'статистика':'статистика',
                                 'категория':'тематика',
                                 'статистика ссылка':'статистика'
                                }, inplace=True)

### Седьмое объединение

In [111]:
bufer = merge_bufer_df(bufer, didenok_twitch, 'Twitch')

Количество дубликатов: 0


In [112]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Twitch тематика,Twitch количество подписчиков,Twitch среднее кол-во зрителей,Twitch уникальные зрители (30 дней),Twitch стоимость за баннер + ссылка в описании 1 мес,Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции,Twitch стоимость нативное голосовое упоминание товара в трансляции,Twitch стоимость ссылка в чат-боте на 1 стрим,Twitch стоимость демонстрация/обзор товара на трансляции,Twitch статистика
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ваша маруся,лайфстайл/музыка,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,Just Chatting / IRL / Multigaming,1 300 000,6 500,546 000,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA


> Мы объединили все таблицы DIDENOK, осталось добавить два столбца: налог и контакты менеджера

In [113]:
nalog = 'НДС (20%)'
manager = 'info@didenokteam.com'

In [114]:
bufer['налог'] = nalog
bufer['контакты менеджера'] = manager

In [115]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Twitch среднее кол-во зрителей,Twitch уникальные зрители (30 дней),Twitch стоимость за баннер + ссылка в описании 1 мес,Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции,Twitch стоимость нативное голосовое упоминание товара в трансляции,Twitch стоимость ссылка в чат-боте на 1 стрим,Twitch стоимость демонстрация/обзор товара на трансляции,Twitch статистика,налог,контакты менеджера
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
3,ваша маруся,лайфстайл/музыка,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НДС (20%),info@didenokteam.com
4,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,6 500,546 000,р.950 000,р.240 000,р.160 000,р.160 000,р.715 000,https://disk.yandex.ru/d/UoxOK0U47TB6rA,НДС (20%),info@didenokteam.com


## Подготовка данных по блогерам от VK

In [116]:
vk_bloggers.head(7)

,Сообщество,Шоу,KPI (стартов просмотра на 1 эпизод на 30-е сутки после релиза),Стоимость интеграции 1-й слот до НДС,Стоимость интеграции 2-й слот до НДС,Продакт плейсмент (30 сек) до НДС,Стоимость рекламной заставки (пре-ролл до 20 сек) до НДС,Стоимость рекламной заставки (мид-ролл до 20 сек) до НДС,Статистика,Контакты менеджера по продажам,Ссылка,Статистика ссылка
1,Азамат Мусагалиев,Большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,Статистика,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,Это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,NaN,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,Я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,NaN,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,Шоу памяти Азамата Мусагалиева NEW,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,NaN,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,Пикабу,Чебатков читает Пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...
6,Женский форум,Женский форум,1 000 000,р.1 800 000,р.1 500 000,р.600 000,р.1 000 000,нет,Статистика,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/zhenforum,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%96...
7,Варвара Щербакова по вашим интересам!,В гостях у Вари,1 000 000,р.1 500 000,р.1 200 000,р.500 000,р.850 000,нет,Статистика,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/varjauletela_group,https://drive.google.com/drive/u/1/folders/1Q1...


> Сделаем предобработку DataFrame

In [117]:
key_word = [] # Список ключевых слов для поиска столбцов и их удаления
drop_word = ' до НДС' # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Сообщество', 'Шоу'] # Список столбцов с именами блогеров для приведения к единому формату

In [118]:
vk_bloggers = pretraining_df(vk_bloggers, key_word, drop_word, upd_columns)

> Удалим сразу не нужный нам столбец "Статистика"

In [119]:
vk_bloggers.columns

Index(['сообщество', 'шоу',
       'kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)',
       'стоимость интеграции 1-й слот', 'стоимость интеграции 2-й слот',
       'продакт плейсмент (30 сек)',
       'стоимость рекламной заставки (пре-ролл до 20 сек)',
       'стоимость рекламной заставки (мид-ролл до 20 сек)', 'статистика',
       'контакты менеджера по продажам', 'ссылка', 'статистика ссылка'],
      dtype='object', name=0)

In [120]:
vk_bloggers.drop(columns=['статистика'], axis=1, inplace=True)

vk_bloggers.rename(columns = {'статистика ссылка':'статистика',
                              'стоимость интеграции 1-й слот':'стоимость интеграция 1 слот',
                              'стоимость интеграции 2-й слот':'стоимость интеграция 2 слот',
                              'продакт плейсмент (30 сек)':'стоимость продакт плейсмент',
                              'стоимость рекламной заставки (пре-ролл до 20 сек)':'стоимость преролл до 20 сек',
                              'стоимость рекламной заставки (мид-ролл до 20 сек)':'стоимость мидролл до 20 сек'
                             }, inplace=True)

In [121]:
vk_bloggers.head()

,сообщество,шоу,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика
1,азамат мусагалиев,большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,NaN,это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",NaN,NaN
3,NaN,я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",NaN,NaN
4,NaN,шоу памяти азамата мусагалиева new,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",NaN,NaN
5,пикабу,чебатков читает пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


> Нам необходимо заполнить данные - так как при часть ячеек в исходной таблице объеденены, то правильное значение ставиться в верхнюю строку, а остальные становятся NaN, и мы заполним значения "сверху вниз". 
> Мы заполним столбцы 'Сообщество', 'Ссылка на канал ', 'Ссылка' и 'Шоу'.  
> Столбец 'Ссылка на канал' то появляется, то исчезает, так что учтём и его.

> К сожалению, мы не сможем сразу заполнить столбец 'Статистика ссылка' и 'ссылка', так как у некоторых каналов нет Статистики и ссылки, и если мы заполним сверху вниз, то у нас получится неверные ссылки - отдельно решим эту проблему.

> В функции pretraining_df мы используем методы .strip(), .lower() и .replace() непосредственно на значениях NaN, которые мы заполнили через .fillna(''). Если не заполнить пропуски, то Python пытается их выполнить, что может привести к неожиданным результатам <em>[В данном проекте, например, символ * из значения NaN был заменен на "8" методом .replace("*")]</em>.
Поэтому надо превратить строки обратно в NaN, чтобы сделать заполнение.

In [122]:
vk_bloggers['ссылка'] = vk_bloggers['ссылка'].replace('', np.nan)
vk_bloggers['контакты менеджера по продажам'] = vk_bloggers['контакты менеджера по продажам'].replace('', np.nan)

In [123]:
try:
    vk_bloggers[['сообщество', 'ссылка', 'шоу', 'контакты менеджера по продажам']] = \
            vk_bloggers[['сообщество', 'ссылка', 'шоу', 'контакты менеджера по продажам']].fillna(method='ffill')
    
except:
    vk_bloggers[['сообщество', 'шоу', 'контакты менеджера по продажам']] = \
            vk_bloggers[['сообщество', 'шоу', 'контакты менеджера по продажам']].fillna(method='ffill')

> Чтобы заполнить столбец 'статистика', сделаем групировку по столбцу 'сообщество', так как у них одинаковые каналы, а значит и одинаковая статистика, выберем только столбец 'статистика' для работы и заполним с помощью .transform('first'), но учтём, что в столбце "ссылка" у нас есть исключение

In [124]:
vk_bloggers['статистика'] = vk_bloggers.groupby('сообщество')['статистика'].transform('first')

In [125]:
vk_bloggers['ссылка'] = vk_bloggers.groupby('сообщество')['ссылка']\
                        .transform(lambda group: group.iloc[0] if 'ок video' not in group.name else group)

In [126]:
# Проверяем
vk_bloggers.head()

,сообщество,шоу,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика
1,азамат мусагалиев,большое шоу,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
2,азамат мусагалиев,это про меня,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
3,азамат мусагалиев,я себя знаю,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
4,азамат мусагалиев,шоу памяти азамата мусагалиева new,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...
5,пикабу,чебатков читает пикабу,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...


> В основной таблице bufer название блогера или его шоу указываются в едином столбце "блогер".  
> В таблице vk_bloggers имя блогера и его шоу разделены по разным столбцам.  
> Мы объединим их, чтобы в дальшейшем мы могли использовать поиск в боте

In [127]:
vk_bloggers['блогер'] = vk_bloggers['сообщество'] + ' - ' + vk_bloggers['шоу']
vk_bloggers.drop(columns=['сообщество', 'шоу'], axis=1 , inplace=True)

In [128]:
#Проверяем результат
vk_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
1,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - большое шоу
2,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - это про меня
3,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - я себя знаю
4,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - шоу памяти азамата мусагал...
5,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...,пикабу - чебатков читает пикабу


> Мы видим, что помимо рекламы в VK, в таблице также есть строки с рекламой в Одноклассниках.  
> Сделаем отдельный датафрейм для них, а из основного датафрейма удалим их.

In [129]:
ok_bloggers = vk_bloggers[vk_bloggers['блогер'].str.contains('ок video')]

In [130]:
ok_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
89,5 000 000,р.1 150 000,по запросу,нет,р.800 000,нет,"Анна Ипатова, TG: @Anna_ipp, a.ipatova@vk.team",https://vk.com/vkvideo,None,ок video - в мире домашних животных
90,5 000 000,р.700 000,по запросу,по запросу,нет,нет,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/vkvideo,None,"ок video - ок, мама, поехали!"
91,2 500 000,нет,нет,нет,р.500 000,нет,Виноградова Наталья natalia.vinogradova@vk.tea...,https://vk.com/vkvideo,None,ок video - психология кино
92,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,Виноградова Наталья natalia.vinogradova@vk.tea...,https://vk.com/vkvideo,None,ок video - полтора спортсмена
93,5 000 000,р.1 100 000,по запросу,р.800 000,р.500 000,нет,Виноградова Наталья natalia.vinogradova@vk.tea...,https://vk.com/vkvideo,None,ок video - шеф против любителя


In [131]:
# Проверим на дубликаты:

duplicates = ok_bloggers[ok_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер


> Сделаем DataFrame с блогерами из VK, удалив строки с блогерами из Одноклассников из исходного DataFrame

In [132]:
vk_bloggers = vk_bloggers[~vk_bloggers['блогер'].str.contains('ок video')]

In [133]:
vk_bloggers.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
1,2 600 000,р.3 500 000,р.3 200 000,р.2 000 000,р.3 000 000,р.2 800 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - большое шоу
2,600 000,р.700 000,по запросу,р.300 000,р.300 000,р.200 000,"Татьяна Бондарева, TG: @TataBondi, tatiana.bon...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - это про меня
3,2 000 000,р.2 800 000,р.2 500 000,по запросу,р.2 000 000,р.1 800 000,"Марина Ванюшкина, TG: @MarinaVan, m.vanyushkin...",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - я себя знаю
4,1 500 000,р.2 500 000,по запросу,по запросу,р.2 000 000,по запросу,"Магда Булавина, TG: @Portenato, m.bulavina@vk....",https://vk.com/azamatmusagaliev,https://disk.yandex.ru/d/_uwN3cl6EO2NAw/%D0%90...,азамат мусагалиев - шоу памяти азамата мусагал...
5,1 000 000,р.1 350 000,по запросу,по запросу,р.1 000 000,по запросу,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/pikabu,https://drive.google.com/drive/u/1/folders/1Aq...,пикабу - чебатков читает пикабу


In [134]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер
27,600 000,Интеграция с участием актеров: 350 000р,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей
28,NaN,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей
29,700 000,Интеграция с участием актеров: 600 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена
30,NaN,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена
32,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик
33,NaN,Интеграция с участием блогера: 1 550 000р,Интеграция с участием блогера: 1 350 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик
36,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня
37,NaN,Интеграция с участием блогера: 1 450 000р,Интеграция с участием блогера: 1 250 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - удиви меня
38,700 000,Интеграция с участием актеров: 650 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка
39,NaN,Интеграция с участием блогера: 1 250 000р,Интеграция с участием блогера: 1 150 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - сходка


> Как мы видим, часть ячеек заполнено как тестовыми, так и числовыми значениями "Интеграция с участием блогера: 650 000р", то есть указана и позиция (базово она у нас указана в столбце), и цены (базово они у нас указаны в строках). Нам надо перевести данные значения в отдельные столбцы, объединив похожие позиции

In [135]:
def custom_split_text(df: pd.DataFrame, column1, column2) -> pd.DataFrame:
    
    """
    Функция форматирует информацию по строкам, создавая новые столбцы на основе содержимого 
    ячеек в указанных столбцах. Основные шаги включают в себя извлечение числовых значений и
    текстовых описаний из указанных столбцов, их дальнейшую обработку и создание новых столбцов
    с отформатированными данными, где мы выделяем позицию, указанную в ячейках, и цену на данную позицию.
    В результате, функция предназначена для структурирования данных и улучшения их читаемости.
    
    
    ПАРАМЕТРЫ:
    - df (pd.DataFrame): Исходный DataFrame, содержащий данные.
    - column1 (str): Название первого столбца, из которого нужно извлечь информацию.
    - column2 (str): Название второго столбца, из которого нужно извлечь информацию.


    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    - filtered_df (pandas DataFrame): Новый DataFrame, содержащий обработанные данные, включая извлеченные числа и описания.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    filtered_df (pandas DataFrame): Новый DataFrame, содержащий обработанные данные,
    включая извлеченные числа и описания.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    filtered_df = custom_split_text(vk_bloggers, 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот')

    """
    
    duplicates_filter = df[df.duplicated('блогер', keep=False)]

    filtered_df = duplicates_filter[['блогер', 
                                     column1,
                                     column2]]    


    for index in filtered_df.index:
        text = filtered_df.at[index, column1]

        # Поиск чисел в тексте
        numbers = re.findall(r'\d[\d\s]+', text)

        if numbers:
            # Извлечение первого найденного числа
            number = re.sub(r'[^0-9\s]', '', numbers[0])
            # Извлечение описания (текста без чисел)
            description = re.sub(r'[0-9]', '', text).strip()
            # Сохраняем числа и описание
            filtered_df.at[index, 'Позиция 1'] = description            
            filtered_df.at[index, 'Цена 1'] = number
        else:
            # Если нет чисел, сохраняем текст в столбце 'Позиция 1' и пустую строку в столбце 'цена 1'
            filtered_df.at[index, 'Позиция 1'] = text
            filtered_df.at[index, 'Цена 1'] = ''
            
    for index in filtered_df.index:
        text = filtered_df.at[index, column2]

        # Поиск чисел в тексте
        numbers = re.findall(r'\d[\d\s]+', text)

        if numbers:
            # Извлечение первого найденного числа
            number = re.sub(r'[^0-9\s]', '', numbers[0])
            # Извлечение описания (текста без чисел)
            description = re.sub(r'[0-9]', '', text).strip()
            # Сохраняем числа и описание
            filtered_df.at[index, 'Цена 2'] = number
            filtered_df.at[index, 'Позиция 2'] = description
        else:
            # Если нет чисел, сохраняем текст в столбце 'Позиция 1' и пустую строку в столбце 'цена 1'
            filtered_df.at[index, 'Позиция 2'] = text
            filtered_df.at[index, 'Цена 2'] = ''            
            
    for i in range(1, 3):
            # Удаление "р" и символов ".:()" из столбцов 'Позиция'
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].str.replace(r'\bр\.?\b', '', regex=True)
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].str.replace(r'[.:()]', '', regex=True)
            # Замена пустых строк на NaN
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].replace('', np.nan)
        filtered_df[f'Позиция {i}'] = filtered_df[f'Позиция {i}'].replace('по запросу', np.nan)    

        filtered_df[f'Цена {i}'] = filtered_df[f'Цена {i}'].replace('', np.nan)    
    
    return filtered_df

> Выведем новые значения в отдельный DataFrame

In [136]:
filtered_df = custom_split_text(vk_bloggers, 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот')

In [137]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350 000,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550 000,Интеграция с участием блогера,450 000
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600 000,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750 000,Интеграция с участием блогера,650 000
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1 150 000,Интеграция с участием актеров,1 050 000


> Сохраним индексы на будущее

In [138]:
indexes_to_filtered_df = list(filtered_df.index)

In [139]:
unique_position = filtered_df['блогер'].str.strip().unique()
len(unique_position)

10

In [140]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350 000,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550 000,Интеграция с участием блогера,450 000
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600 000,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750 000,Интеграция с участием блогера,650 000
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1 150 000,Интеграция с участием актеров,1 050 000


In [141]:
unique_position = filtered_df['Позиция 1'].str.strip().unique()
unique_position

array(['Интеграция с участием актеров', 'Интеграция с участием блогера',
       'Интеграция с участием комиков из выпуска',
       'Интеграция с участием Сергея Орлова', nan,
       'Интеграция с участием ведущих', 'Пушной, Крастер, Старый',
       'Мамикс', 'интеграция до  сек', 'устник до  сек'], dtype=object)

In [142]:
filtered_df.columns

Index(['блогер', 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот',
       'Позиция 1', 'Цена 1', 'Позиция 2', 'Цена 2'],
      dtype='object', name=0)

> Отчистим столбец с ценами от букв и сторонних символов, оставив только цифры

In [143]:
filtered_df['Позиция 1'] = filtered_df['Позиция 1'].fillna('')
filtered_df['Цена 1'].fillna(0, inplace=True)
# Уберем неразрывные пробелы из значений столбца 'Цена 1'
filtered_df['Цена 1'] = filtered_df['Цена 1'].str.replace('\xa0', '')
filtered_df['Цена 1'] = filtered_df['Цена 1'].str.replace(' ', '')
# Теперь преобразуем значения в числа с плавающей точкой
filtered_df['Цена 1'] = filtered_df['Цена 1'].astype(float)


filtered_df['Позиция 2'] = filtered_df['Позиция 2'].fillna('')
filtered_df['Цена 2'].fillna(0, inplace=True)
# Уберем неразрывные пробелы из значений столбца 'Цена 1'
filtered_df['Цена 2'] = filtered_df['Цена 2'].str.replace('\xa0', '')
filtered_df['Цена 2'] = filtered_df['Цена 2'].str.replace(' ', '')
# Теперь преобразуем значения в числа с плавающей точкой
filtered_df['Цена 2'] = filtered_df['Цена 2'].astype(float)


filtered_df['Позиция 1'] = filtered_df['Позиция 1'].str.strip()
filtered_df['Позиция 2'] = filtered_df['Позиция 2'].str.strip()

In [144]:
filtered_df.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Позиция 1,Цена 1,Позиция 2,Цена 2
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,Интеграция с участием актеров,350000.0,,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,Интеграция с участием блогера,550000.0,Интеграция с участием блогера,450000.0
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,Интеграция с участием актеров,600000.0,,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,Интеграция с участием блогера,750000.0,Интеграция с участием блогера,650000.0
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,Интеграция с участием актеров,1150000.0,Интеграция с участием актеров,1050000.0


In [145]:
def get_column_values_filtered_df(df: pd.DataFrame,
                                  key: int,
                                  column_with_position: str,
                                  column_with_price: str,
                                  column_list: str) -> pd.DataFrame:
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
    df - DataFrame, в котором делаем преобразования;
    key - Ключ в виде int, который нужно добавить к названию, чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
    column_with_position - Столбец, в котором указаны позиции;
    column_with_price - Столбец, в котором указаны цены на позиции;
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
    Создаём список с уникальными значениями позиций, приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
    spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                               .str.lower()\
                                               .str.replace('/\n', ', ')\
                                               .str.replace('\n', ', ')\
                                               .str.replace('*', '')\
                                               .unique()
    
    #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
    #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
    spisok = spisok[~pd.isna(spisok)]
        
        
    # Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
    
    # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
        df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
        where(df[column_with_position] == unique_position[i], other='')
    
    
    Удаляем исходные столбцы, чтобы не было повтора:
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)
    
    
    Возвращаем обновлённый DataFrame
    return df    
                
    """
#     column_list = []

    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]
    
    for i in range(len(unique_position)):
        
        new_column_name = f'{unique_position[i]}_{key} слот'
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')

            
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [146]:
column_list = []

for key in range(1, 3):
    column_with_position = f'Позиция {key}'
    price_column = f'Цена {key}'
    filtered_df_new = get_column_values_filtered_df(filtered_df,
                                                    key,
                                                    column_with_position,
                                                    price_column,
                                                    column_list)

In [147]:
filtered_df_new.head()

,блогер,стоимость интеграция 1 слот,стоимость интеграция 2 слот,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",...,устник до сек_1 слот,_2 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,-_2 слот,"Пушной, Крастер, Старый_2 слот",нет_2 слот
27,bubblegunshow - 9 панчей,Интеграция с участием актеров: 350 000р,по запросу,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,bubblegunshow - 9 панчей,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,bubblegunshow - арена,Интеграция с участием актеров: 600 000р,по запросу,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,bubblegunshow - арена,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,bubblegunshow - кто твой подписчик,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
filtered_df_new.columns

Index(['блогер', 'стоимость интеграция 1 слот', 'стоимость интеграция 2 слот',
       'Интеграция с участием актеров_1 слот',
       'Интеграция с участием блогера_1 слот',
       'Интеграция с участием комиков из выпуска_1 слот',
       'Интеграция с участием Сергея Орлова_1 слот', '_1 слот',
       'Интеграция с участием ведущих_1 слот',
       'Пушной, Крастер, Старый_1 слот', 'Мамикс_1 слот',
       'интеграция до  сек_1 слот', 'устник до  сек_1 слот', '_2 слот',
       'Интеграция с участием блогера_2 слот',
       'Интеграция с участием актеров_2 слот',
       'Интеграция с участием комиков из выпуска_2 слот',
       'Интеграция с участием Сергея Орлова_2 слот',
       'Интеграция с участием ведущих_2 слот', '-_2 слот',
       'Пушной, Крастер, Старый_2 слот', 'нет_2 слот'],
      dtype='object', name=0)

> Удалим столбцы, не несущие дополнительной информации

In [149]:
filtered_df_new.drop(columns=['стоимость интеграция 1 слот',
                              'стоимость интеграция 2 слот',
                              '_1 слот',
                              '_2 слот',
                              '-_2 слот',
                              'нет_2 слот'], axis=1, inplace=True)

In [150]:
filtered_df_new.head()

,блогер,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",Мамикс_1 слот,интеграция до сек_1 слот,устник до сек_1 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,"Пушной, Крастер, Старый_2 слот"
27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,bubblegunshow - 9 панчей,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN
29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,bubblegunshow - арена,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN
32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN


In [151]:
filtered_df_new.reset_index(inplace=True)
filtered_df_new.head()

,index,блогер,Интеграция с участием актеров_1 слот,Интеграция с участием блогера_1 слот,Интеграция с участием комиков из выпуска_1 слот,Интеграция с участием Сергея Орлова_1 слот,Интеграция с участием ведущих_1 слот,"Пушной, Крастер, Старый_1 слот",Мамикс_1 слот,интеграция до сек_1 слот,устник до сек_1 слот,Интеграция с участием блогера_2 слот,Интеграция с участием актеров_2 слот,Интеграция с участием комиков из выпуска_2 слот,Интеграция с участием Сергея Орлова_2 слот,Интеграция с участием ведущих_2 слот,"Пушной, Крастер, Старый_2 слот"
0,27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,bubblegunshow - 9 панчей,NaN,550000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450000.0,NaN,NaN,NaN,NaN,NaN
2,29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30,bubblegunshow - арена,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,650000.0,NaN,NaN,NaN,NaN,NaN
4,32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1050000.0,NaN,NaN,NaN,NaN


> Сделаем два DataFrame с чётными и нечётными индексами, так как в них дублируются строки с одними и теми же блогерами

In [152]:
filtered_df_1 = filtered_df_new[filtered_df_new.index % 2 == 0]
filtered_df_2 = filtered_df_new[filtered_df_new.index % 2 != 0]

In [153]:
indexes_to_filtered_df_1 = filtered_df_1['index'].tolist()
indexes_to_filtered_df_1

[27, 29, 32, 36, 38, 43, 57, 60, 71, 75]

In [154]:
indexes_to_filtered_df_2 = filtered_df_2['index'].tolist()
indexes_to_filtered_df_2

[28, 30, 33, 37, 39, 44, 58, 61, 72, 76]

> С помощью merge объединим два получившихся DataFrame, чтобы не было дубликатов по блогерам

In [155]:
filtered_df_merge = pd.merge(filtered_df_1, filtered_df_2, on='блогер', how='outer')

In [156]:
filtered_df_merge.columns

Index(['index_x', 'блогер', 'Интеграция с участием актеров_1 слот_x',
       'Интеграция с участием блогера_1 слот_x',
       'Интеграция с участием комиков из выпуска_1 слот_x',
       'Интеграция с участием Сергея Орлова_1 слот_x',
       'Интеграция с участием ведущих_1 слот_x',
       'Пушной, Крастер, Старый_1 слот_x', 'Мамикс_1 слот_x',
       'интеграция до  сек_1 слот_x', 'устник до  сек_1 слот_x',
       'Интеграция с участием блогера_2 слот_x',
       'Интеграция с участием актеров_2 слот_x',
       'Интеграция с участием комиков из выпуска_2 слот_x',
       'Интеграция с участием Сергея Орлова_2 слот_x',
       'Интеграция с участием ведущих_2 слот_x',
       'Пушной, Крастер, Старый_2 слот_x', 'index_y',
       'Интеграция с участием актеров_1 слот_y',
       'Интеграция с участием блогера_1 слот_y',
       'Интеграция с участием комиков из выпуска_1 слот_y',
       'Интеграция с участием Сергея Орлова_1 слот_y',
       'Интеграция с участием ведущих_1 слот_y',
       'Пушн

> Удалим пустные столбцы

In [157]:
columns_with_all_nan = filtered_df_merge.columns[filtered_df_merge.isna().all()].tolist()
filtered_df_merge.drop(columns=columns_with_all_nan, axis=1, inplace=True)

In [158]:
filtered_df_merge.head()

,index_x,блогер,Интеграция с участием актеров_1 слот_x,Интеграция с участием комиков из выпуска_1 слот_x,"Пушной, Крастер, Старый_1 слот_x",интеграция до сек_1 слот_x,Интеграция с участием актеров_2 слот_x,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",index_y,Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
0,27,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,28,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
1,29,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,30,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
2,32,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,33,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
3,36,bubblegunshow - удиви меня,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,37,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
4,38,bubblegunshow - сходка,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,39,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Удалим столбцы со старыми индексами

In [159]:
filtered_df_merge.drop(columns=['index_x', 'index_y'], axis=1, inplace=True)

In [160]:
filtered_df_merge.head()

,блогер,Интеграция с участием актеров_1 слот_x,Интеграция с участием комиков из выпуска_1 слот_x,"Пушной, Крастер, Старый_1 слот_x",интеграция до сек_1 слот_x,Интеграция с участием актеров_2 слот_x,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
0,bubblegunshow - 9 панчей,350000.0,NaN,NaN,NaN,NaN,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
1,bubblegunshow - арена,600000.0,NaN,NaN,NaN,NaN,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
2,bubblegunshow - кто твой подписчик,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN
3,bubblegunshow - удиви меня,1150000.0,NaN,NaN,NaN,1050000.0,NaN,NaN,1450000.0,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN
4,bubblegunshow - сходка,650000.0,NaN,NaN,NaN,NaN,NaN,NaN,1250000.0,NaN,NaN,NaN,NaN,1150000.0,NaN,NaN


> Объединим получившийся DataFrame с исходным по столбцу "блогер"

In [161]:
vk_bloggers = vk_bloggers.merge(filtered_df_merge, on='блогер', how='outer')

In [162]:
indexes_to_filtered_df_2

[28, 30, 33, 37, 39, 44, 58, 61, 72, 76]

In [163]:
indexes_to_filtered_df_1

[27, 29, 32, 36, 38, 43, 57, 60, 71, 75]

In [164]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
26,600 000,Интеграция с участием актеров: 350 000р,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
27,NaN,Интеграция с участием блогера: 550 000р,Интеграция с участием блогера: 450 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
28,700 000,Интеграция с участием актеров: 600 000р,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
29,NaN,Интеграция с участием блогера: 750 000р,Интеграция с участием блогера: 650 000р,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
31,1 600 000,Интеграция с участием актеров: 1 150 000р,Интеграция с участием актеров: 1 050 000р,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN


> Отчистим столбцы 'стоимость интеграции 1-й слот', 'стоимость интеграции 2-й слот' от тех значений, которые мы перевели в отдельные столбцы

In [165]:
# Создаем паттерн для поиска строк, содержащих 'интеграция', 'устник' и '('
pattern = r'интеграция|устник|\(.*\)'  # Это регулярное выражение будет искать любое из указанных слов или '('

# Определите список слотов, которые вы хотите обработать
slots = ['стоимость интеграция 1 слот', 'стоимость интеграция 2 слот']

for slot_column in slots:
    for i in range(1, 3):
        vk_bloggers[f'стоимость интеграция {i} слот'] = vk_bloggers[f'стоимость интеграция {i} слот'].str.lower()

    # Применяем фильтрацию с использованием регулярного выражения
    filtered_duplicates = vk_bloggers[slot_column].str.contains(pattern, case=False, na=False)

    # Извлекаем значения в виде списка
    duplicates_position = vk_bloggers[slot_column][filtered_duplicates].tolist()

    # Заменяем найденные значения на np.nan
    vk_bloggers.loc[filtered_duplicates, slot_column] = np.nan

In [166]:
duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates.head()

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
26,600 000,NaN,по запросу,по запросу,р.350 000,р.250 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - 9 панчей,...,NaN,NaN,550000.0,NaN,NaN,NaN,NaN,450000.0,NaN,NaN
28,700 000,NaN,по запросу,по запросу,р.450 000,р.350 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - арена,...,NaN,NaN,750000.0,NaN,NaN,NaN,NaN,650000.0,NaN,NaN
31,1 600 000,NaN,NaN,по запросу,р.1 000 000,р.800 000,"Владислав Смирнов, TG: @vladislavsmirnov001, v...",https://vk.com/bubblegun_show,None,bubblegunshow - кто твой подписчик,...,NaN,NaN,1550000.0,NaN,NaN,NaN,NaN,1350000.0,NaN,NaN


> Удалим дубликаты из основного DataFrame vk_bloggers по индексам

In [167]:
indexes_to_duplicates = duplicates[duplicates.index % 2 == 0]
indexes_to_duplicates = indexes_to_duplicates.index.tolist()
indexes_to_duplicates

[26, 28, 32, 36, 38, 42, 56, 60, 70, 74]

In [168]:
vk_bloggers.drop(indexes_to_duplicates, inplace=True)

> Отлично, мы избавились от дубликатов и от неправильных значений в столбцe с ценами на интеграцию!

In [169]:
# Проверим на дубликаты

duplicates = vk_bloggers[vk_bloggers.duplicated('блогер', keep=False)]
duplicates

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y


> В тоже время мы сохранили все значение

In [170]:
vk_bloggers.loc[vk_bloggers['блогер'] == 'vk video - кошмарный сон а4']

,kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза),стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость преролл до 20 сек,стоимость мидролл до 20 сек,контакты менеджера по продажам,ссылка,статистика,блогер,...,Интеграция с участием комиков из выпуска_2 слот_x,"Пушной, Крастер, Старый_2 слот_x",Интеграция с участием блогера_1 слот_y,Интеграция с участием Сергея Орлова_1 слот_y,Интеграция с участием ведущих_1 слот_y,Мамикс_1 слот_y,устник до сек_1 слот_y,Интеграция с участием блогера_2 слот_y,Интеграция с участием Сергея Орлова_2 слот_y,Интеграция с участием ведущих_2 слот_y
75,NaN,NaN,нет,NaN,NaN,NaN,Варвара Каинова - v.kainova@vk.team,https://vk.com/vkvideo,https://drive.google.com/drive/folders/1BpIYrN...,vk video - кошмарный сон а4,...,NaN,NaN,NaN,NaN,NaN,NaN,2500000.0,NaN,NaN,NaN


> Переименуем столбцы, чтобы подвести их к формату DataFrame bufer

In [171]:
[col for col in vk_bloggers]

['kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)',
 'стоимость интеграция 1 слот',
 'стоимость интеграция 2 слот',
 'стоимость продакт плейсмент',
 'стоимость преролл до 20 сек',
 'стоимость мидролл до 20 сек',
 'контакты менеджера по продажам',
 'ссылка',
 'статистика',
 'блогер',
 'Интеграция с участием актеров_1 слот_x',
 'Интеграция с участием комиков из выпуска_1 слот_x',
 'Пушной, Крастер, Старый_1 слот_x',
 'интеграция до  сек_1 слот_x',
 'Интеграция с участием актеров_2 слот_x',
 'Интеграция с участием комиков из выпуска_2 слот_x',
 'Пушной, Крастер, Старый_2 слот_x',
 'Интеграция с участием блогера_1 слот_y',
 'Интеграция с участием Сергея Орлова_1 слот_y',
 'Интеграция с участием ведущих_1 слот_y',
 'Мамикс_1 слот_y',
 'устник до  сек_1 слот_y',
 'Интеграция с участием блогера_2 слот_y',
 'Интеграция с участием Сергея Орлова_2 слот_y',
 'Интеграция с участием ведущих_2 слот_y']

In [172]:
vk_bloggers.rename(columns = {'kpi (стартов просмотра на 1 эпизод на 30-е сутки после релиза)':'среднее кол-во просмотров',
                              'контакты менеджера по продажам':'контакты менеджера',
                              'Интеграция с участием актеров_1 слот_x':'стоимость интеграция с участием актеров 1 слот',
                              'Интеграция с участием комиков из выпуска_1 слот_x':'стоимость интеграция с участием комиков из выпуска 1 слот',
                              'Пушной, Крастер, Старый_1 слот_x':'стоимость интеграция с участием Пушного, Крастера, Старого 1 слот',
                              'интеграция до  сек_1 слот_x':'стоимость интеграция до 60 секунд 1 слот',
                              'Интеграция с участием актеров_2 слот_x':'стоимость интеграция с участием актеров 2 слот',
                              'Интеграция с участием комиков из выпуска_2 слот_x':'стоимость интеграция с участием комиков из выпуска 2 слот',
                              'Пушной, Крастер, Старый_2 слот_x':'стоимость интеграция с участием Пушного, Крастера, Старого 2 слот',
                              'Интеграция с участием блогера_1 слот_y':'стоимость интеграция с участием блогера 1 слот',
                              'Интеграция с участием Сергея Орлова_1 слот_y':'стоимость интеграция с участием Сергея Орлова 1 слот',
                              'Интеграция с участием ведущих_1 слот_y':'стоимость интеграция с участием ведущих 1 слот',
                              'Мамикс_1 слот_y':'стоимость интеграция с участием Мамикса 1 слот',
                              'устник до  сек_1 слот_y':'стоимость устник до 30 сек 1 слот',
                              'Интеграция с участием блогера_2 слот_y':'стоимость интеграция с участием блогера 2 слот',
                              'Интеграция с участием Сергея Орлова_2 слот_y':'стоимость интеграция с участием Сергея Орлова 2 слот',
                              'Интеграция с участием ведущих_2 слот_y':'стоимость интеграция с участием ведущих 2 слот'}, inplace = True)

In [173]:
bufer_columns = ['среднее кол-во просмотров',
                 'стоимость интеграция 1 слот',
                 'стоимость интеграция 2 слот',
                 'стоимость продакт плейсмент',
                 'стоимость преролл до 20 сек',
                 'стоимость мидролл до 20 сек',
                 'контакты менеджера',
                 'ссылка',
                 'статистика',
                 'блогер']

In [174]:
ok_bloggers.columns = bufer_columns

### Восьмое объединение

In [175]:
bufer = merge_bufer_df(bufer, vk_bloggers, 'VK Видео')

Количество дубликатов: 0


In [176]:
# Заполним столбец "контакты менеджера" данными из столбца "VK Видео контакты менеджера"
bufer['контакты менеджера'] = bufer['контакты менеджера'].fillna(bufer['VK Видео контакты менеджера'])

# Удалить столбец "VK Видео контакты менеджера"
bufer.drop(['VK Видео контакты менеджера'], axis=1, inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

### Девятое объединение

In [177]:
common_columns_vk_bloggers = bufer.columns.intersection(ok_bloggers.columns)
print(common_columns_vk_bloggers)

Index(['блогер', 'контакты менеджера'], dtype='object')


> У нас нет общих столбцов, так как к тем, что выше, мы добавим название соц сети ещё, но мы это предугадали в функции merge_bufer_df

In [178]:
bufer = merge_bufer_df(bufer, ok_bloggers, 'OK Шоу')

Количество дубликатов: 0


In [179]:
# Заполним столбец "контакты менеджера" данными из столбца "VK Видео контакты менеджера"
bufer['контакты менеджера'] = bufer['контакты менеджера'].fillna(bufer['OK Шоу контакты менеджера'])

# Удалить столбец "OK Шоу контакты менеджера"
bufer.drop(['OK Шоу контакты менеджера'], axis=1, inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

In [180]:
bufer.tail(10)

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK Видео стоимость интеграция с участием ведущих 2 слот,OK Шоу среднее кол-во просмотров,OK Шоу стоимость интеграция 1 слот,OK Шоу стоимость интеграция 2 слот,OK Шоу стоимость продакт плейсмент,OK Шоу стоимость преролл до 20 сек,OK Шоу стоимость мидролл до 20 сек,OK Шоу ссылка,OK Шоу статистика,OK Шоу блогер
157,дзен - утренний дзен (кликклак),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,дзен - история еды с сергеем минаевым new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,ок video - в мире домашних животных,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 150 000,по запросу,нет,р.800 000,нет,https://vk.com/vkvideo,None,ок video - в мире домашних животных
160,"ок video - ок, мама, поехали!",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.700 000,по запросу,по запросу,нет,нет,https://vk.com/vkvideo,None,"ок video - ок, мама, поехали!"
161,ок video - психология кино,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2 500 000,нет,нет,нет,р.500 000,нет,https://vk.com/vkvideo,None,ок video - психология кино
162,ок video - полтора спортсмена,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,https://vk.com/vkvideo,None,ок video - полтора спортсмена
163,ок video - шеф против любителя,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,р.800 000,р.500 000,нет,https://vk.com/vkvideo,None,ок video - шеф против любителя
164,ок video - кинокухня,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 100 000,по запросу,по запросу,р.500 000,нет,https://vk.com/vkvideo,None,ок video - кинокухня
165,ок video - игристое шоу new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,нет,нет,нет,р.1 500 000,р.1 200 000,https://vk.com/vkvideo,None,ок video - игристое шоу new
166,ок video - ок на связи!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5 000 000,р.1 500 000,р.1 000 000,по запросу,р.800 000,р.600 000,https://vk.com/vkvideo,None,ок video - ок на связи!


## Подготовка данных по блогерам от BlogoSphere

### Подготовка таблицы blogosphere_youtube

In [181]:
blogosphere_youtube.head()

5,NaN,Название канала,Шоу,Подписчики,Среднее кол-во просмотров на видео,Статистика аудитории,Стандартные форматы,NaN,NaN,NaN,...,NaN,NaN,NaN,Пример рекламы 1,Пример рекламы 2,Пример рекламы 3,Среднее CPV,NaN,Ссылка,Статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,...,NaN,Формат 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,...,Цена* от,Описание,Цена* от,NaN,NaN,NaN,Стандартный Формат 1,Стандартный Формат 2,NaN,NaN
8,Э,NaN,"Осторожно, Собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,...,1 310 000 ₽,Product Placement,1 470 000 ₽,Eduson,Экоплант,Травелата,"1,31₽","0,96₽",NaN,NaN
9,NaN,Ксения Собчак,Осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,...,NaN,NaN,NaN,Astons,CGPods,SkillFactory,"1,38₽","0,93₽",https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,NaN,Красота требует КЭШ,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,...,NaN,NaN,NaN,Золотое яблоко,4FORMS,FOREO,"1,20₽","1,08₽",NaN,NaN


> Переименуем сразу имена столбцов, чтобы нам было проще ориентироваться в данных

In [182]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот']

In [183]:
bufer_columns = ['Эксклюзивность',
                 'Название канала',
                 'Шоу',
                 'количество подписчиков',
                 'среднее кол-во просмотров',
                 'Статистика аудитории',
                 'Ф1 позиция',
                 'Ф1 Цена от',
                 'Ф2 позиция',
                 'Ф2 Цена от',
                 'Ф3 позиция',
                 'Ф3 Цена от',
                 'Ф4 позиция',
                 'Ф4 Цена от',
                 'Ф5 позиция',
                 'Ф5 Цена от',                 
                 'Пример рекламы 1',
                 'Пример рекламы 2',
                 'Пример рекламы 3',
                 'Среднее CPV формат 1',
                 'Среднее CPV формат 2',
                 'ссылка',
                 'Статистика ссылка']

In [184]:
blogosphere_youtube.columns = bufer_columns

> Сделаем предобработку DataFrame

In [185]:
key_word = ['Эксклюзивность', # Список ключевых слов для поиска столбцов и их удаления
            'Пример рекламы 1',
            'Пример рекламы 2',
            'Пример рекламы 3',
            'Среднее CPV формат 1',
            'Среднее CPV формат 2'] 

drop_word = '' # Список ключевых слов для их удаления из названия столбцов

upd_columns = ['Название канала', 'Шоу'] # Список столбцов с именами блогеров для приведения к единому формату

In [186]:
blogosphere_youtube = pretraining_df(blogosphere_youtube, key_word, drop_word, upd_columns)

In [187]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
6,NaN,NaN,NaN,NaN,NaN,Формат 1,NaN,Формат 2,NaN,Формат 3,NaN,Формат 4,NaN,Формат 5,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,Описание,Цена* от,NaN,NaN
8,NaN,"осторожно, собчак",NaN,1 000 000,NaN,1 слот,1 310 000 ₽,2 слот,960 000 ₽,3 слот,860 000 ₽,Преролл,1 310 000 ₽,Product Placement,1 470 000 ₽,NaN,NaN
9,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,Преролл,560 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
10,NaN,красота требует кэш,NaN,800 000,NaN,1 слот,960 000 ₽,2 Слот,860 000 ₽,Преролл,660 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
blogosphere_youtube[['название канала',
                     'шоу',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'статистика аудитории',
                     'ссылка',
                     'статистика ссылка']] = \
                    blogosphere_youtube[['название канала',
                                         'шоу',
                                         'количество подписчиков',
                                         'среднее кол-во просмотров',
                                         'статистика аудитории',
                                         'ссылка',
                                         'статистика ссылка']].fillna(method='ffill')

> Удалим первые две строки, так как в них указано неполное название столбцов и пока что в них актуальные пропуски, пока мы не запомнили "вверх"

In [189]:
blogosphere_youtube.dropna(subset=['шоу'], inplace=True)
blogosphere_youtube.reset_index(drop=True, inplace=True)

In [190]:
blogosphere_youtube[['название канала',
                     'шоу',
                     'количество подписчиков',
                     'среднее кол-во просмотров',
                     'статистика аудитории',
                     'ссылка',
                     'статистика ссылка']] = \
                    blogosphere_youtube[['название канала',
                                         'шоу',
                                         'количество подписчиков',
                                         'среднее кол-во просмотров',
                                         'статистика аудитории',
                                         'ссылка',
                                         'статистика ссылка']].fillna(method='backfill')

In [191]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,1 слот,1 310 000 ₽,2 слот,960 000 ₽,3 слот,860 000 ₽,Преролл,1 310 000 ₽,Product Placement,1 470 000 ₽,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,1 слот,760 000 ₽,2 Слот,510 000 ₽,Преролл,560 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,1 слот,960 000 ₽,2 Слот,860 000 ₽,Преролл,660 000 ₽,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...
3,rogov live,обзоры,460 000,400 000,Подробнее,1 слот\nссылка после описания ролика,1 005 000 ₽,2 слот\nссылка после описания ролика,905 000 ₽,3 слот,905 000 ₽,Преролл,1 005 000 ₽,Мидролл,905 000 ₽,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...
4,без фильтров,интервью,20 000,100 000,Подробнее,1 слот,240 000 ₽,2 слот,210 000 ₽,NaN,NaN,Преролл,115 000 ₽,Мидролл,115 000 ₽,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...


> У нас есть ещё одна интересная особенность в данных: реклама в Shorts сделана как отдельная тематика видео, а не как стоимость. Сделаем из неё отдельный DataFrame, чтобы потом объединить его с основным DataFrame

In [192]:
blogosphere_youtube_shorts = blogosphere_youtube.loc[blogosphere_youtube['шоу'] == 'shorts']
blogosphere_youtube_shorts

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ф1 позиция,ф1 цена от,ф2 позиция,ф2 цена от,ф3 позиция,ф3 цена от,ф4 позиция,ф4 цена от,ф5 позиция,ф5 цена от,ссылка,статистика ссылка
19,александр пушной,shorts,1 780 000,200 000,Подробнее,Интеграция,300 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/c/PushnoyRU,https://drive.google.com/drive/folders/1-UqeBp...
42,дневник дизайнера,shorts,985 000,100 000,Подробнее,1 cлот,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCbhIVqiD-Know...,https://drive.google.com/drive/folders/1-enyaA...
69,василий уткин,shorts,615 000,60 000,Подробнее,Ролик до 1 мин,84 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCwrRor4IvrByH...,https://drive.google.com/drive/folders/11uM9Ql...
77,джарахов,shorts,3 740 000,60 000,Подробнее,1 слот,505 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@dzharakhov/videos,https://drive.google.com/drive/folders/1x-28-y...
79,comigration,shorts,91 100,100 000,Подробнее,1 cлот,цена обсуждаема,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@comigration,https://drive.google.com/drive/folders/13_EecF...
84,на шуме,shorts,51 000,4 000,Подробнее,1 слот,150 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtube.com/@nashume,https://drive.google.com/drive/folders/14uUuwu...
87,nellifornication,shorts,2 500 000,1 000 000,Подробнее,Интеграция,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@Nellifornication,https://drive.google.com/drive/folders/1ZnvSiW...


> Переименуем столбцы

In [193]:
blogosphere_youtube_shorts.rename(columns = {'ф1 цена от':'стоимость shorts до 1 мин',
                                             'среднее кол-во просмотров':'среднее кол-во просмотров shorts'
                                            }, inplace=True)

In [194]:
blogosphere_youtube_shorts.columns

Index(['название канала', 'шоу', 'количество подписчиков',
       'среднее кол-во просмотров shorts', 'статистика аудитории',
       'ф1 позиция', 'стоимость shorts до 1 мин', 'ф2 позиция', 'ф2 цена от',
       'ф3 позиция', 'ф3 цена от', 'ф4 позиция', 'ф4 цена от', 'ф5 позиция',
       'ф5 цена от', 'ссылка', 'статистика ссылка'],
      dtype='object')

In [195]:
columns_with_shorts = ['название канала',
                       'среднее кол-во просмотров shorts',
                       'стоимость shorts до 1 мин']

# Удаление всех столбцов, кроме тех, что в списке columns_to_keep
blogosphere_youtube_shorts = blogosphere_youtube_shorts[columns_with_shorts]
blogosphere_youtube_shorts

,название канала,среднее кол-во просмотров shorts,стоимость shorts до 1 мин
19,александр пушной,200 000,300 000 ₽
42,дневник дизайнера,100 000,1 000 000 ₽
69,василий уткин,60 000,84 000 ₽
77,джарахов,60 000,505 000 ₽
79,comigration,100 000,цена обсуждаема
84,на шуме,4 000,150 000 ₽
87,nellifornication,1 000 000,720 000 ₽


> Удалим данные строки из основного DataFrame

In [196]:
blogosphere_youtube = blogosphere_youtube.drop(blogosphere_youtube[blogosphere_youtube['шоу'] == 'shorts'].index)

#### Перейдём к основному DataFrame

> Сделаем новые столбцы для уникальных форматов, которые указаны в столбцах  'Ф1 позиция', 'Ф2 позиция', 'Ф3 позиция', 'Ф4 позиция',  'Ф5 позиция', указав также их цены из столбцов 'Ф1 Цена от', 'Ф2 Цена от', 'Ф3 Цена от', 'Ф4 Цена от', 'Ф5 Цена от'.   
> В некоторых новых столбцах получится по 1-5 цен, но так как это уникальный формат и он может встретиться в будущем, мы их оставим.  
> Некоторые позиции могут повторяться в разных столбцах, поэтому для разных столбцов с позициями мы будем указывать ключ с int, чтобы в дальнейшем объединить столбцы с одинакомыми позициями

> Так как у нас повторяются столцы и индексы, которые мы хотели бы использовать, то сделаем через цикл

In [197]:
def get_column_values(df: pd.DataFrame,
                      key: int,
                      column_with_position: str,
                      column_with_price: str,
                      column_list: str) -> pd.DataFrame:
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
    df - DataFrame, в котором делаем преобразования;
    key - Ключ в виде int, который нужно добавить к названию, 
          чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
    column_with_position - Столбец, в котором указаны позиции;
    column_with_price - Столбец, в котором указаны цены на позиции;
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
    Создаём список с уникальными значениями позиций, приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
    spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                               .str.lower()\
                                               .str.replace('/\n', ', ')\
                                               .str.replace('\n', ', ')\
                                               .str.replace('*', '')\
                                               .unique()
    
    #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
    #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
    spisok = spisok[~pd.isna(spisok)]
        
        
    # Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
    
    # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
        df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
        where(df[column_with_position] == unique_position[i], other='')
    
    
    Удаляем исходные столбцы, чтобы не было повтора:
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)
    
    
    Возвращаем обновлённый DataFrame
    return df    
                
    """
    
    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]
    
    for i in range(len(unique_position)):
        new_column_name = f'{unique_position[i]}_{key}'
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')

            
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [198]:
column_list = []

for column_number in range(1, 6):
    position_column = f'ф{column_number} позиция'
    price_column = f'ф{column_number} цена от'
    blogosphere_youtube = get_column_values(blogosphere_youtube,
                                            column_number,
                                            position_column,
                                            price_column,
                                            column_list)

In [199]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ссылка,статистика ссылка,1 слот_1,1 слот\nссылка после описания ролика_1,Мини-интеграция до 1 мин_1,...,Преролл_3,Выпуск под клиента_3,Преролл_4,Мидролл_4,Преролл \nдо 20 сек_4,Product Placement_5,Мидролл_5,Спецвыпуск_5,Спонсорство_5,Product placement_5
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,1 310 000 ₽,NaN,NaN,...,NaN,NaN,1 310 000 ₽,NaN,NaN,1 470 000 ₽,NaN,NaN,NaN,NaN
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,760 000 ₽,NaN,NaN,...,560 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,960 000 ₽,NaN,NaN,...,660 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rogov live,обзоры,460 000,400 000,Подробнее,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,1 005 000 ₽,NaN,...,NaN,NaN,1 005 000 ₽,NaN,NaN,NaN,905 000 ₽,NaN,NaN,NaN
4,без фильтров,интервью,20 000,100 000,Подробнее,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,NaN,NaN,...,NaN,NaN,115 000 ₽,NaN,NaN,NaN,115 000 ₽,NaN,NaN,NaN


> Посмотрим на список новых столбцов, предварительно отсортировав его

In [200]:
sorted_column_names = sorted(column_list)
sorted_column_names

['1 слот\nссылка после описания ролика_1',
 '1 слот_1',
 '2  слот_2',
 '2 Слот_2',
 '2 слот\nссылка после описания ролика_2',
 '2 слот_2',
 '3 слот_3',
 'Product Placement_2',
 'Product Placement_5',
 'Product placement_5',
 'Выпуск под клиента_3',
 'Интеграция в стрим_1',
 'Мидролл_4',
 'Мидролл_5',
 'Мини-интеграция до 1 мин_1',
 'Нативная интеграция_1',
 'Нативная интеграция_2',
 'Преролл \nдо 20 сек_4',
 'Преролл_3',
 'Преролл_4',
 'Спецвыпуск_5',
 'Спонсорство_5']

> Объединим столбцы с общими позициями. Сделаем это через словарь

In [201]:
dict_new_columns = {'стоимость интеграция 1 слот':['1 слот\nссылка после описания ролика_1',
                                                    '1 слот_1'],
                    
                    'стоимость интеграция 2 слот':['2  слот_2',
                                                   '2 Слот_2',
                                                   '2 слот\nссылка после описания ролика_2',
                                                   '2 слот_2'],
                                        
                    'стоимость продакт плейсмент':['Product Placement_2',
                                                   'Product Placement_5',
                                                   'Product placement_5'],
                    
                    'стоимость выпуск под клиента':['Выпуск под клиента_3',
                                                    'Спецвыпуск_5'],
                    
                    'стоимость мидролл до 30 сек':['Мидролл_4',
                                                   'Мидролл_5'],
                    
                    'стоимость нативная интеграция':['Нативная интеграция_1',
                                                     'Нативная интеграция_2',],
                    
                    'стоимость преролл до 30 сек':['Преролл \nдо 20 сек_4',
                                                   'Преролл_3',
                                                   'Преролл_4']
                   }

In [202]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        blogosphere_youtube[col] = blogosphere_youtube[col].fillna('').astype(str)
    
    blogosphere_youtube[new_column] = blogosphere_youtube[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    blogosphere_youtube[new_column] = blogosphere_youtube[new_column].replace('', np.nan)

In [203]:
blogosphere_youtube.drop(columns=['1 слот\nссылка после описания ролика_1',
                                  '1 слот_1',
                                  '2  слот_2',
                                  '2 Слот_2',
                                  '2 слот\nссылка после описания ролика_2',
                                  '2 слот_2',
                                  '3 слот_3',
                                  'Product Placement_2',
                                  'Product Placement_5',
                                  'Product placement_5',
                                  'Выпуск под клиента_3',
                                  'Мидролл_4',
                                  'Мидролл_5',
                                  'Нативная интеграция_1',
                                  'Нативная интеграция_2',
                                  'Преролл \nдо 20 сек_4',
                                  'Преролл_3',
                                  'Преролл_4',
                                  'Спецвыпуск_5'], axis=1, inplace=True)

In [204]:
blogosphere_youtube.head()

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ссылка,статистика ссылка,Мини-интеграция до 1 мин_1,Интеграция в стрим_1,Спонсорство_5,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,1 310 000 ₽,960 000 ₽,1 470 000 ₽,NaN,NaN,NaN,1 310 000 ₽
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,760 000 ₽,510 000 ₽,NaN,NaN,NaN,NaN,560 000 ₽
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,960 000 ₽,860 000 ₽,NaN,NaN,NaN,NaN,660 000 ₽
3,rogov live,обзоры,460 000,400 000,Подробнее,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,NaN,NaN,1 005 000 ₽,905 000 ₽,NaN,NaN,905 000 ₽,NaN,1 005 000 ₽
4,без фильтров,интервью,20 000,100 000,Подробнее,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115 000 ₽,NaN,115 000 ₽


> Подготовим названия столбцов к дальнейшей работе: объединим имя блогера и название шоу в один столбец "канал"

In [205]:
[col for col in blogosphere_youtube.columns]

['название канала',
 'шоу',
 'количество подписчиков',
 'среднее кол-во просмотров',
 'статистика аудитории',
 'ссылка',
 'статистика ссылка',
 'Мини-интеграция до 1 мин_1',
 'Интеграция в стрим_1',
 'Спонсорство_5',
 'стоимость интеграция 1 слот',
 'стоимость интеграция 2 слот',
 'стоимость продакт плейсмент',
 'стоимость выпуск под клиента',
 'стоимость мидролл до 30 сек',
 'стоимость нативная интеграция',
 'стоимость преролл до 30 сек']

In [206]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот']

In [207]:
blogosphere_youtube.rename(columns = {'1 слот для профильных клиентов_1':'стоимость интеграция для профильных клиентов 1 слот',
                                      '1 слот для непрофильных клиентов_1':'стоимость интеграция для непрофильных клиентов 1 слот',
                                      'Ролик до 1 мин_1':'стоимость отдельное видео до 1 мин',
                                      'Интеграция в стрим_1':'стоимость интеграция в стрим',
                                      '2 слот для профильных клиентов_2':'стоимость интеграция для профильных клиентов 2 слот',
                                      '2 слот для непрофильных клиентов_2':'стоимость интеграция для непрофильных клиентов 2 слот',
                                      '3 слот_3':'стоимость интеграция 3 слот',
                                      'Спонсорство_5':'стоимость спонсорство',
                                      'стоимость за интеграцию 1-ый слот':'стоимость интеграция 1 слот',
                                      '4 слот_1':'стоимость интеграция 4 слот',
                                      '5 слот_1':'стоимость интеграция 5 слот',
                                      'Мини-интеграция до 1 мин_1':'стоимость мини-интеграция до 1 мин',
                                     }, inplace=True)


> Присоединим столбец со стоимостью Shorts к единому DataFrame с помощью merge

In [208]:
blogosphere_youtube = blogosphere_youtube.merge(blogosphere_youtube_shorts, on='название канала', how='left')
blogosphere_youtube

,название канала,шоу,количество подписчиков,среднее кол-во просмотров,статистика аудитории,ссылка,статистика ссылка,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин
0,ксения собчак,"осторожно, собчак",3 430 000,1 000 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,1 310 000 ₽,960 000 ₽,1 470 000 ₽,NaN,NaN,NaN,1 310 000 ₽,NaN,NaN
1,ксения собчак,осторожно: новости!,3 430 000,550 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,760 000 ₽,510 000 ₽,NaN,NaN,NaN,NaN,560 000 ₽,NaN,NaN
2,ксения собчак,красота требует кэш,3 430 000,800 000,Подробнее,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,NaN,NaN,960 000 ₽,860 000 ₽,NaN,NaN,NaN,NaN,660 000 ₽,NaN,NaN
3,rogov live,обзоры,460 000,400 000,Подробнее,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,NaN,NaN,1 005 000 ₽,905 000 ₽,NaN,NaN,905 000 ₽,NaN,1 005 000 ₽,NaN,NaN
4,без фильтров,интервью,20 000,100 000,Подробнее,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115 000 ₽,NaN,115 000 ₽,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,nellifornication,стандратный формат,2 500 000,500 000,Подробнее,https://www.youtube.com/@Nellifornication,https://drive.google.com/drive/folders/1ZnvSiW...,NaN,NaN,NaN,720 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽
80,nellifornication,тревел,2 500 000,800 000,Подробнее,https://www.youtube.com/@Nellifornication,https://drive.google.com/drive/folders/1ZnvSiW...,NaN,NaN,NaN,1 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,1 000 000,720 000 ₽
81,павел гительман,тревел-выпуски,2 500 000,1 000 000,Подробнее,https://www.youtube.com/@gitelman/featured,https://drive.google.com/drive/folders/1ZnvSiW...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,павел гительман,выпуски из студии,2 500 000,1 000 000,Подробнее,https://www.youtube.com/@gitelman/featured,https://drive.google.com/drive/folders/1ZnvSiW...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> Сделаем единый столбец с именем блогера, объединив столбцы 'название канала' и 'шоу'

In [209]:
blogosphere_youtube['блогер'] = \
                blogosphere_youtube['название канала'] + ' - ' + blogosphere_youtube['шоу']

blogosphere_youtube.drop(columns=['статистика аудитории',
                                  'название канала',
                                  'шоу'], axis=1, inplace=True)

> Сделаем столбец "статистика" по примеру других DataFrame

In [210]:
blogosphere_youtube.rename(columns = {'статистика ссылка':'статистика'}, inplace=True)

> У нас есть дубликат - обработаем его

In [211]:
duplicates_blogosphere_youtube = blogosphere_youtube[blogosphere_youtube.duplicated('блогер', keep=False)]
duplicates_blogosphere_youtube

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,среднее кол-во просмотров shorts,стоимость shorts до 1 мин,блогер


In [212]:
filtered_blogosphere_youtube_1 = duplicates_blogosphere_youtube[duplicates_blogosphere_youtube.index % 2 != 0]
filtered_blogosphere_youtube_2 = duplicates_blogosphere_youtube[duplicates_blogosphere_youtube.index % 2 == 0]

In [213]:
filtered_blogosphere_youtube_2 = filtered_blogosphere_youtube_2.dropna(axis=1)

In [214]:
# Создаем паттерн для поиска строк, содержащих слова стоимость и блогер
pattern = r'стоимость|блогер'

# Только их и оставить в DataFrame filtered_blogosphere_youtube_2
filtered_blogosphere_youtube_2 = filtered_blogosphere_youtube_2.filter(regex=pattern, axis=1)

# Объединим новый DataFrame с основным
blogosphere_youtube = blogosphere_youtube.merge(filtered_blogosphere_youtube_2, on='блогер', how='outer')

# Удалим дубликат из обновного DataFrame
indexes_to_duplicates_blogosphere_youtube = filtered_blogosphere_youtube_2.index.tolist()
blogosphere_youtube.drop(indexes_to_duplicates_blogosphere_youtube, inplace=True)

> Проверяем результат

In [215]:
duplicates_blogosphere_youtube = blogosphere_youtube[blogosphere_youtube.duplicated('блогер', keep=False)]
duplicates_blogosphere_youtube

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость мини-интеграция до 1 мин_x,стоимость интеграция в стрим_x,стоимость спонсорство_x,стоимость интеграция 1 слот_x,стоимость интеграция 2 слот_x,стоимость продакт плейсмент_x,...,стоимость интеграция в стрим_y,стоимость спонсорство_y,стоимость интеграция 1 слот_y,стоимость интеграция 2 слот_y,стоимость продакт плейсмент_y,стоимость выпуск под клиента_y,стоимость мидролл до 30 сек_y,стоимость нативная интеграция_y,стоимость преролл до 30 сек_y,стоимость shorts до 1 мин_y


In [216]:
duplicates_blogosphere_youtube.columns

Index(['количество подписчиков', 'среднее кол-во просмотров', 'ссылка',
       'статистика', 'стоимость мини-интеграция до 1 мин_x',
       'стоимость интеграция в стрим_x', 'стоимость спонсорство_x',
       'стоимость интеграция 1 слот_x', 'стоимость интеграция 2 слот_x',
       'стоимость продакт плейсмент_x', 'стоимость выпуск под клиента_x',
       'стоимость мидролл до 30 сек_x', 'стоимость нативная интеграция_x',
       'стоимость преролл до 30 сек_x', 'среднее кол-во просмотров shorts',
       'стоимость shorts до 1 мин_x', 'блогер',
       'стоимость мини-интеграция до 1 мин_y',
       'стоимость интеграция в стрим_y', 'стоимость спонсорство_y',
       'стоимость интеграция 1 слот_y', 'стоимость интеграция 2 слот_y',
       'стоимость продакт плейсмент_y', 'стоимость выпуск под клиента_y',
       'стоимость мидролл до 30 сек_y', 'стоимость нативная интеграция_y',
       'стоимость преролл до 30 сек_y', 'стоимость shorts до 1 мин_y'],
      dtype='object')

In [217]:
def rename_endswith_xy(df):
    
    """
    Функция выполняет преобразования в DataFrame, объединяя столбцы, заканчивающиеся на "_x" и "_y".

    ПАРАМЕТРЫ:
        - df (pd.DataFrame): DataFrame, в котором выполняются преобразования.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
        - pd.DataFrame: Измененный DataFrame после выполнения преобразований.

    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
        blogosphere_youtube = rename_endswith_xy(blogosphere_youtube)
    """
    
    columns_x = df.columns[df.columns.str.endswith("_x")]

    # Получите список столбцов, которые заканчиваются на "_y"
    columns_y = df.columns[df.columns.str.endswith("_y")]

    # Объедините оба списка в один
    base_name = np.concatenate([columns_x, columns_y])

    base_names_with_x_y = []

    # Проходим по всем столбцам DataFrame
    for column in base_name:
        # Разбиваем название столбца по "_x" и "_y"
        parts = column.split('_')

        # Если у нас есть и "_x" и "_y" и их части до "_x" и "_y" совпадают
        base_names_with_x_y.append(parts[0])  

    # base_name = pd.Series(base_name)
    unique_base_names = pd.Series(base_names_with_x_y).unique()

    # unique_base_names[1]

    for i in range(len(unique_base_names)):
        df[f'{unique_base_names[i]}_x'] = df[f'{unique_base_names[i]}_x'].fillna('').astype(str)
        df[f'{unique_base_names[i]}_y'] = df[f'{unique_base_names[i]}_y'].fillna('').astype(str)

        df[f'{unique_base_names[i]}'] = \
            df[[f'{unique_base_names[i]}_x', f'{unique_base_names[i]}_y']].agg(''.join, axis=1)

        df.drop(columns=[f'{unique_base_names[i]}_x', f'{unique_base_names[i]}_y'], axis=1, inplace=True)

        df[f'{unique_base_names[i]}'] = df[f'{unique_base_names[i]}'].replace('', np.nan)
        
    return df

In [218]:
# Проверяем
blogosphere_youtube.loc[blogosphere_youtube['блогер'] == 'без фильтров - интервью']

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,стоимость мини-интеграция до 1 мин_x,стоимость интеграция в стрим_x,стоимость спонсорство_x,стоимость интеграция 1 слот_x,стоимость интеграция 2 слот_x,стоимость продакт плейсмент_x,...,стоимость интеграция в стрим_y,стоимость спонсорство_y,стоимость интеграция 1 слот_y,стоимость интеграция 2 слот_y,стоимость продакт плейсмент_y,стоимость выпуск под клиента_y,стоимость мидролл до 30 сек_y,стоимость нативная интеграция_y,стоимость преролл до 30 сек_y,стоимость shorts до 1 мин_y
4,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
blogosphere_youtube = rename_endswith_xy(blogosphere_youtube)

In [220]:
# Проверяем
blogosphere_youtube.loc[blogosphere_youtube['блогер'] == 'без фильтров - интервью']

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,среднее кол-во просмотров shorts,блогер,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,стоимость shorts до 1 мин
4,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,без фильтров - интервью,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115 000 ₽,NaN,115 000 ₽,NaN


In [221]:
# # Заполняем пропуски в столбцах списка list_merge пустыми строками
# list_merge = ['стоимость за преролл до 30 сек_x',
#               'стоимость за преролл до 30 сек_y']

# blogosphere_youtube[list_merge] = blogosphere_youtube[list_merge].fillna('')

# # Создаем новый столбец на основе объединенных значений
# blogosphere_youtube['стоимость за преролл до 30 сек'] = \
#     blogosphere_youtube[['стоимость за преролл до 30 сек_x',
#                          'стоимость за преролл до 30 сек_y']].agg(''.join, axis=1)

# # Удалим те столбцы, которые объединяли
# blogosphere_youtube.drop(columns=['стоимость за преролл до 30 сек_x',
#                                   'стоимость за преролл до 30 сек_y'], axis=1, inplace=True)

# # Переведём пустые строки в NaN
# blogosphere_youtube[['стоимость за преролл до 30 сек']] = \
#                                     blogosphere_youtube[['стоимость за преролл до 30 сек']].replace('', np.nan)


In [222]:
blogosphere_youtube.head()

,количество подписчиков,среднее кол-во просмотров,ссылка,статистика,среднее кол-во просмотров shorts,блогер,стоимость мини-интеграция до 1 мин,стоимость интеграция в стрим,стоимость спонсорство,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость продакт плейсмент,стоимость выпуск под клиента,стоимость мидролл до 30 сек,стоимость нативная интеграция,стоимость преролл до 30 сек,стоимость shorts до 1 мин
0,3 430 000,1 000 000,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,"ксения собчак - осторожно, собчак",NaN,NaN,NaN,1 310 000 ₽,960 000 ₽,1 470 000 ₽,NaN,NaN,NaN,1 310 000 ₽,NaN
1,3 430 000,550 000,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,ксения собчак - осторожно: новости!,NaN,NaN,NaN,760 000 ₽,510 000 ₽,NaN,NaN,NaN,NaN,560 000 ₽,NaN
2,3 430 000,800 000,https://www.youtube.com/channel/UCvQXaJTjA3jRu...,https://drive.google.com/drive/folders/1TsQLLd...,NaN,ксения собчак - красота требует кэш,NaN,NaN,NaN,960 000 ₽,860 000 ₽,NaN,NaN,NaN,NaN,660 000 ₽,NaN
3,460 000,400 000,https://www.youtube.com/channel/UCefO_zFbQc6-z...,https://drive.google.com/drive/folders/1-PTy90...,NaN,rogov live - обзоры,NaN,NaN,NaN,1 005 000 ₽,905 000 ₽,NaN,NaN,905 000 ₽,NaN,1 005 000 ₽,NaN
4,20 000,100 000,https://www.youtube.com/@bezfiltrovshow/,https://drive.google.com/drive/u/0/folders/1Gg...,NaN,без фильтров - интервью,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115 000 ₽,NaN,115 000 ₽,NaN


### Десятое объединение

In [223]:
bufer = merge_bufer_df(bufer, blogosphere_youtube, 'YouTube')

Количество дубликатов: 0


In [224]:
def show_duplicates(df: pd.DataFrame) -> str:
    
    """
    Удаляет дубликаты из DataFrame и выводит информацию о количестве дубликатов в столбце 'блогер'.
    
    ПАРАМЕТРЫ:
        df (pd.DataFrame): Исходный DataFrame.

    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
        Строка с информацией о дубликатах, отсортированных по столбцу 'блогер'.

    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
        duplicates_info = show_duplicates(my_dataframe)
        print(duplicates_info)
    """
    
    df = df.drop_duplicates()

    duplicates_df = df[df.duplicated('блогер', keep=False)]

    print("Количество дубликатов:", len(duplicates_df))

    return duplicates_df.sort_values(by='блогер')

In [225]:
show_duplicates(bufer)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,OK Шоу стоимость мидролл до 20 сек,OK Шоу ссылка,OK Шоу статистика,OK Шоу блогер,YouTube среднее кол-во просмотров,YouTube стоимость мини-интеграция до 1 мин,YouTube стоимость интеграция в стрим,YouTube стоимость спонсорство,YouTube стоимость выпуск под клиента,YouTube стоимость нативная интеграция


In [226]:
# indices_to_remove = [244, 246]  # Индексы строк для удаления

# bufer = bufer.drop(indices_to_remove)

### Подготовка таблицы blogosphere_tg_vk_zen

In [227]:
blogosphere_tg_vk_zen.head()

4,NaN,Имя блогера,Telegram,NaN,NaN,NaN,NaN,TikTok,NaN,NaN,...,NaN,NaN,ссылка,Статистика ссылка,ссылка,Статистика ссылка,ссылка,Статистика ссылка,ссылка,Статистика ссылка
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,URL,Статистика,Подписчики,Средний охват поста,Цена* от...,URL,Статистика,Подписчики,...,Средний охват поста,Цена* от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Э,Редакция. Наука,Редакция. Наука,Подробнее,35 000,10 000,30 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/redakciya_nauka,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN
8,Э,Тамара Эйдельман,Уроки истории с Тамарой Эйдельман,Подробнее,58 000,30 000,135 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/eidelman,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN
9,Э,Михаил Шапошников,Mikhail Shaposhnikov LIVE,Подробнее,25 000,20 000,220 000 ₽,misha_des,NaN,268 600,...,800,50 000 ₽,https://t.me/misha_des,https://drive.google.com/drive/folders/14K8YiX...,https://www.tiktok.com/@misha_des,NaN,NaN,https://drive.google.com/drive/folders/14K8YiX...,https://zen.yandex.ru/mishades?lang=ru,https://zen.yandex.ru/mishades?lang=ru


In [228]:
[col for col in blogosphere_tg_vk_zen.columns]

[nan,
 'Имя блогера',
 'Telegram',
 nan,
 nan,
 nan,
 nan,
 'TikTok',
 nan,
 nan,
 nan,
 nan,
 'VK',
 nan,
 nan,
 nan,
 nan,
 'Яндекс.Дзен',
 nan,
 nan,
 nan,
 nan,
 'ссылка',
 'Статистика ссылка',
 'ссылка',
 'Статистика ссылка',
 'ссылка',
 'Статистика ссылка',
 'ссылка',
 'Статистика ссылка']

In [229]:
bufer_columns_blogosphere_tg_vk_zen = ['Эксклюзивность',
                                       'Имя блогера',
                                       'Telegram блогер',
                                       'Telegram статистика',
                                       'Telegram количество подписчиков',
                                       'Telegram среднее кол-во просмотров поста',
                                       'Telegram стоимость фотопост',
                                       'TikTok блогер',
                                       'TikTok статистика',
                                       'TikTok количество подписчиков',
                                       'TikTok среднее кол-во просмотров',
                                       'TikTok стоимость видеопост',
                                       'VK группа блогер',
                                       'VK группа блогер статистика',
                                       'VK группа количество подписчиков',
                                       'VK группа среднее кол-во просмотров поста',
                                       'VK группа стоимость фотопост',
                                       'Дзен блогер',
                                       'Дзен статистика',
                                       'Дзен количество подписчиков',
                                       'Дзен среднее кол-во просмотров',
                                       'Дзен стоимость пост',
                                       'Telegram ссылка',
                                       'Telegram cтатистика',
                                       'TikTok ссылка',
                                       'TikTok статистика',
                                       'VK группа ссылка',
                                       'VK группа статистика',
                                       'Дзен ссылка',
                                       'Дзен статистика']

In [230]:
blogosphere_tg_vk_zen.columns = bufer_columns_blogosphere_tg_vk_zen

In [231]:
blogosphere_tg_vk_zen

,Эксклюзивность,Имя блогера,Telegram блогер,Telegram статистика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram стоимость фотопост,TikTok блогер,TikTok статистика,TikTok количество подписчиков,...,Дзен среднее кол-во просмотров,Дзен стоимость пост,Telegram ссылка,Telegram cтатистика,TikTok ссылка,TikTok статистика,VK группа ссылка,VK группа статистика,Дзен ссылка,Дзен статистика
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,URL,Статистика,Подписчики,Средний охват поста,Цена* от...,URL,Статистика,Подписчики,...,Средний охват поста,Цена* от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Э,Редакция. Наука,Редакция. Наука,Подробнее,35 000,10 000,30 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/redakciya_nauka,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN
8,Э,Тамара Эйдельман,Уроки истории с Тамарой Эйдельман,Подробнее,58 000,30 000,135 000 ₽,NaN,NaN,NaN,...,NaN,NaN,https://t.me/eidelman,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN
9,Э,Михаил Шапошников,Mikhail Shaposhnikov LIVE,Подробнее,25 000,20 000,220 000 ₽,misha_des,NaN,268 600,...,800,50 000 ₽,https://t.me/misha_des,https://drive.google.com/drive/folders/14K8YiX...,https://www.tiktok.com/@misha_des,NaN,NaN,https://drive.google.com/drive/folders/14K8YiX...,https://zen.yandex.ru/mishades?lang=ru,https://zen.yandex.ru/mishades?lang=ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,freekicksrus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
key_word = ['Эксклюзивность']# Список ключевых слов для поиска столбцов и их удаления
             
drop_word = '' # Список ключевых слов для их удаления из названия столбцов

upd_columns = ['Имя блогера',
               'Telegram блогер',
               'TikTok блогер',
               'VK группа блогер',
               'Дзен блогер'] # Список столбцов с именами блогеров для приведения к единому формату

In [233]:
blogosphere_tg_vk_zen = pretraining_df(blogosphere_tg_vk_zen, key_word, drop_word, upd_columns)

> Назовём сразу столбцы как в едином DataFrame bufer, чтобы потом не делить данный DataFrame на части и не объединять их через функцию merge_bufer_df

In [234]:
bufer_columns_blogosphere_tg_vk_zen = ['блогер',
                                       'Telegram блогер',
                                       'Telegram статистика',
                                       'Telegram количество подписчиков',
                                       'Telegram среднее кол-во просмотров поста',
                                       'Telegram стоимость фотопост',
                                       'TikTok блогер',
                                       'TikTok статистика',
                                       'TikTok количество подписчиков',
                                       'TikTok среднее кол-во просмотров',
                                       'TikTok стоимость видеопост',
                                       'VK группа блогер',
                                       'VK группа статистика',
                                       'VK группа количество подписчиков',
                                       'VK группа среднее кол-во просмотров поста',
                                       'VK группа стоимость фотопост',
                                       'Дзен блогер',
                                       'Дзен статистика',
                                       'Дзен количество подписчиков',
                                       'Дзен среднее кол-во просмотров',
                                       'Дзен стоимость пост',
                                       'Telegram ссылка',
                                       'Telegram cтатистика',
                                       'TikTok ссылка',
                                       'TikTok статистика',
                                       'VK группа ссылка',
                                       'VK группа статистика',
                                       'Дзен ссылка',
                                       'Дзен статистика']

In [235]:
blogosphere_tg_vk_zen.columns = bufer_columns_blogosphere_tg_vk_zen

In [236]:
blogosphere_tg_vk_zen.head()

,блогер,Telegram блогер,Telegram статистика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram стоимость фотопост,TikTok блогер,TikTok статистика,TikTok количество подписчиков,TikTok среднее кол-во просмотров,...,Дзен среднее кол-во просмотров,Дзен стоимость пост,Telegram ссылка,Telegram cтатистика,TikTok ссылка,TikTok статистика,VK группа ссылка,VK группа статистика,Дзен ссылка,Дзен статистика
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,url,Статистика,Подписчики,Средний охват поста,Цена* от...,url,Статистика,Подписчики,Средний охват поста,...,Средний охват поста,Цена* от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,редакция. наука,редакция. наука,Подробнее,35 000,10 000,30 000 ₽,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.me/redakciya_nauka,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-0D_wF...,NaN,NaN
8,тамара эйдельман,уроки истории с тамарой эйдельман,Подробнее,58 000,30 000,135 000 ₽,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.me/eidelman,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN,NaN,https://drive.google.com/drive/folders/1-8iJuY...,NaN,NaN
9,михаил шапошников,mikhail shaposhnikov live,Подробнее,25 000,20 000,220 000 ₽,misha_des,NaN,268 600,NaN,...,800,50 000 ₽,https://t.me/misha_des,https://drive.google.com/drive/folders/14K8YiX...,https://www.tiktok.com/@misha_des,NaN,NaN,https://drive.google.com/drive/folders/14K8YiX...,https://zen.yandex.ru/mishades?lang=ru,https://zen.yandex.ru/mishades?lang=ru


> Удалим лишние пустые строки, в которых у нас нет информации из-за особенности DataFrame, в котором под таблицей указали запасные данные: удалить строки, в которых значения равны NaN как в столбце 'блогер', так и в столбце 'telegram блогер' как в самых заполненных столбцах

In [237]:
blogosphere_tg_vk_zen = blogosphere_tg_vk_zen.dropna(subset=['блогер', 'Telegram блогер'], how='all')

# Отдельно удалим первую строку с NaN в столбец 'блогер', которая представляем собой название столбцов в виде строки
blogosphere_tg_vk_zen = blogosphere_tg_vk_zen.dropna(subset=['блогер'])

> Заполним значение столбца "имя блогера" с помощью метода .fillna(method='ffill')

In [238]:
blogosphere_tg_vk_zen[['блогер']] = blogosphere_tg_vk_zen[['блогер']].fillna(method='ffill')

> Удалим лишние столбцы, где нет ссылки на статистику

In [239]:
blogosphere_tg_vk_zen.drop(columns=['Telegram статистика',
                                    'TikTok статистика',
                                    'VK группа статистика',
                                    'Дзен статистика',
                                    'Telegram cтатистика'], axis=1, inplace=True)

In [240]:
blogosphere_tg_vk_zen.head()

,блогер,Telegram блогер,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram стоимость фотопост,TikTok блогер,TikTok количество подписчиков,TikTok среднее кол-во просмотров,TikTok стоимость видеопост,VK группа блогер,...,VK группа среднее кол-во просмотров поста,VK группа стоимость фотопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,Telegram ссылка,TikTok ссылка,VK группа ссылка,Дзен ссылка
7,редакция. наука,редакция. наука,35 000,10 000,30 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/redakciya_nauka,NaN,NaN,NaN
8,тамара эйдельман,уроки истории с тамарой эйдельман,58 000,30 000,135 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/eidelman,NaN,NaN,NaN
9,михаил шапошников,mikhail shaposhnikov live,25 000,20 000,220 000 ₽,misha_des,268 600,NaN,по запросу,NaN,...,NaN,NaN,дневник дизайнера,3 100,800,50 000 ₽,https://t.me/misha_des,https://www.tiktok.com/@misha_des,NaN,https://zen.yandex.ru/mishades?lang=ru
10,аня консервы,фасоликус томатикус,38 000,30 000,90 000 ₽,a_conservi,99 200,40 000,80 000 ₽,conservi,...,25 000,65 000 ₽,NaN,NaN,NaN,NaN,https://t.me/fasolicustomaticus,https://www.tiktok.com/@a_conservi,https://vk.com/conservishit,NaN
11,солодников,солодников,32 000,30 000,80 000 ₽,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/s/solodnikov,NaN,NaN,NaN


### Одинадцатое объединение

> У нас есть общие столбцы - на основе их и присоединим blogosphere_tg_vk_zen к bufer

In [241]:
common_columns_blogosphere_tg_vk_zen = bufer.columns.intersection(blogosphere_tg_vk_zen.columns)
print(common_columns_blogosphere_tg_vk_zen)

Index(['блогер', 'Telegram блогер', 'Telegram ссылка',
       'Telegram количество подписчиков',
       'Telegram среднее кол-во просмотров поста',
       'Telegram стоимость фотопост', 'VK группа блогер', 'VK группа ссылка',
       'VK группа количество подписчиков',
       'VK группа среднее кол-во просмотров поста',
       'VK группа стоимость фотопост'],
      dtype='object')


> Так как мы сразу указали название соц. сети в названии столбцов, чтобы не делить на несколько DataFrame, то объединим не с помощью функции, а вручную с помощью merge

In [242]:
bufer = bufer.merge(blogosphere_tg_vk_zen, \
                    left_on=common_columns_blogosphere_tg_vk_zen.tolist(), \
                    right_on=common_columns_blogosphere_tg_vk_zen.tolist(), \
                    how='outer')

In [243]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,TikTok блогер,TikTok количество подписчиков,TikTok среднее кол-во просмотров,TikTok стоимость видеопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,TikTok ссылка,Дзен ссылка
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ваша маруся,лайфстайл/музыка,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,TikTok блогер,TikTok количество подписчиков,TikTok среднее кол-во просмотров,TikTok стоимость видеопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,TikTok ссылка,Дзен ссылка


In [245]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('sales@blogo-sphere.com', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

## Подготовка данных по блогерам от Insight People

**Имеющиеся DataFrame:**  

* insight_instagram
* insight_threads
* insight_telegram
* insight_vk_ls
* insight_vk_group
* insight_youtube
* insight_zen
* insight_likee
* insight_yappy
* insight_rutube
<!-- * insight_ok -->

> Предварительно сделаем предобработку DataFrame от Insight People

In [246]:
insight_df = [insight_instagram,
              insight_threads,
              insight_telegram,
              insight_vk_ls,
              insight_vk_group,
              insight_youtube,
              insight_zen,
              insight_likee,
              insight_yappy,
              insight_rutube,]
#               insight_ok]

In [247]:
key_word = ['CPU', 'ER', 'CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов

In [248]:
for i in range(len(insight_df)):
    df = insight_df[i]  # Получаем текущий DataFrame по индексу
    
    upd_columns = []  # Пустой список столбцов с именами блогеров для приведения к единому формату

    if 'Фамилия' in df.columns:
        upd_columns.append('Фамилия')
    if 'Имя' in df.columns:
        upd_columns.append('Имя')
    if 'Имя блогера' in df.columns:
        upd_columns.append('Имя блогера')
    if 'Название канала' in df.columns:
        upd_columns.append('Название канала')
    if 'ФИО' in df.columns:
        upd_columns.append('ФИО')
    if 'Фамилия имя' in df.columns:
        upd_columns.append('Фамилия имя')

    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы insight_instagram

In [249]:
insight_instagram.head()

3,город,фамилия имя,ссылка,тематика,количество подписчиков,охват пост,охват сторис,охват серии сторис,стоимость за фотопост,стоимость за сторис за 15 секунд,стоимость за серию сторис до 60 секунд,стоимость за видеопост / reels
4,Москва,манукян давид,https://instagram.com/dava_m,"Артист, стример, лайфстайл, развлечения, кино,...",13 500 000,1 900 000,700 000,1 750 000,"р.1 244 445,00","р.497 778,00","р.835 555,00","р.1 866 667,00"
5,Москва,лазарьянц карина,https://instagram.com/karinakross,"Развлечения, юмор, лайфстайл",10 400 000,950 000,500 000,1 250 000,"р.832 500,00",под запрос,"р.832 500,00","р.1 165 500,00"
6,Москва,хилькевич анна,https://instagram.com/annakhilkevich,"Лайфстайл, мама, актриса",9 900 000,410 000,120 000,300 000,"р.510 000,00","р.170 000,00","р.425 000,00","р.600 000,00"
7,Москва,левда алина,https://www.instagram.com/botanovna_/,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",7 600 000,950 000,420 000,1 050 000,"р.1 322 222,00",под запрос,"р.622 222,00","р.1 555 555,00"
8,Москва,ершов евгений,https://instagram.com/ershovich,"Лайфстайл, контент-мейкер, развлечения",5 300 000,450 000,400 000,1 000 000,"р.422 222,00","р.116 111,00","р.306 111,00","р.570 000,00"


> Сделаем общий столбец с именем и фамилией блогера, и так как данное действие мы будем делать практически у всех DataFrame от Insight, то сделаем это функцией, а так как мы будем делать это для большинства DataFrame от Insight People, то сделаем это функцией

In [250]:
[col for col in insight_instagram.columns]

['город',
 'фамилия имя',
 'ссылка',
 'тематика',
 'количество подписчиков',
 'охват пост',
 'охват сторис',
 'охват серии сторис',
 'стоимость за фотопост',
 'стоимость за сторис за 15 секунд',
 'стоимость за серию сторис до 60 секунд',
 'стоимость за видеопост / reels']

In [251]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика']

In [252]:
insight_instagram.rename(columns = {'фамилия имя':'блогер',
                                    'стоимость за фотопост':'стоимость фотопост',
                                    'охват пост':'среднее кол-во просмотров фотопоста',
                                    'охват сторис':'среднее кол-во просмотров 1 сторис',
                                    'охват серии сторис':'среднее кол-во просмотров серии сторис',
                                    'стоимость за сторис за 15 секунд':'стоимость 1 сторис до 15 сек',
                                    'стоимость за серию сторис до 60 секунд':'стоимость серия сторис до 60 сек',
                                    'стоимость за видеопост / reels':'стоимость reels / видеопост'
                                    }, inplace=True)

#### Проверка на дубликаты

> **ДОПИСАТЬ**

In [253]:
def show_duplicates(df):
    
    """
    """
    
    df = df.drop_duplicates()

    duplicates_df = df[df.duplicated('блогер', keep=False)]

    print("Количество дубликатов:", len(duplicates_df))

    return duplicates_df.sort_values(by='блогер')

In [254]:
show_duplicates(insight_instagram)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров серии сторис,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия сторис до 60 сек,стоимость reels / видеопост


> В исходном DataFrame у нас есть дубликат, который отличается только значением в столбце "Instagram стоимость за серию сторис до 60 сек". Удалим один из них по индексу

In [255]:
duplicates_insight_instagram = insight_instagram[insight_instagram.duplicated('блогер', keep=False)]
duplicates_insight_instagram

3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров серии сторис,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия сторис до 60 сек,стоимость reels / видеопост


In [256]:
indexes_to_duplicates_insight_instagram = duplicates_insight_instagram[duplicates_insight_instagram.index % 2 == 0]
indexes_to_duplicates_insight_instagram = indexes_to_duplicates_insight_instagram.index.tolist()
indexes_to_duplicates_insight_instagram

[]

In [257]:
insight_instagram.drop(indexes_to_duplicates_insight_instagram, inplace=True)

In [258]:
show_duplicates(insight_instagram)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров фотопоста,среднее кол-во просмотров 1 сторис,среднее кол-во просмотров серии сторис,стоимость фотопост,стоимость 1 сторис до 15 сек,стоимость серия сторис до 60 сек,стоимость reels / видеопост


### Двенадцатое объединение

In [259]:
bufer = merge_bufer_df(bufer, insight_instagram, 'Instagram')

Количество дубликатов: 0


In [260]:
bufer.head()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,TikTok стоимость видеопост,Дзен блогер,Дзен количество подписчиков,Дзен среднее кол-во просмотров,Дзен стоимость пост,TikTok ссылка,Дзен ссылка,Instagram город,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек
0,катя клэп,лайфстайл/бьюти,катя клэп,https://www.instagram.com/kateclapp/,лайфстайл/бьюти,10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,влад бумага а4,челленджи,влад бумага а4,https://www.instagram.com/a4omg/,челленджи,7 100 000,900 000,400 000,1 000 000,60 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,катя адушкина,музыка/лайфстайл/танцы,катя адушкина,https://www.instagram.com/katusha_adushkina/,музыка/лайфстайл/танцы,6 300 000,700 000,260 000,600 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ваша маруся,лайфстайл/музыка,ваша маруся,https://www.instagram.com/vashamarousia/,лайфстайл/музыка,3 300 000,185 000,115 000,350 000,30 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,амина tenderlybae,твич/лайфстайл,амина tenderlybae,https://www.instagram.com/tenderlybae/,твич/лайфстайл,3 900 000,600 000,300 000,650 000,110 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Подготовка таблицы insight_threads

In [261]:
insight_threads.head()

3,город,фамилия,имя,ссылка,тематика,количество подписчиков,стоимость за фотопубликацию,стоимость за видеопубликацию
4,Москва,манукян,давид,https://www.threads.net/@dava_m,"Артист, стример, лайфстайл, развлечения, кино,...",70 800,под запрос,под запрос
5,Москва,акилова,алина,https://www.threads.net/@alina_akilova,"Лайфстайл, бьюти, мама",50 900,под запрос,под запрос
6,Москва,серов,никита,https://www.threads.net/@serovski,"Лайфстайл, семья, путешествия, авто",43 700,под запрос,под запрос
7,Москва,левда,алина,https://www.threads.net/@botanovna_,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",42 400,под запрос,под запрос
8,Москва,нигай,карина,https://www.threads.net/@karina_nigay,"Лайфстайл, фешн, мода, бьюти, путешествия, семья",32 500,под запрос,под запрос


In [262]:
def name_familia(df: pd.DataFrame) -> pd.DataFrame:
    
    """
    Функция объединяет два столбца 'имя' и 'фамилия' нашего DataFrame df в единый столбец 'блогер'.
    Далее мы удаляем столбцы 'имя' и 'фамилия' из DataFrame.
    
    
    ПАРАМЕТРЫ:
    
    df (DataFrame) - DataFrame, в котором мы обновляем столбцы
    
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    
    df (DataFrame) - обновлённый DataFrame
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    insight_instagram = name_familia(insight_instagram)
    
    """
    
    df['блогер'] = df['фамилия'] + ' ' + df['имя']

    df.drop(columns=['имя', 'фамилия'], axis= 1, inplace=True)
    
    return df

In [263]:
insight_threads = name_familia(insight_threads)

In [264]:
insight_threads.head()

3,город,ссылка,тематика,количество подписчиков,стоимость за фотопубликацию,стоимость за видеопубликацию,блогер
4,Москва,https://www.threads.net/@dava_m,"Артист, стример, лайфстайл, развлечения, кино,...",70 800,под запрос,под запрос,манукян давид
5,Москва,https://www.threads.net/@alina_akilova,"Лайфстайл, бьюти, мама",50 900,под запрос,под запрос,акилова алина
6,Москва,https://www.threads.net/@serovski,"Лайфстайл, семья, путешествия, авто",43 700,под запрос,под запрос,серов никита
7,Москва,https://www.threads.net/@botanovna_,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",42 400,под запрос,под запрос,левда алина
8,Москва,https://www.threads.net/@karina_nigay,"Лайфстайл, фешн, мода, бьюти, путешествия, семья",32 500,под запрос,под запрос,нигай карина


In [265]:
insight_threads.rename(columns = {'стоимость за фотопубликацию':'стоимость фотопост',
                                  'стоимость за видеопубликацию':'стоимость видеопост'
                                 }, inplace=True)

#### Проверка на дубликаты

In [266]:
show_duplicates(insight_threads)

Количество дубликатов: 2


3,город,ссылка,тематика,количество подписчиков,стоимость фотопост,стоимость видеопост,блогер
15,Москва,https://www.threads.net/@leratumanova,Лайфстайл,13 200,под запрос,под запрос,туманова валерия
16,Москва,https://www.threads.net/@leratumanova,"Лайфстайл, артист, семья, путешествия, мода, б...",13 100,под запрос,под запрос,туманова валерия


> Опять есть дубликат - удалим тот, где меньше всего данных в столбце "тематика", так как в единственном столбце "количество подписчиков", где есть изменения, цифры практически не отличаются

In [267]:
duplicates_insight_threads = insight_threads[insight_threads.duplicated('блогер', keep=False)]
duplicates_insight_threads

3,город,ссылка,тематика,количество подписчиков,стоимость фотопост,стоимость видеопост,блогер
15,Москва,https://www.threads.net/@leratumanova,Лайфстайл,13 200,под запрос,под запрос,туманова валерия
16,Москва,https://www.threads.net/@leratumanova,"Лайфстайл, артист, семья, путешествия, мода, б...",13 100,под запрос,под запрос,туманова валерия


In [268]:
indexes_to_duplicates_insight_threads = duplicates_insight_threads[duplicates_insight_threads.index % 2 != 0]
indexes_to_duplicates_insight_threads = indexes_to_duplicates_insight_threads.index.tolist()
indexes_to_duplicates_insight_threads

[15]

In [269]:
insight_threads.drop(indexes_to_duplicates_insight_threads, inplace=True)

In [270]:
show_duplicates(insight_threads)

Количество дубликатов: 0


3,город,ссылка,тематика,количество подписчиков,стоимость фотопост,стоимость видеопост,блогер


### Тринадцатое объединение

> У нас нет общих столбцов, поэтому объединим по имени блогера

In [271]:
bufer = merge_bufer_df(bufer, insight_threads, 'Threads')

Количество дубликатов: 0


### Подготовка таблицы insight_telegram

In [272]:
insight_telegram.head()

3,город,фамилия имя,ссылка,тематика,количество подписчиков,средний охват публикации,кружок,стоимость за фотопост,стоимость за видеопост
4,Барнаул,горбунова надежда,https://t.me/brai_sa,"Лайфстайл, Сериальные квесты",276 663,154 000,"р.448 000,00","р.448 000,00","р.512 000,00"
5,Нижний Новгород,шишкова оксана,https://t.me/oxgerman,"Лайфстайл, семья, путешествия, еда",216 000,120 000,"р.180 000,00","р.300 000,00","р.360 000,00"
6,Москва,левда алина,https://t.me/+-wxY1yGLRhU0NzMy,"Лайфстайл,мама,семья,отношения,бьюти,путешеств...",200 000,35 000,"р.260 000,00","р.292 500,00","р.364 000,00"
7,Краснодар,морозова алина,https://t.me/Alinamorozovaofficial,"Нутрициолог, ЗОЖ",184 403,84 000,"р.144 000,00","р.144 000,00","р.144 000,00"
8,Кемерово,гаврилова анжелика,https://t.me/anj0731,"Семья, лайфстайл, бьюти-обзоры",155 000,130 000,"р.150 000,00","р.210 000,00","р.300 000,00"


> Переименуем столбцы под общий формат

In [273]:
[col for col in insight_telegram.columns]

['город',
 'фамилия имя',
 'ссылка',
 'тематика',
 'количество подписчиков',
 'средний охват публикации',
 'кружок',
 'стоимость за фотопост',
 'стоимость за видеопост']

In [274]:
[col for col in bufer.columns if 'Telegram' in col]

['Telegram блогер',
 'Telegram ссылка',
 'Telegram тематика',
 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',
 'Telegram статистика']

In [275]:
insight_telegram.rename(columns = {'фамилия имя':'блогер',
                                   'средний охват публикации':'среднее кол-во просмотров поста',
                                   'кружок':'стоимость кружочек до 60 сек',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост'
                                  }, inplace = True)

#### Проверка на дубликаты

In [276]:
show_duplicates(insight_telegram)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров поста,стоимость кружочек до 60 сек,стоимость фотопост,стоимость видеопост


> Дубликатов нет - может объединять

### Четырнадцатое объединение

> Исправим одну неточность в имени блогера

In [277]:
insight_telegram.loc\
    [insight_telegram['блогер'] == 'сивцев станислав', 'блогер'] = 'станислав сивцев'

In [278]:
bufer = merge_bufer_df(bufer, insight_telegram, 'Telegram')

Количество дубликатов: 46


In [279]:
show_duplicates(bufer).head(6)

Количество дубликатов: 46


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город
420,акилова алина,"Лайфстайл, бьюти",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Москва
227,акилова алина,"Лайфстайл, бьюти, мама",акилова алина,https://instagram.com/alina_akilova,"Лайфстайл, бьюти, мама",3 900 000,350 000,250 000,NaN,NaN,...,625 000,"р.422 222,00",Москва,https://www.threads.net/@alina_akilova,"Лайфстайл, бьюти, мама",50 900,под запрос,под запрос,акилова алина,NaN
288,ахметзянов айрат,"Фудблогер, рецепты,готовка",ахметзянов айрат,https://instagram.com/bistrie.recepty2.0,"Фудблогер, рецепты,готовка",256 000,120 000,48 000,NaN,NaN,...,120 000,"р.30 720,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,ахметзянов айрат,"Food / Еда, рецепты",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Казань
418,бурмистрова татьяна,"Лайфстайл, семья, путешествия, развлечения, об...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Владивосток
250,бурмистрова татьяна,"Мама, лайфстайл",бурмистрова татьяна,https://instagram.com/tassi_stories,"Мама, лайфстайл",1 000 000,400 000,250 000,NaN,NaN,...,625 000,"р.600 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> У нас есть определённая проблема с DataFrame insight_telegram - почти половина каналов не заполнились с помощью merge, а сделались новыми строками - скорее всего это произошло из-за особого форматирования данных столбца "блогер", так как проверка ниже показывает, что данные равны по значению, и нам надо объединить данные строки в единую строку.

In [280]:
# Проверка
test_dub = bufer.loc[bufer['блогер'] == 'ахметзянов айрат']
test_dub.reset_index()

,index,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город
0,288,ахметзянов айрат,"Фудблогер, рецепты,готовка",ахметзянов айрат,https://instagram.com/bistrie.recepty2.0,"Фудблогер, рецепты,готовка",256 000,120 000,48 000,NaN,...,120 000,"р.30 720,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,444,ахметзянов айрат,"Food / Еда, рецепты",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Казань


In [281]:
string1 = test_dub.iloc[0]['блогер']
string2 = test_dub.iloc[1]['блогер']

print(repr(string1))
print(repr(string2))

'ахметзянов айрат'
'ахметзянов айрат'


In [282]:
# Сравним их
if repr(string1) == repr(string2):
    print("Строки идентичны")
else:
    print("Строки различаются")

Строки идентичны


> Отдельно стоит отметить, что даже если совпадают имена блогера, то бывают случаи, когда другие значения, в том числе по ценам, но чаще всего по столбцу "тематика", не совпадают, из-за чего мы не можем удалить дубликаты - учтём это в функции ниже

In [283]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
duplicates_bufer_sample = duplicates_bufer.loc[duplicates_bufer['блогер'] == 'бурмистрова татьяна']
duplicates_bufer_sample[['блогер', 'тематика']]

,блогер,тематика
250,бурмистрова татьяна,"Мама, лайфстайл"
418,бурмистрова татьяна,"Лайфстайл, семья, путешествия, развлечения, об..."


> Напишем функцию, которая будет заполненять пропущенные значения в DataFrame, объединенять уникальные значений в столбцах, а также удалять дубликаты строк и выполняет обработку столбцов на основе заданных правил, включая разделение значений и обновление 'тематики' для каждого блогера.

In [284]:
def duplicates_stroka(df: pd.DataFrame,
                      col_to_check: str,
                      columns_to_upd: str) -> pd.DataFrame:
    
    """
    Эта функция выполняет заполнение пропущенных значений в датафрейме, 
    объединение уникальных значений в столбцах, а также удаляет дубликаты строк и
    выполняет обработку столбцов на основе заданных правил, включая разделение значений и обновление 'тематики' для каждого блогера.
    
    
    ПАРАМЕТРЫ:
    df (DataFrame) - DataFrame, в котором мы будет делать преобразования
    col_to_check (str) - столбец, по которому мы будет смотреть дубликаты (чаще всего имя блогера)
    
    
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (DataFrame) - обновлённый DataFrame  
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    col_to_check = 'блогер'
    bufer = duplicates_bufer(bufer, col_to_check)
    """

    for col in df.columns:
        if col != col_to_check: # Пропускаем столбец, по которому выполняется проверка
            
            df['temp_fill'] = np.nan

# Создается временный столбец с именем 'temp_fill', который содержит последнее (по индексу)
# непустое значение из столбца col для каждой группы, сгруппированной по значениям в столбце col_to_check.
# Это выполняется с помощью bufer.groupby(col_to_check)[col].transform('last').
        
            df['temp_fill'] = df.groupby(col_to_check)[col].transform('last')  # Создание временного столбца

# Пропущенные значения в столбце col заполняются соответствующими значениями
# из временного столбца 'temp_fill' с использованием bufer[col].fillna(bufer['temp_fill']).
        
            df[col] = df[col].fillna(df['temp_fill'])  # Заполнение пропущенных значений

# Сбрасываются индексы в DataFrame bufer с помощью bufer.reset_index(drop=True). 
# Это делается для того, чтобы индексы начались с нуля после заполнения пропущенных значений.
            df = df.reset_index(drop=True)  # Сброс индексов

# Удаляется временный столбец 'temp_fill' с помощью bufer = bufer.drop(columns=['temp_fill']).    
            df = df.drop(columns=['temp_fill'])  # Удаление временного столбца

# # Удаляются дубликаты строк в DataFrame bufer с помощью bufer = bufer.drop_duplicates().    
#             df = df.drop_duplicates()
   
#     df = df.drop_duplicates()
    
    for column in df.columns:
        
        if 'тематика' in column:
            df[column] = df[column].astype(str)
            df[column] = df[column].fillna('')
            
            df[column] = df[column].str.strip()\
                                    .str.lower()\
                                    .str.replace('/', ', ')\
                                    .str.replace(r'\n', ', ')\
                                    .str.replace('*', '')\
                                    .str.replace(', ', ',')\
                                    .str.replace(',', ', ')
            df[column] = df[column].str.strip()
#             df[column] = df[column].str.split(', ')    
    
    for blogger in df[col_to_check].unique():
        blogger_rows = df[df[col_to_check] == blogger]
        unique_thematics = set()
        for column in blogger_rows.columns[blogger_rows.columns.str.contains(columns_to_upd)]:
            for value in blogger_rows[column].str.split(', ').dropna():
                unique_thematics.update(value)
        unique_thematics = ', '.join(sorted(unique_thematics))
        df.loc[df[col_to_check] == blogger, blogger_rows.columns[blogger_rows.columns.str.contains(columns_to_upd)]] = unique_thematics
    
    
    for column in df.columns:
        if columns_to_upd in column:    
            df[column] = df[column].str.lstrip('none, ')
#             df[column] = df[column].str.lstrip(', ')
            

    
    df = df.drop_duplicates()


    return df

In [285]:
# До 
bufer.shape

(452, 134)

> Сделаем преобразование

In [286]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [287]:
# После

bufer.shape

(429, 134)

> Мы убрали почти 40 дубликатов, кроме тех, где есть отличия в ценах

> Ниже видно, что заполнились все столбцы и объединились правильно по блогеру, а также заполнились все столбцы, где есть слово "тематика" суммарным списком тематик

In [288]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город


In [289]:
bufer.loc[bufer['блогер'] == 'бурмистрова татьяна']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram среднее кол-во просмотров серии сторис,Instagram стоимость серия сторис до 60 сек,Threads город,Threads ссылка,Threads тематика,Threads количество подписчиков,Threads стоимость фотопост,Threads стоимость видеопост,Threads блогер,Telegram город
250,бурмистрова татьяна,"лайфстайл, мама, обзоры, путешествия, развлече...",бурмистрова татьяна,https://instagram.com/tassi_stories,"лайфстайл, мама, обзоры, путешествия, развлече...",1 000 000,400 000,250 000,None,None,...,625 000,"р.600 000,00",None,None,"лайфстайл, мама, обзоры, путешествия, развлече...",None,None,None,None,Владивосток


### Подготовка таблицы insight_vk_ls

In [290]:
insight_vk_ls.head()

3,город,фамилия имя,ссылка,тематика,количество подписчиков,средние просмотры поста,средие просмотры клипа вк,стоимость за vk clip,стоимость за фотопост,стоимость за видеопост,"передача прав на paid social , ( 1 месяц)"
4,Москва,гренц полина,https://vk.com/grentsanilop,"Актриса, лайфстайл, бьюти, ЗОЖ",386 180,5 000,17 000,"р.70 000,00","р.60 000,00","р.70 000,00","р.10 000,00"
5,Казань,шарипов руслан,https://vk.com/sharipoffru,"Лайфстайл, юмор, развлечения, папа",357 000,15 000,15 000,"р.66 000,00","р.44 000,00","р.66 000,00","р.44 000,00"
6,Москва,данилова арина,https://vk.com/id230302965,"Тиктокер, артист, лайфстайл",251 000,10 000,20 000,"р.56 000,00","р.40 000,00","р.56 000,00",под запрос
7,Москва,курчанов евгений,https://vk.com/kuuurchanov,"Лайфстайл, контент-мейкер, развлечения",192 500,20 000,20 000,"р.50 000,00","р.45 000,00","р.50 000,00","р.30 000,00"
8,Нижний Новгород,федулова елизавета,https://vk.com/indigo_liza,"Лайфстайл, контент-мейкер, отношения",120 000,40 000,15 000,"р.38 400,00","р.80 000,00","р.38 400,00","р.53 333,00"


In [291]:
insight_vk_ls.rename(columns = {'фамилия имя':'блогер',
                                'средние просмотры поста':'среднее кол-во просмотров поста',
                                'средие просмотры клипа вк':'среднее кол-во просмотров вк клипа',
                                'стоимость за vk clip':'стоимость вк клип',
                                'стоимость за фотопост':'стоимость фотопост',
                                'стоимость за видеопост':'стоимость видеопост',
                                'передача прав на paid social , ( 1 месяц)':'стоимость передачи прав на 1 месяц'
                               }, inplace=True)

#### Проверка на дубликаты

In [292]:
show_duplicates(insight_vk_ls)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость вк клип,стоимость фотопост,стоимость видеопост,стоимость передачи прав на 1 месяц


### Пятнадцатое объединение

In [293]:
bufer = merge_bufer_df(bufer, insight_vk_ls, 'VK личная страница')

Количество дубликатов: 0


In [294]:
# До

bufer.shape

(463, 145)

In [295]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [296]:
# После

bufer.shape

(463, 145)

In [297]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница блогер,VK личная страница ссылка,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц


In [298]:
bufer[bufer['блогер'] == 'юлия ланге']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница блогер,VK личная страница ссылка,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц


### Подготовка таблицы insight_vk_group

In [299]:
insight_vk_group.head()

3,город,фамилия имя,ссылка,тематика,количество подписчиков,средние просмотры публикации,средние просмотры клипа вк,стоимость за vk clip,стоимость за фотопост,стоимость за видеопост,"передача прав на paid social , ( 1 месяц)"
4,Москва,лазарьянц карина,https://vk.com/krosskarina,"Развлечение, юмор, лайфстайл",1 400 000,230 000,500 000,"р.650 000,00","р.410 000,00","р.650 000,00",под запрос
5,Москва,манукян давид,https://vk.com/dava,"Артист, стример, лайфстайл, развлечения, кино,...",1 200 000,500 000,500 000,"р.850 000,00","р.680 000,00","р.850 000,00",под запрос
6,Тюмень,самчук елена,https://vk.com/samchuk_studio,"Развлечения, юмор, лайфстайл",801 000,800 000,600 000,"р.1 125 000,00","р.550 000,00",под запрос,"р.150 000,00"
7,Москва,ершов женя,https://vk.com/ershov_officiall,"Лайфстайл, контент-мейкер, развлечения",611 000,350 000,400 000,"р.340 000,00","р.240 000,00","р.340 000,00",под запрос
8,Москва,хилькевич анна,https://vk.com/khilkevich.public,"Актриса, лайфстайл, развлечения, юмор, семья",399 000,150 000,200 000,"р.500 000,00","р.350 000,00","р.500 000,00","р.130 000,00"


In [300]:
insight_vk_group.rename(columns = {'фамилия имя':'блогер',
                                   'средние просмотры публикации':'среднее кол-во просмотров поста',
                                   'средние просмотры клипа вк':'среднее кол-во просмотров вк клипа',
                                   'стоимость за vk clip':'стоимость вк клип',
                                   'стоимость за фотопост':'стоимость фотопост',
                                   'стоимость за видеопост':'стоимость видеопост',
                                   'передача прав на paid social , ( 1 месяц)':'стоимость передачи прав на 1 месяц'
                                  }, inplace=True)

#### Проверка на дубликаты

In [301]:
show_duplicates(insight_vk_group)

Количество дубликатов: 0


3,город,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость вк клип,стоимость фотопост,стоимость видеопост,стоимость передачи прав на 1 месяц


### Шестнадцатое объединение

In [302]:
bufer = merge_bufer_df(bufer, insight_vk_group, 'VK группа')

Количество дубликатов: 58


In [303]:
show_duplicates(bufer).head(6)

Количество дубликатов: 58


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц
472,байдавлетова анна,"Артист, певица, лайфстайл, мода, животные, бью...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Москва,"р.39 600,00"
380,байдавлетова анна,"артист, бьюти, животные, контент-мейкер, лайфс...",None,None,"артист, бьюти, животные, контент-мейкер, лайфс...",None,None,None,None,None,...,"артист, бьюти, животные, контент-мейкер, лайфс...",None,None,None,None,None,None,None,NaN,NaN
457,брыкун анна,дизайн,None,None,дизайн,None,None,None,None,None,...,дизайн,12 000,2 000,20 000,"р.39 690,00","р.11 340,00","р.39 690,00","р.7 560,00",NaN,NaN
502,брыкун анна,Дизайн,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тверь,"р.5 248,00"
500,буркова мария,"Семья, отношения, путешествия, лайфстайл",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тюмень,"р.32 000,00"
372,буркова мария,"дети, развлечения, семья",буркова мария,https://www.instagram.com/sinitsynamv/,"дети, развлечения, семья",98 000,30 000,13 000,None,None,...,"дети, развлечения, семья",None,None,None,None,None,None,None,NaN,NaN


In [304]:
# До

bufer.shape

(508, 147)

In [305]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [306]:
# После

bufer.shape

(479, 147)

In [307]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates.sort_values(by='блогер')

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница тематика,VK личная страница количество подписчиков,VK личная страница среднее кол-во просмотров поста,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц


### Подготовка таблицы insight_youtube

> Название канала

In [308]:
insight_youtube.head()

3,город,название канала,фамилия имя,ссылка,тематика,количество подписчиков,просмотры видео среднее кол-во,просмотры shorts среднее кол-во,стоимость преролла,стоимость интеграции (до 1 минуты),стоимость shorts,стоимость эксклюзивного ролика
4,Москва,dava,манукян давид,https://youtube.com/channel/UCbh7BxJWCMnWLwJ-t...,"Артист, стример, лайфстайл, развлечения, кино,...",5 440 000,450 000,-,р.700 000,"р.1 000 000,00",-,"р.1 360 000,00"
5,Санкт-Петербург,mamasobolina,соболева мария,https://www.youtube.com/channel/UCv6aQfBFfbepf...,"Семья, лайфстайл, дети, шортс-формат",5 200 000,60 000,50 000,-,-,"р.60 000,00","р.135 000,00"
6,Тверь,виола,кузьмина варвара,https://youtube.com/channel/UCyCKV6zM59D8yT24e...,"Юмор, развлечения (Шортс)",3 820 000,332 000,600 000,-,-,"р.90 000,00",-
7,Тверь,druemae,параманов андрей,https://youtube.com/channel/UC4WC9TkTTtnsVz6Ay...,"Шортс, развлечения, лайфстайл, юмор",3 740 000,110 000,120 000,"р.135 000,00","р.150 000,00","р.135 000,00","р.225 000,00"
8,Москва,dinara salimova,файзрахманова динара,https://www.youtube.com/c/DinaraSalimova/videos,"Развлечения, юмор, лайфстайл, шортс, обзоры",3 560 000,60 000,70 000,"р.45 000,00","р.75 000,00","р.67 500,00","р.135 000,00"


In [309]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция']

In [310]:
insight_youtube.rename(columns = {'фамилия имя':'блогер',
                                  'просмотры видео среднее кол-во':'среднее кол-во просмотров видео',
                                  'просмотры shorts среднее кол-во':'среднее кол-во просмотров shorts',
                                  'стоимость преролла':'стоимость преролл до 30 сек',
                                  'стоимость интеграции (до 1 минуты)':'стоимость интеграция 1 слот',
                                  'стоимость shorts':'стоимость за shorts до 1 мин',
                                  'стоимость эксклюзивного ролика':'стоимость выпуск под клиента'
                                 }, inplace=True)

#### Проверка на дубликаты

In [311]:
show_duplicates(insight_youtube)

Количество дубликатов: 2


3,город,название канала,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,стоимость преролл до 30 сек,стоимость интеграция 1 слот,стоимость за shorts до 1 мин,стоимость выпуск под клиента
19,Москва,kirill sarychev,сарычев кирилл,https://www.youtube.com/c/KIRILLSARYCHEV,"Спорт, борьба, развлечения, юмор, лайфстайл",1 390 000,400 000,-,"р.220 000,00","р.300 000,00",-,"р.1 200 000,00"
32,Москва,биполярка,сарычев кирилл,https://www.youtube.com/channel/UCmCyOwGrgqAPH...,"Спорт, борьба, развлечения, юмор, лайфстайл",269 000,120 000,-,"р.150 000,00","р.280 000,00",-,"р.500 000,00"


> Два канала сделаны как разные строки по одному блогеру - исправим это: переименуем вторую строчку на название канала - 'биполярка сарычева'

In [312]:
insight_youtube.loc\
    [insight_youtube['название канала'] == 'биполярка', 'блогер']  = 'биполярка сарычева'

In [313]:
# duplicates_insight_youtube = insight_youtube[insight_youtube.duplicated('блогер', keep=False)]
# duplicates_insight_youtube

# indexes_insight_youtube_sarychev = duplicates_insight_youtube_sarychev.index.tolist()

# indexes_insight_youtube_sarychev

# insight_youtube.drop(indexes_insight_youtube_sarychev, inplace=True)

# columns_with_sarychev = ['количество подписчиков',
#                          'среднее кол-во просмотров',
#                          'среднее кол-во просмотров shorts',
#                          'стоимость за преролл до 30 сек',
#                          'стоимость за интеграцию до 1 мин',
#                          'стоимость за shorts до 1 мин',
#                          'стоимость за выпуск под клиента',
#                          'блогер']

# # Удаление всех столбцов, кроме тех, что в списке columns_with_sarychev
# duplicates_insight_youtube_sarychev = duplicates_insight_youtube_sarychev[columns_with_sarychev]
# duplicates_insight_youtube_sarychev

In [314]:
show_duplicates(insight_youtube)

Количество дубликатов: 0


3,город,название канала,блогер,ссылка,тематика,количество подписчиков,среднее кол-во просмотров видео,среднее кол-во просмотров shorts,стоимость преролл до 30 сек,стоимость интеграция 1 слот,стоимость за shorts до 1 мин,стоимость выпуск под клиента


### Семнадцатое объединение

In [315]:
bufer = merge_bufer_df(bufer, insight_youtube, 'YouTube')

Количество дубликатов: 36


In [316]:
show_duplicates(bufer).head(6)

Количество дубликатов: 36


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин
274,бубнов денис,"контент-мейкер, лайфстайл, развлечения",None,None,"контент-мейкер, лайфстайл, развлечения",None,None,None,None,None,...,30 000,"р.46 771,00","р.25 984,00","р.46 771,00","р.17 323,00",None,None,NaN,NaN,NaN
491,бубнов денис,"Развлечения, юмор, лайфхаки, обзоры",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тула,корявый vlog,-
486,гогунская милана,"Лайфстайл, развлечения, юмор, артист, музыка, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Москва,milana star,"р.468 000,00"
311,гогунская милана,"артист, детство, лайфстайл, музыка, развлечени...",гогунская милана,https://instagram.com/milana__star,"артист, детство, лайфстайл, музыка, развлечени...",993 000,75 000,40 000,None,None,...,None,None,None,None,None,Москва,"р.51 000,00",NaN,NaN,NaN
487,горбунова надежда,Сериальные квесты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Барнаул,ная,"р.391 500,00"
377,горбунова надежда,"pov, лайфстайл, сериальные квесты, стиль",горбунова надежда,https://instagram.com/braiisa,"pov, лайфстайл, сериальные квесты, стиль",173 000,75 200,15 900,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN


In [317]:
# До

bufer.shape

(516, 150)

In [318]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [319]:
# После

bufer.shape

(498, 150)

In [320]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница среднее кол-во просмотров вк клипа,VK личная страница стоимость вк клип,VK личная страница стоимость фотопост,VK личная страница стоимость видеопост,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин


### Подготовка таблицы insight_zen

In [321]:
insight_zen.head()

3,город,фамилия,название канала,ссылка,категория,количество подписчиков,средний охват публикации,стоимость размещения за фотопост,стоимость размещения за видеопост
4,Калининград,самоха антон,антон самоха,https://dzen.ru/anton_samoha,"Юрист, адвокат",80 300,800,"р.4 950,00","р.9 240,00"
5,Казань,ахметзянов айрат,bistrie.recepty2.0,https://zen.yandex.ru/bistrierecepty,"Кулинария, еда, рецепты",67 200,3 000,"р.17 500,00","р.37 500,00"
6,Смоленск,бондарев алексей,леша аргумент,http://zen.yandex.ru/bitovuxa,"Развлечения, юмор",47 400,9 000,"р.24 000,00","р.36 000,00"
7,Санкт-Петербург,фокин валентин,валентин фокин,https://zen.yandex.ru/profile/editor/id/5ef6d4...,"Развлечения, пранки",44 300,500,"р.8 000,00","р.15 000,00"
8,Москва,беляничева анастасия,анабель шпилька,https://zen.yandex.ru/anabel_shpilka,Юмор,30 300,9 000,"р.24 000,00","р.36 000,00"


#### Обрежем DataFrame по высоте

> Так как мы загружаем данные с CSV, то мы видим не только данные, которые видят все, но и скрытые данные - обрежем данные, чтобы они соответствовали действительности

In [322]:
def trim_df(df: pd.DataFrame, columns: str, number: int) -> pd.DataFrame:
    
    """
    Данная функция trim_df предназначена для обработки и фильтрации данных в DataFrame,
    в которой мы обрезаем DataFrame по определённому размеру числа number в столбце columns

    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - DataFrame, который нужно обработать и фильтровать..
    columns (str) - строка, представляющая название столбца в DataFrame, который должен быть числовой или можно его перевести
    который будет обрабатываться, то есть порог, по которому мы обрезаем DataFrame.
    number (int) - целое число, которое используется для фильтрации данных в указанном столбце.
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - обновлённый DataFrame.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    insight_zen = trim_df(insight_zen, 'количество подписчиков', 999)
        
    """
    
    # Уберем неразрывные пробелы из значений столбца 'количество подписчиков'
    df[columns] = df[columns].str.replace('\xa0', '')
    df[columns] = df[columns].str.replace(' ', '')

    # Теперь преобразуем значения в числа с плавающей точкой
    df[columns] = df[columns].astype(float)

    #Обрежем DataFrame по-определённому размеру столбца columns.
    
    df = df[df[columns] >= number]
    
    #Переведём обратно в str, чтобы у нас не было проблем с объединением.

    df[columns] = df[columns].astype(str)
    
    return df

In [323]:
insight_zen = trim_df(insight_zen, 'количество подписчиков', 999)

In [324]:
[col for col in bufer.columns if 'Дзен' in col]

['Дзен блогер',
 'Дзен количество подписчиков',
 'Дзен среднее кол-во просмотров',
 'Дзен стоимость пост',
 'Дзен ссылка']

In [325]:
[col for col in insight_zen.columns]

['город',
 'фамилия',
 'название канала',
 'ссылка',
 'категория',
 'количество подписчиков',
 'средний охват публикации',
 'стоимость размещения за фотопост',
 'стоимость размещения за видеопост']

In [326]:
insight_zen.rename(columns = {'фамилия':'блогер',
                              'категория':'тематика',
                              'средний охват публикации':'среднее кол-во просмотров',
                              'стоимость размещения за фотопост':'стоимость пост',
                              'стоимость размещения за видеопост':'стоимость видеопост'
                             }, inplace=True)

#### Проверка на дубликаты

> У нас есть дубликаты, но помимо столбца "тематика", также отличаются  столбцы с количествами, а также с городом

In [327]:
show_duplicates(insight_zen)

Количество дубликатов: 6


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост
51,Москва,гущина,алена гущина.,https://zen.yandex.ru/id/6132025e71c8776747711662,"Развлечения, юмор",3600.0,5 000,р.9 000,р.20 000
52,Владимир,гущина,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,Развлечения,3600.0,3 000,р.3 750,р.10 000
29,Москва,гущина алена,viviannru,https://zen.yandex.ru/id/6132025e71c8776747711662,Развлечения,6600.0,3 000,"р.3 750,00","р.10 000,00"
37,Владимир,гущина алена,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"Развлечения, юмор",3500.0,5 000,"р.9 000,00","р.20 000,00"
27,Якутск,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,DIY,7600.0,400,"р.2 500,00","р.5 000,00"
28,Москва,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,Творчество,7300.0,300,"р.2 500,00","р.5 000,00"


> Обновим столбец "Тематика"

In [328]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
insight_zen = duplicates_stroka(insight_zen, col_to_check, columns_to_upd)

In [329]:
show_duplicates(insight_zen)

Количество дубликатов: 6


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост
47,Москва,гущина,алена гущина.,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,5 000,р.9 000,р.20 000
48,Владимир,гущина,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,3 000,р.3 750,р.10 000
25,Москва,гущина алена,viviannru,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",6600.0,3 000,"р.3 750,00","р.10 000,00"
33,Владимир,гущина алена,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3500.0,5 000,"р.9 000,00","р.20 000,00"
23,Якутск,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7600.0,400,"р.2 500,00","р.5 000,00"
24,Москва,лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7300.0,300,"р.2 500,00","р.5 000,00"


> Обновим столбец "Город"

In [330]:
col_to_check = 'блогер'
columns_to_upd = 'город'
insight_zen = duplicates_stroka(insight_zen, col_to_check, columns_to_upd)

In [331]:
show_duplicates(insight_zen)

Количество дубликатов: 6


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост
47,"Владимир, Москва",гущина,алена гущина.,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,5 000,р.9 000,р.20 000
48,"Владимир, Москва",гущина,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3600.0,3 000,р.3 750,р.10 000
25,"Владимир, Москва",гущина алена,viviannru,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",6600.0,3 000,"р.3 750,00","р.10 000,00"
33,"Владимир, Москва",гущина алена,алена гущина,https://zen.yandex.ru/id/6132025e71c8776747711662,"развлечения, юмор",3500.0,5 000,"р.9 000,00","р.20 000,00"
23,"Москва, Якутск",лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7600.0,400,"р.2 500,00","р.5 000,00"
24,"Москва, Якутск",лесниченко александра,"саня, которая luniyе",https://zen.yandex.ru/luniye,"diy, творчество",7300.0,300,"р.2 500,00","р.5 000,00"


> Удалим дубликаты, оставив только первое вхождение, так как чаще всего именно в них самые большие и видимо правильные значения

In [332]:
insight_zen = insight_zen.drop_duplicates(subset='блогер', keep='first')

In [333]:
show_duplicates(insight_zen)

Количество дубликатов: 0


3,город,блогер,название канала,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость пост,стоимость видеопост


### Восемнадцатое объединение

In [334]:
bufer = merge_bufer_df(bufer, insight_zen, 'Дзен')

Количество дубликатов: 22


In [335]:
show_duplicates(bufer).head(6)

Количество дубликатов: 22


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин,Дзен город,Дзен название канала,Дзен тематика,Дзен стоимость видеопост
369,ахметзянов айрат,"food , none, готовка, еда, рецепты, фудблогер",ахметзянов айрат,https://instagram.com/bistrie.recepty2.0,"food , none, готовка, еда, рецепты, фудблогер",256 000,120 000,48 000,None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
499,ахметзянов айрат,"еда, кулинария, рецепты",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Казань,bistrie.recepty2.0,"еда, кулинария, рецепты","р.37 500,00"
489,бондаренко ирина,"лайфстайл, развелечения, семья, юмор",None,None,"лайфстайл, развелечения, семья, юмор",None,None,None,None,None,...,None,None,None,Новосибирск,bondarinki,"р.30 000,00",NaN,NaN,NaN,NaN
507,бондаренко ирина,"лайфстайл, развелечения, семья",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Новосибирск,бондаринки,"лайфстайл, развелечения, семья","р.9 625,00"
452,горшков алексей,"лайфстайл, лайфхаки, обзоры, помощь людям, раз...",None,None,"лайфстайл, лайфхаки, обзоры, помощь людям, раз...",None,None,None,None,None,...,"р.14 001,00",None,None,Москва,леха медведь,-,NaN,NaN,NaN,NaN
526,горшков алексей,"лайфстайл, развлечения",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Москва,алексей медведь,"лайфстайл, развлечения","р.22 500,00"


In [336]:
# До

bufer.shape

(568, 154)

In [337]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [338]:
# После

bufer.shape

(557, 154)

In [339]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,VK личная страница стоимость передачи прав на 1 месяц,VK группа город,VK группа стоимость передачи прав на 1 месяц,YouTube город,YouTube название канала,YouTube стоимость за shorts до 1 мин,Дзен город,Дзен название канала,Дзен тематика,Дзен стоимость видеопост


### Подготовка таблицы insight_likee

In [340]:
insight_likee.head()

3,город,фамилия,имя,ссылка,тематика,количество подписчиков,средний просмотры публикации,стоимость за видеопост (60 сек.)
4,Нижний Новгород,федулова,елизавета,https://likee.video/@Lizzka_Strizzzz,"Лайфстайл, контент-мейкер, отношения",8 500 000,15 000,"р.13 500,00"
5,Москва,гогунская,милана,https://likee.video/@milana_star_offi,"Лайфстайл, развлечения, юмор, артист, музыка, ...",4 200 000,250 000,"р.200 000,00"
6,Уфа,мазитова,регина,https://l.likee.video/p/NWuFzx,"Неформалка, Юмор",3 400 000,50 000,"р.49 500,00"
7,Уфа,кашаева,динара,https://mobile.likeevideo.ru/p/jDTQ9q,"Лайфстайл, бьюти, POV",2 600 000,15 000,"р.13 500,00"
8,Тверь,кузьмина,варвара,https://l.likee.video/p/fSuROB,"Лайфстайл,юмор",1 100 000,120 000,"р.108 000,00"


In [341]:
insight_likee = name_familia(insight_likee)

In [342]:
[col for col in bufer.columns if 'TikTok' in col]

['TikTok блогер',
 'TikTok количество подписчиков',
 'TikTok среднее кол-во просмотров',
 'TikTok стоимость видеопост',
 'TikTok ссылка']

In [343]:
[col for col in insight_likee.columns]

['город',
 'ссылка',
 'тематика',
 'количество подписчиков',
 'средний просмотры публикации',
 'стоимость за видеопост (60 сек.)',
 'блогер']

In [344]:
insight_likee.rename(columns = {'средний просмотры публикации':'среднее кол-во просмотров',
                                'стоимость за видеопост (60 сек.)':'стоимость видеопост'
                               }, inplace=True)

#### Обрежем DataFrame по высоте

In [345]:
insight_likee = trim_df(insight_likee, 'количество подписчиков', 27000)

In [346]:
insight_likee.tail()

3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер
39,Великий Новгород,https://l.likee.video/p/8Gth4,POV,38800.0,15 000,"р.13 500,00",юрахно евгений
40,Саранск,https://l.likee.video/p/WDUdAG,"Лайфстайл,юмор",35000.0,20 000,"р.18 000,00",валеев артур
41,Тула,https://l.likee.video/p/rwpo7V,"POV, юмор, развлечения",34100.0,100 000,"р.90 000,00",шакиров эмиль
42,Махачкала,https://l.likee.video/p/svYssZ,"Развлечение,ПОВ",27400.0,10 000,"р.9 000,00",мирзаев сулейман
43,Тверь,https://l.likee.video/p/HVfP6y,"Лайфстайл, развлечения, юмор",27000.0,50 000,"р.45 000,00",самодуров андрей


#### Проверка на дубликаты

In [347]:
show_duplicates(insight_likee)

Количество дубликатов: 0


3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер


### Девятнадцатое объединение

In [348]:
bufer = merge_bufer_df(bufer, insight_likee, 'Likee')

Количество дубликатов: 0


In [349]:
# Посмотрим на размер DataFrame

bufer.shape

(579, 161)

### Подготовка таблицы insight_yappy

In [350]:
insight_yappy.head()

3,город,фамилия,имя,ссылка на профиль в yappy,тематика,количество подписчиков,среднее кол-во просмотров,стоимость за видеопост (60 сек.)
4,Москва,лазарьянц,карина,https://yappy.media/profile/25517079000644b88e...,"Развлечения, юмор, лайфстайл",308 600,5 000,"р.935 000,00"
5,Москва,хилькевич,анна,https://yappy.media/profile/eedaea1dc6f948ebb0...,"Лайфстайл, мама, актриса",243 300,60 000,"р.510 000,00"
6,Ростов-на-Дону,кобзарева,лаура,https://yappy.media/s/p_6nnXi2kGciIS2awRqfvKuY,"Танцы, развлечения, лайфстайл",239 600,10 000,"р.12 500,00"
7,Москва,акилова,алина,https://yappy.media/profile/70134d60962b485c90...,"Лайфстайл, семья, бьюти",123 500,11 000,"р.99 000,00"
8,Самара,графинина,елизавета,https://yappy.media/s/p_1UqVXvm0yaJqruortHOtba,"Животные, лайфстайл",82 400,8 000,"р.10 000,00"


In [351]:
insight_yappy = name_familia(insight_yappy)

In [352]:
[col for col in bufer.columns if 'TikTok' in col]

['TikTok блогер',
 'TikTok количество подписчиков',
 'TikTok среднее кол-во просмотров',
 'TikTok стоимость видеопост',
 'TikTok ссылка']

In [353]:
[col for col in insight_yappy.columns]

['город',
 'ссылка на профиль в yappy',
 'тематика',
 'количество подписчиков',
 'среднее кол-во просмотров',
 'стоимость за видеопост (60 сек.)',
 'блогер']

In [354]:
insight_yappy.rename(columns = {'ссылка на профиль в yappy':'ссылка',
                                'стоимость за видеопост (60 сек.)':'стоимость видеопост',
                               }, inplace=True)

#### Обрежем DataFrame по высоте

In [355]:
insight_yappy = trim_df(insight_yappy, 'количество подписчиков', 3000)

In [356]:
insight_yappy.tail()

3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер
75,Брянск,https://yappy.media/s/p_55liVoClPxs7XJBQo2JO8c,"Лайфстайл, юмор, отношения",4400.0,10 000,"р.12 500,00",абрамочкина дарья
76,Омск,https://yappy.media/profile/2284c0fbef8345deba...,"Юмор, развлечения, POV",4200.0,10 000,"р.12 500,00",сурыкин сергей
77,Москва,https://yappy.media/profile/d888416bb4404b01b6...,"Лайфстайл, POV, юмор, развлечения",4200.0,6 000,"р.7 500,00",габай янина
78,Владимир,https://yappy.media/profile/1abc9f5e7d4c444987...,"Лайфстайл, бьюти, юмор",4000.0,5 000,"р.6 250,00",бебчук екатерина
79,Уфа,https://yappy.media/profile/539f87e6fa6c4afda4...,Астролог,3000.0,5 000,"р.6 250,00",ахунова гульназ


#### Проверка на дубликаты

In [357]:
show_duplicates(insight_yappy)

Количество дубликатов: 0


3,город,ссылка,тематика,количество подписчиков,среднее кол-во просмотров,стоимость видеопост,блогер


### Двадцатое объединение

In [358]:
bufer = merge_bufer_df(bufer, insight_yappy, 'Yappy')

Количество дубликатов: 0


### Подготовка таблицы insight_rutube

> У нас есть проблема с именем блогера: оно сделано как **ФИО**, а нам нужно **имя и фамилия**, чтобы правильно объединить по строкам - сделаем преобразования, чтобы подвести к правильному формату

In [359]:
insight_rutube.head()

3,город,фио,ссылка,количество подписчиков,средние просмотры за месяц,стоимость преролла,стоимость интеграции (до 1 минуты)
4,Тверь,аркатов станислав александрович,https://rutube.ru/channel/23684374/,Фудблогер,118 300,25 030,"р.50 060,00"
5,Краснодар,солодов юрий юрьевич,https://rutube.ru/channel/23722477/,"Лайфстайл, влог",24 500,47 000,"р.82 250,00"
6,Казань,ахмадуллин наиль радикович,https://rutube.ru/channel/23694775/,"Путешествия, лайфстайл",18 500,27 524,под запрос
7,Воронеж,рязанова татьяна алексеевна,https://rutube.ru/channel/19363597/,"Бьюти, лайфстайл",16 500,50 066,"р.87 615,00"
8,Воронеж,лаврова валерия владимировна,https://rutube.ru/channel/26070201/,Художник,15 600,40 063,"р.70 115,00"


In [360]:
def manipulate_text(text: str) -> str:
    words = text.split()  # Разделение строки на слова
    if len(words) >= 3:
        words.pop(2)  # Удаление третьего слова
    return ' '.join(words)  # Объединение слов обратно в строку

In [361]:
insight_rutube['фио'] = insight_rutube['фио'].apply(manipulate_text).str.strip()

In [362]:
[col for col in insight_rutube.columns]

['город',
 'фио',
 'ссылка',
 'количество подписчиков',
 'средние просмотры за месяц',
 'стоимость преролла',
 'стоимость интеграции (до 1 минуты)']

In [363]:
[col for col in insight_youtube.columns]

['YouTube город',
 'YouTube название канала',
 'YouTube блогер',
 'YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость за shorts до 1 мин',
 'YouTube стоимость выпуск под клиента']

In [364]:
insight_rutube.rename(columns = {'фио':'блогер',
                                 'средние просмотры за месяц':'среднее кол-во просмотров видео',
                                 'стоимость преролла':'стоимость преролл до 30 сек',
                                 'стоимость интеграции (до 1 минуты)':'стоимость интеграция 1 слот',
                                 'количество подписчиков':'тематика'
                                }, inplace=True)

#### Обрежем DataFrame по высоте

In [365]:
insight_rutube.head()

3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
4,Тверь,аркатов станислав,https://rutube.ru/channel/23684374/,Фудблогер,118 300,25 030,"р.50 060,00"
5,Краснодар,солодов юрий,https://rutube.ru/channel/23722477/,"Лайфстайл, влог",24 500,47 000,"р.82 250,00"
6,Казань,ахмадуллин наиль,https://rutube.ru/channel/23694775/,"Путешествия, лайфстайл",18 500,27 524,под запрос
7,Воронеж,рязанова татьяна,https://rutube.ru/channel/19363597/,"Бьюти, лайфстайл",16 500,50 066,"р.87 615,00"
8,Воронеж,лаврова валерия,https://rutube.ru/channel/26070201/,Художник,15 600,40 063,"р.70 115,00"


In [366]:
insight_rutube = trim_df(insight_rutube, 'среднее кол-во просмотров видео', 3299)
insight_rutube.tail()

3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
96,Москва,лавренюк евгений,https://rutube.ru/channel/23662042/,5 348,41352.0,NaN,NaN
97,Москва,гуслистов михаил,https://rutube.ru/channel/23685098/,5 346,43213.0,NaN,NaN
98,Краснодар,колодник дарья,https://rutube.ru/channel/23706945/,5 327,30000.0,NaN,NaN
99,Владимир,шуклецова анастасия,https://rutube.ru/channel/9570650/,5 199,65385.0,NaN,NaN
116,Тверь,садикова заира,https://rutube.ru/channel/23712253/,22,4000.0,NaN,NaN


In [367]:
insight_rutube = insight_rutube.loc[0: 70]

In [368]:
insight_rutube.tail(15)

3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
56,Воронеж,бабарин алексей,https://rutube.ru/channel/23705479/,"Лайфстайл, влог",4800.0,2 563,"р.4 490,00"
57,Москва,заричная анастасия,https://rutube.ru/channel/7456516/,Лайфстайл,4600.0,7 890,под запрос
58,Краснодар,бойко мария,https://rutube.ru/channel/23724373/,"Лайфстайл, путешествия",4600.0,41 000,"р.71 750,00"
59,Кемерово,наумов данил,https://rutube.ru/channel/23663217/,Врач,4300.0,10 988,"р.19 233,00"
60,Ярославль,якушевич вячеслав,https://rutube.ru/channel/23467085/,"Лайфстайл, фудблогер",4265.0,38 444,"р.67 280,00"
61,Ростов-на-Дону,курдус дмитрий,https://rutube.ru/channel/23684825/,Геймер,4200.0,2 440,"р.4 270,00"
62,Волгоград,хамидова мария,https://rutube.ru/channel/6882983/,Фудблогер,3900.0,37 829,"р.87 500,00"
63,Воронеж,омаров гаджимурад,https://rutube.ru/channel/12999482/,"Лайфстайл, влог",3800.0,10 022,"р.17 535,00"
64,Волгоград,трубчанинова раиса,https://rutube.ru/channel/23706142/,"Лайфстайл, семья",3700.0,15 095,под запрос
65,Пенза,захватова людмила,https://rutube.ru/channel/23650043/,Спорт,3600.0,1 942,"р.3 395,00"


#### Проверка на дубликаты

In [369]:
show_duplicates(insight_rutube)

Количество дубликатов: 2


3,город,блогер,ссылка,тематика,среднее кол-во просмотров видео,стоимость преролл до 30 сек,стоимость интеграция 1 слот
52,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,Фудблогер,5200.0,7 389,под запрос
70,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,6 278,14235.0,под запрос,под запрос


### Двадцать первое объединение

In [370]:
bufer = merge_bufer_df(bufer, insight_rutube, 'RuTube')

Количество дубликатов: 2


### Подготовка таблицы insight_ok

In [371]:
# insight_ok.head()

In [372]:
# insight_ok = name_familia(insight_ok)

In [373]:
# [col for col in bufer.columns if 'OK' in col]

In [374]:
# [col for col in insight_ok.columns]

#### Обрежем DataFrame по высоте

In [375]:
# insight_ok = trim_df(insight_ok, 'количество подписчиков', 53)

#### Проверка на дубликаты

In [376]:
# show_duplicates(insight_ok)

### Двадцать второе объединение

In [377]:
# bufer = merge_bufer_df(bufer, insight_ok, 'Одноклассники')

> Запустим функцию duplicates_stroka, чтобы обновить столбец bufer

In [378]:
# col_to_check = 'блогер'
# columns_to_upd = 'тематика'
# bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

> Посмотрим на размер DataFrame bufer на данный момент

In [379]:
# bufer.shape

> Заполним столбец контакты и налог

In [380]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('info@insightpeople.ru', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

## Подготовка данных по блогерам от NextUp

In [381]:
nextup_df = [nextup_instagram,
             nextup_telegram,
             nextup_youtube,
             nextup_twitch,
             nextup_vk]

In [382]:
key_word = ['CPV', ]  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Имя блогера']

In [383]:
for df in nextup_df:
    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы nextup_instagram

In [384]:
nextup_instagram.head()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,тематика,ца по городам,ца по возрасту,ца по полу,охват фотопостов за 30 дней,охват reels за 30 дней,охват stories за 14 дней,стоимость фотопост,стоимость reels,стоимость серия stories (3 шт. по 15 сек.),статистика ссылка
7,NaN,П,алла брулетова,https://instagram.com/bruletova_,1 960 000,статистика,"Фешн, Лайфстайл, Бьюти","Москва – 4,5%\nСПб – 1,2%","13–17 лет – 4,4%\n18–24 года – 29,7%\n25–34 го...","Мужской – 77,3%\nЖенский – 22,6%",750 000,1 200 000,110 000,430 000 ₽,500 000 ₽,150 000 ₽,https://disk.yandex.ru/d/Os8m8Vm5zfygPg
8,NaN,Э,настя badbaarbie,https://instagram.com/badbaarbie,1 850 000,статистика,"Фешн, Лайфстайл, Бьюти","Москва – 8,2%\nСПб – 2,6%","13–17 лет – 19,5%\n18–24 года – 49,5%\n25–34 г...","Мужской – 31,4%\nЖенский – 68,5%",200 000,380 000,75 000,360 000 ₽,286 000 ₽,145 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,NaN,Э,эвелина лунаком,https://instagram.com/lyna_com2000,1 700 000,статистика,"Лайфстайл, Бьюти","Москва – 16,9%\nСПб – 5%","13–17 лет – 14,6%\n18–24 года – 62,2%\n25–34 г...","Мужской – 24,4%\nЖенский – 75,5%",500 000,1 000 000,250 000,572 000 ₽,930 000 ₽,500 000 ₽,https://drive.google.com/drive/u/0/folders/1F0...
10,NaN,П,саша трусова,https://www.instagram.com/avtrusova,1 250 000,статистика,"Спорт, Бьюти","Москва – 9%\nСПб – 3,4%","13–17 лет – 14,3%\n18–24 года – 39,2%\n25–34 г...","Мужской – 23%\nЖенский – 76,9%",650 000,500 000,130 000,430 000 ₽,572 000 ₽,358 000 ₽,https://drive.google.com/drive/folders/1BdjD3x...
11,NaN,Э,ира (inst_rinna),https://instagram.com/inst_rinna,1 000 000,статистика,"Танцы, Лайфстайл, Бьюти","Москва – 17,4%\nСПб – 5,3%","13–17 лет – 17,7%\n18–24 года – 62,2%\n25–34 г...","Мужской – 53%\nЖенский – 46,9%",650 000,250 000,230 000,340 000 ₽,350 000 ₽,280 000 ₽,https://drive.google.com/drive/u/0/folders/1xT...


In [385]:
def nextup_col_upd(df: pd.DataFrame, nextup_columns_upd: list) -> pd.DataFrame:
    
    """
    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - .
    nextup_columns_upd (list) - .
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - DataFrame с обновлёнными столбцами nextup_columns_upd.
    
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    """
    

#     for col in df.columns:
#         df[nextup_columns_upd] = df[nextup_columns_upd].apply(lambda x: str(x).lower())\
#                                                         .str.replace('/\n', ', ')\
#                                                         .str.replace('\n', ', ')\
#                                                         .str.replace('*', '')\
#                                                         .str.strip()
        
    for col in nextup_columns_upd:
        df[col] = df[col].apply(lambda x: str(x).lower())\
                        .str.replace('\n', ', ')\
                        .str.replace('*', '')\
                        .str.strip()
        
    return df

In [386]:
nextup_instagram_columns_upd = ['тематика', 'ца по городам', 'ца по возрасту', 'ца по полу']
nextup_instagram = nextup_col_upd(nextup_instagram, nextup_instagram_columns_upd)

In [387]:
nextup_instagram.head()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,тематика,ца по городам,ца по возрасту,ца по полу,охват фотопостов за 30 дней,охват reels за 30 дней,охват stories за 14 дней,стоимость фотопост,стоимость reels,стоимость серия stories (3 шт. по 15 сек.),статистика ссылка
7,NaN,П,алла брулетова,https://instagram.com/bruletova_,1 960 000,статистика,"фешн, лайфстайл, бьюти","москва – 4,5%, спб – 1,2%","13–17 лет – 4,4%, 18–24 года – 29,7%, 25–34 го...","мужской – 77,3%, женский – 22,6%",750 000,1 200 000,110 000,430 000 ₽,500 000 ₽,150 000 ₽,https://disk.yandex.ru/d/Os8m8Vm5zfygPg
8,NaN,Э,настя badbaarbie,https://instagram.com/badbaarbie,1 850 000,статистика,"фешн, лайфстайл, бьюти","москва – 8,2%, спб – 2,6%","13–17 лет – 19,5%, 18–24 года – 49,5%, 25–34 г...","мужской – 31,4%, женский – 68,5%",200 000,380 000,75 000,360 000 ₽,286 000 ₽,145 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,NaN,Э,эвелина лунаком,https://instagram.com/lyna_com2000,1 700 000,статистика,"лайфстайл, бьюти","москва – 16,9%, спб – 5%","13–17 лет – 14,6%, 18–24 года – 62,2%, 25–34 г...","мужской – 24,4%, женский – 75,5%",500 000,1 000 000,250 000,572 000 ₽,930 000 ₽,500 000 ₽,https://drive.google.com/drive/u/0/folders/1F0...
10,NaN,П,саша трусова,https://www.instagram.com/avtrusova,1 250 000,статистика,"спорт, бьюти","москва – 9%, спб – 3,4%","13–17 лет – 14,3%, 18–24 года – 39,2%, 25–34 г...","мужской – 23%, женский – 76,9%",650 000,500 000,130 000,430 000 ₽,572 000 ₽,358 000 ₽,https://drive.google.com/drive/folders/1BdjD3x...
11,NaN,Э,ира (inst_rinna),https://instagram.com/inst_rinna,1 000 000,статистика,"танцы, лайфстайл, бьюти","москва – 17,4%, спб – 5,3%","13–17 лет – 17,7%, 18–24 года – 62,2%, 25–34 г...","мужской – 53%, женский – 46,9%",650 000,250 000,230 000,340 000 ₽,350 000 ₽,280 000 ₽,https://drive.google.com/drive/u/0/folders/1xT...


> Удалим первые два столбца, которые не несут в себе полезной информации, а так как нам это надо будет делать ещё у нескольких DataFrame, то сделаем функцию

In [388]:
def nextup_columns_to_drop(df: pd.DataFrame,
                           columns_to_drop: list,
                           statistika: bool) -> pd.DataFrame:
    
    """
    Функция удаляет определённые столбцы по индексу, которые мы передаём в листе columns_to_drop.
    Также у нас есть флаг statistika, и если стоит True, то также удаляется столбец "статистика"
    
    ПАРАМЕТРЫ:
    df (pd.DataFrame) - DataFrame, который мы редактируем.
    columns_to_drop: (list) - список с индексами столбцов для удаления.
    statistika (bool) - флаг для удаления столбца "Статистика".
 
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    df (pd.DataFrame) - обновлённый DataFrame.
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    columns_to_drop = [0, 1]
    nextup_instagram = nextup_columns_to_drop(nextup_instagram, columns_to_drop, True)
    
    
    """
    df = df.drop(df.columns[columns_to_drop], axis=1)
    
    if statistika:
        df.drop(columns=['статистика'], axis=1, inplace=True)
    
    return df  # Вы должны вернуть измененный DataFrame

In [389]:
columns_to_drop = [0, 1]
nextup_instagram = nextup_columns_to_drop(nextup_instagram, columns_to_drop, True)

In [390]:
nextup_instagram.head()

6,имя блогера,ссылка,кол-во подписчиков,тематика,ца по городам,ца по возрасту,ца по полу,охват фотопостов за 30 дней,охват reels за 30 дней,охват stories за 14 дней,стоимость фотопост,стоимость reels,стоимость серия stories (3 шт. по 15 сек.),статистика ссылка
7,алла брулетова,https://instagram.com/bruletova_,1 960 000,"фешн, лайфстайл, бьюти","москва – 4,5%, спб – 1,2%","13–17 лет – 4,4%, 18–24 года – 29,7%, 25–34 го...","мужской – 77,3%, женский – 22,6%",750 000,1 200 000,110 000,430 000 ₽,500 000 ₽,150 000 ₽,https://disk.yandex.ru/d/Os8m8Vm5zfygPg
8,настя badbaarbie,https://instagram.com/badbaarbie,1 850 000,"фешн, лайфстайл, бьюти","москва – 8,2%, спб – 2,6%","13–17 лет – 19,5%, 18–24 года – 49,5%, 25–34 г...","мужской – 31,4%, женский – 68,5%",200 000,380 000,75 000,360 000 ₽,286 000 ₽,145 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,эвелина лунаком,https://instagram.com/lyna_com2000,1 700 000,"лайфстайл, бьюти","москва – 16,9%, спб – 5%","13–17 лет – 14,6%, 18–24 года – 62,2%, 25–34 г...","мужской – 24,4%, женский – 75,5%",500 000,1 000 000,250 000,572 000 ₽,930 000 ₽,500 000 ₽,https://drive.google.com/drive/u/0/folders/1F0...
10,саша трусова,https://www.instagram.com/avtrusova,1 250 000,"спорт, бьюти","москва – 9%, спб – 3,4%","13–17 лет – 14,3%, 18–24 года – 39,2%, 25–34 г...","мужской – 23%, женский – 76,9%",650 000,500 000,130 000,430 000 ₽,572 000 ₽,358 000 ₽,https://drive.google.com/drive/folders/1BdjD3x...
11,ира (inst_rinna),https://instagram.com/inst_rinna,1 000 000,"танцы, лайфстайл, бьюти","москва – 17,4%, спб – 5,3%","13–17 лет – 17,7%, 18–24 года – 62,2%, 25–34 г...","мужской – 53%, женский – 46,9%",650 000,250 000,230 000,340 000 ₽,350 000 ₽,280 000 ₽,https://drive.google.com/drive/u/0/folders/1xT...


In [391]:
[col for col in nextup_instagram.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'тематика',
 'ца по городам',
 'ца по возрасту',
 'ца по полу',
 'охват фотопостов за 30 дней',
 'охват reels за 30 дней',
 'охват stories за 14 дней',
 'стоимость фотопост',
 'стоимость reels',
 'стоимость серия stories (3 шт. по 15 сек.)',
 'статистика ссылка']

In [392]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'Instagram город',
 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек']

In [393]:
nextup_instagram.rename(columns = {'имя блогера':'блогер',
                                   'кол-во подписчиков':'количество подписчиков',
                                   'охват фотопостов за 30 дней':'среднее кол-во просмотров фотопоста',
                                   'охват reels за 30 дней':'среднее кол-во просмотров reels / видеопоста',
                                   'охват stories за 14 дней':'среднее кол-во просмотров 1 сторис',
                                   'стоимость reels':'стоимость reels / видеопост',
                                   'стоимость серия stories (3 шт. по 15 сек.)':'стоимость серия из 3х сторис до 45 сек',
                                   'статистика ссылка':'статистика'
                                  }, inplace=True)

#### Проверка на дубликаты

In [394]:
show_duplicates(nextup_instagram)

Количество дубликатов: 0


6,блогер,ссылка,количество подписчиков,тематика,ца по городам,ца по возрасту,ца по полу,среднее кол-во просмотров фотопоста,среднее кол-во просмотров reels / видеопоста,среднее кол-во просмотров 1 сторис,стоимость фотопост,стоимость reels / видеопост,стоимость серия из 3х сторис до 45 сек,статистика


### Двадцать третье объединение

In [395]:
bufer = merge_bufer_df(bufer, nextup_instagram, 'Instagram')

Количество дубликатов: 4


In [396]:
show_duplicates(bufer).head(6)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
447,лиза барашик,"бьюти, влоги, лайфстайл",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл",373 000,100 000,55 000,100 000,20 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
699,лиза барашик,"лайфстайл, бьюти, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"лайфстайл, бьюти, путешествия",376 000,100 000,55 000,250 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
356,ознобишин дмитрий,"рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,Фудблогер,5200.0,7 389,под запрос,NaN,NaN,NaN
357,ознобишин дмитрий,"рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,6 278,14235.0,под запрос,под запрос,NaN,NaN,NaN


> Обновим столбец "статистика"

In [397]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [398]:
show_duplicates(bufer).head(6)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
447,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
699,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",376 000,100 000,55 000,250 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
356,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None
357,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None


> Лиза Барашик на эксклюзиве у агенства DIDENOK, поэтому мы должны оставить именно строчку с ними, то есть первую

In [399]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
duplicates_bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
356,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None
357,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,Санкт-Петербург,ознобишин дмитрий,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None
447,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"
699,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",376 000,100 000,55 000,250 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"


In [400]:
max_index = duplicates_bufer.index.max()
bufer.drop(max_index, inplace=True)

In [401]:
bufer[bufer['блогер'] == 'лиза барашик']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube город,RuTube блогер,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу
447,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,None,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%"


### Подготовка таблицы nextup_telegram

In [402]:
nextup_telegram.head()

5,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,охват поста за 30 дней,стоимость текстовый пост,стоимость фотопост,стоимость видеокружок,NaN
6,NaN,П,леон (слово пацана),https://t.me/iminlovewleo,868 358,статистика,750 000,1 100 000 ₽,1 200 000 ₽,900 000 ₽,Статистика ссылка
7,NaN,П,илья exile,https://t.me/exilemusic13,778 550,статистика,650 000,700 000 ₽,860 000 ₽,700 000 ₽,https://disk.yandex.ru/d/ajGlE9Fx7SrxGw
8,NaN,П,саша парадеевич,https://t.me/paradeevich,665 324,статистика,500 000,1 200 000 ₽,1 218 000 ₽,1 200 000 ₽,https://disk.yandex.ru/d/FiihzOmrftH8WQ
9,NaN,П,леша кореш,https://t.me/koreshzy,497 989,статистика,250 000,360 000 ₽,400 000 ₽,360 000 ₽,https://disk.yandex.ru/d/eJbfo3TwOAKkHw
10,NaN,П,костя плохой парень,https://t.me/plohoypacan,400 634,статистика,280 000,—,—,—,https://disk.yandex.ru/d/dIhdb9_BG6ldmw


In [403]:
nextup_telegram = nextup_telegram.dropna(subset=['имя блогера'])

In [404]:
nextup_telegram.tail()

5,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,охват поста за 30 дней,стоимость текстовый пост,стоимость фотопост,стоимость видеокружок,NaN
59,NaN,Э,даша (polovinkago),https://t.me/polovinkago,8 047,статистика,2 700,7 000 ₽,10 000 ₽,6 000 ₽,https://drive.google.com/drive/u/0/folders/1YQ...
60,NaN,Э,валерия (vafom),https://t.me/vafombase,7 052,статистика,3 000,8 000 ₽,10 000 ₽,7 150 ₽,https://disk.yandex.ru/d/ag6ldz_lYcHMtA
61,NaN,П,саша маслова,https://t.me/sashamaslova,6 766,статистика,2 800,15 000 ₽,19 000 ₽,15 000 ₽,https://disk.yandex.ru/d/cMifIiN2pWcSAQ
62,NaN,Э,марина ясно,https://t.me/yyyyyasno,4 563,по запросу,5 000,7 000 ₽,10 000 ₽,6 000 ₽,NaN
63,NaN,П,даша (lovesomemama),https://t.me/lovesomechannel,2 689,статистика,1 200,10 000 ₽,12 000 ₽,17 000 ₽,https://disk.yandex.ru/d/MOEcnIl-sKYc2w


In [405]:
columns_to_drop = [0, 1]
nextup_telegram = nextup_columns_to_drop(nextup_telegram, columns_to_drop, True)

In [406]:
[col for col in nextup_telegram.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'охват поста за 30 дней',
 'стоимость текстовый пост',
 'стоимость фотопост',
 'стоимость видеокружок']

In [407]:
[col for col in bufer.columns if 'Telegram' in col]

['Telegram блогер',
 'Telegram ссылка',
 'Telegram тематика',
 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',
 'Telegram статистика',
 'Telegram город']

In [408]:
nextup_telegram.rename(columns = {'имя блогера':'блогер',
                                  'кол-во подписчиков':'количество подписчиков',
                                  'охват поста за 30 дней':'среднее кол-во просмотров поста',
                                  'стоимость текстовый пост (24 часа)':'стоимость текстовое сообщение без фото',
                                  'стоимость фотопост (24 часа)':'стоимость фотопост',
                                  'стоимость видеокружок (24 часа)':'стоимость кружочек до 60 сек',
                                  'статистика ссылка':'статистика'
                                 }, inplace=True)

#### Проверка на дубликаты

In [409]:
show_duplicates(nextup_telegram)

Количество дубликатов: 0


5,блогер,ссылка,количество подписчиков,среднее кол-во просмотров поста,стоимость текстовый пост,стоимость фотопост,стоимость видеокружок


### Двадцать четвёртое объединение

In [410]:
bufer = merge_bufer_df(bufer, nextup_telegram, 'Telegram')

Количество дубликатов: 84


In [411]:
show_duplicates(bufer).head(6)

Количество дубликатов: 84


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок
767,алина олешева,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78 500 ₽,71 500 ₽
584,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",624 000,300 000,110 000,260 000,None,...,None,"лайфстайл, музыка",None,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%",NaN,NaN
575,алла брулетова,"бьюти, лайфстайл, фешн",алла брулетова,https://instagram.com/bruletova_,"бьюти, лайфстайл, фешн",1 960 000,750 000,110 000,1 200 000,None,...,None,"бьюти, лайфстайл, фешн",None,None,None,"москва – 4,5%, спб – 1,2%","13–17 лет – 4,4%, 18–24 года – 29,7%, 25–34 го...","мужской – 77,3%, женский – 22,6%",NaN,NaN
766,алла брулетова,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70 000 ₽,60 000 ₽
765,алёна эксекверт,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 000 ₽,20 000 ₽
600,алёна эксекверт,"бьюти, лайфстайл",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл",247 000,90 000,35 000,200 000,None,...,None,"бьюти, лайфстайл",None,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%",NaN,NaN


In [412]:
# До

bufer.shape

(796, 180)

In [413]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [414]:
# После

bufer.shape

(756, 180)

In [415]:
show_duplicates(bufer)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок
722,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%",60 000 ₽,75 000 ₽
758,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%",60 000 ₽,75 000 ₽
289,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None,None,None
290,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None,None,None


In [416]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок
289,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",5200.0,7 389,под запрос,None,None,None,None,None
290,ознобишин дмитрий,"6 278, none, рецепты, фудблогер",ознобишин дмитрий,https://instagram.com/chef_dmitriy_official,"6 278, none, рецепты, фудблогер",417 000,45 000,18 000,None,None,...,https://rutube.ru/channel/1591109/,"6 278, none, рецепты, фудблогер",14235.0,под запрос,под запрос,None,None,None,None,None
722,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%",60 000 ₽,75 000 ₽
758,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%",60 000 ₽,75 000 ₽


In [417]:
telegram_columns = duplicates.filter(like='Telegram')
telegram_columns

,Telegram блогер,Telegram ссылка,Telegram тематика,Telegram количество подписчиков,Telegram среднее кол-во просмотров поста,Telegram средняя вовлечённость,Telegram стоимость фотопост,Telegram стоимость видеопост,Telegram стоимость кружочек до 60 сек,Telegram стоимость текстовое сообщение без фото,Telegram статистика,Telegram город,Telegram стоимость текстовый пост,Telegram стоимость видеокружок
289,None,None,"6 278, none, рецепты, фудблогер",None,None,None,None,None,None,None,None,None,None,None
290,None,None,"6 278, none, рецепты, фудблогер",None,None,None,None,None,None,None,None,None,None,None
722,лиза барашик,https://t.me/barrashikliza,"бьюти, влоги, лайфстайл, путешествия",82 000,17 000,-,р.60 000,р.105 000,р.75 000,р.40 000,https://drive.google.com/drive/folders/1MoeDga...,None,60 000 ₽,75 000 ₽
758,лиза барашик,https://t.me/barrashikliza,"бьюти, влоги, лайфстайл, путешествия",86 517,50 000,-,72 000 ₽,р.105 000,р.75 000,р.40 000,https://drive.google.com/drive/folders/1MoeDga...,None,60 000 ₽,75 000 ₽


> Оставим только первое значение данного блогера, так как блогер "лиза барашик" на эксклюзиве у DidenokTeam

In [418]:
duplicates_bufer = bufer[bufer.duplicated('блогер', keep=False)]
max_index = duplicates.index.max()
bufer.drop(max_index, inplace=True)

In [419]:
bufer[bufer['блогер'] == 'лиза барашик']

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube ссылка,RuTube тематика,RuTube среднее кол-во просмотров видео,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок
722,лиза барашик,"бьюти, влоги, лайфстайл, путешествия",лиза барашик,https://www.instagram.com/barrashik/,"бьюти, влоги, лайфстайл, путешествия",373 000,100 000,55 000,100 000,20 000,...,None,"бьюти, влоги, лайфстайл, путешествия",None,None,None,"москва – 22,4% спб – 8,6%","13–17 лет – 21,7% 18–24 года – 64,8% 25–34 год...","мужской – 7,6% женский – 92,3%",60 000 ₽,75 000 ₽


### Подготовка таблицы nextup_youtube

In [420]:
nextup_youtube.head()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,тематика,ца по возрасту,ца по полу,ца по странам,охват видео за 30 дней,стоимость интеграция / 1 слот,стоимость интеграция / 2 слот,стоимость преролл,NaN
7,NaN,П,илья exile,https://www.youtube.com/@ExileShow,4 350 000,статистика,Развлекательное,13–17 лет – 6%;\n18–24 года – 61%;\n25–34 года...,Мужской – 76%\nЖенский – 23%,"Россия – 63,1%.",3 500 000,1 572 000 ₽,1 300 000 ₽,1 250 000 ₽,https://disk.yandex.ru/d/ajGlE9Fx7SrxGw
8,NaN,Э,настя badbaarbie,https://www.youtube.com/@badbarbara,4 320 000,статистика,"Развлекательное, Лайфстайл","13–17 лет – 14,4%;\n18–24 года – 58,8%;\n25–34...",Мужской – 7%\nЖенский – 93%,"Россия – 65,5%.",70 000,200 000 ₽,175 000 ₽,100 000 ₽,https://disk.yandex.ru/d/EDbrK6por7PaNA
9,NaN,П,приятный ильдар,https://www.youtube.com/@pleasantildar,3 820 000,статистика,"Развлекательное, Комментаторство","13–17 лет– 3,7%;\n18–24 года – 57,6%;\n25–34 г...",Мужской – 19%\nЖенский – 81%,Россия – 75%,1 000 000,715 000 ₽,600 000 ₽,600 000 ₽,https://disk.yandex.ru/d/TSHuC9PsJrkVnQ
10,NaN,П,даша боровик,https://www.youtube.com/@borovikdasha,2 780 000,статистика,Развлекательное,"13–17 лет – 5,9%;\n18–24 года – 25,6%;\n25–34 ...","Мужской – 35,7%\nЖенский – 64%","Россия – 61,3%.",280 000,NaN,NaN,NaN,https://drive.google.com/drive/folders/1gNch8O...
11,NaN,П,сабина хайрова,https://www.youtube.com/@Sabinakhayrova_,1 780 000,статистика,"Развлекательное, Лайфстайл",13–17 лет – 6%;\n18–24 года – 37%;\n25–34 года...,Мужской – 57%\nЖенский – 41%,Россия – 66%.,1 000 000,—,—,—,https://drive.google.com/drive/u/0/folders/1lc...


In [421]:
columns_to_drop = [0, 1]
nextup_youtube = nextup_columns_to_drop(nextup_youtube, columns_to_drop, True)

In [422]:
nextup_youtube_columns_upd = ['тематика', 'ца по возрасту', 'ца по полу', 'ца по странам']
nextup_youtube = nextup_col_upd(nextup_youtube, nextup_youtube_columns_upd)

In [423]:
[col for col in nextup_youtube.columns]

[nan,
 'имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'тематика',
 'ца по возрасту',
 'ца по полу',
 'ца по странам',
 'охват видео за 30 дней',
 'стоимость интеграция / 1 слот',
 'стоимость интеграция / 2 слот',
 'стоимость преролл']

In [424]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала',
 'YouTube стоимость за shorts до 1 мин']

In [425]:
nextup_youtube.rename(columns = {'имя блогера':'блогер',
                                 'кол-во подписчиков':'количество подписчиков',
                                 'охват видео за 30 дней':'среднее кол-во просмотров видео',
                                 'стоимость интеграция / 1 слот':'стоимость интеграция 1 слот',
                                 'стоимость интеграция / 2 слот':'стоимость интеграция 2 слот',
                                 'стоимость преролл':'стоимость преролл до 30 сек',
                                 'статистика ссылка':'статистика'
                                }, inplace=True)

In [426]:
nextup_youtube.head()

6,NaN,блогер,ссылка,количество подписчиков,тематика,ца по возрасту,ца по полу,ца по странам,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость преролл до 30 сек
7,NaN,илья exile,https://www.youtube.com/@ExileShow,4 350 000,развлекательное,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%.",3 500 000,1 572 000 ₽,1 300 000 ₽,1 250 000 ₽
8,NaN,настя badbaarbie,https://www.youtube.com/@badbarbara,4 320 000,"развлекательное, лайфстайл","13–17 лет – 14,4%;, 18–24 года – 58,8%;, 25–34...","мужской – 7%, женский – 93%","россия – 65,5%.",70 000,200 000 ₽,175 000 ₽,100 000 ₽
9,NaN,приятный ильдар,https://www.youtube.com/@pleasantildar,3 820 000,"развлекательное, комментаторство","13–17 лет– 3,7%;, 18–24 года – 57,6%;, 25–34 г...","мужской – 19%, женский – 81%",россия – 75%,1 000 000,715 000 ₽,600 000 ₽,600 000 ₽
10,NaN,даша боровик,https://www.youtube.com/@borovikdasha,2 780 000,развлекательное,"13–17 лет – 5,9%;, 18–24 года – 25,6%;, 25–34 ...","мужской – 35,7%, женский – 64%","россия – 61,3%.",280 000,NaN,NaN,NaN
11,NaN,сабина хайрова,https://www.youtube.com/@Sabinakhayrova_,1 780 000,"развлекательное, лайфстайл","13–17 лет – 6%;, 18–24 года – 37%;, 25–34 года...","мужской – 57%, женский – 41%",россия – 66%.,1 000 000,—,—,—


In [427]:
columns_to_drop = [0]
nextup_youtube = nextup_columns_to_drop(nextup_youtube, columns_to_drop, False)

In [428]:
nextup_youtube.head()

6,блогер,ссылка,количество подписчиков,тематика,ца по возрасту,ца по полу,ца по странам,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость преролл до 30 сек
7,илья exile,https://www.youtube.com/@ExileShow,4 350 000,развлекательное,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%.",3 500 000,1 572 000 ₽,1 300 000 ₽,1 250 000 ₽
8,настя badbaarbie,https://www.youtube.com/@badbarbara,4 320 000,"развлекательное, лайфстайл","13–17 лет – 14,4%;, 18–24 года – 58,8%;, 25–34...","мужской – 7%, женский – 93%","россия – 65,5%.",70 000,200 000 ₽,175 000 ₽,100 000 ₽
9,приятный ильдар,https://www.youtube.com/@pleasantildar,3 820 000,"развлекательное, комментаторство","13–17 лет– 3,7%;, 18–24 года – 57,6%;, 25–34 г...","мужской – 19%, женский – 81%",россия – 75%,1 000 000,715 000 ₽,600 000 ₽,600 000 ₽
10,даша боровик,https://www.youtube.com/@borovikdasha,2 780 000,развлекательное,"13–17 лет – 5,9%;, 18–24 года – 25,6%;, 25–34 ...","мужской – 35,7%, женский – 64%","россия – 61,3%.",280 000,NaN,NaN,NaN
11,сабина хайрова,https://www.youtube.com/@Sabinakhayrova_,1 780 000,"развлекательное, лайфстайл","13–17 лет – 6%;, 18–24 года – 37%;, 25–34 года...","мужской – 57%, женский – 41%",россия – 66%.,1 000 000,—,—,—


#### Проверка на дубликаты

In [429]:
show_duplicates(nextup_youtube)

Количество дубликатов: 0


6,блогер,ссылка,количество подписчиков,тематика,ца по возрасту,ца по полу,ца по странам,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость интеграция 2 слот,стоимость преролл до 30 сек


### Двадцать пятое объединение

In [430]:
bufer = bufer.astype(str)

In [431]:
bufer = merge_bufer_df(bufer, nextup_youtube, 'YouTube')

Количество дубликатов: 32


In [432]:
show_duplicates(bufer).head(6)

Количество дубликатов: 32


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам
772,алина олешева,лайфстайл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"13–17 лет– 2,8%;, 18–24 года – 56,1%;, 25–34 г...","мужской – 35,8%, женский – 64,1%","россия – 80,5%."
679,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",624 000,300 000,110 000,260 000,None,...,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%",78 500 ₽,71 500 ₽,NaN,NaN,NaN
10,алёна эксекверт,"бьюти, лайфстайл",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл",247 000,90 000,35 000,200 000,None,...,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%",35 000 ₽,20 000 ₽,NaN,NaN,NaN
773,алёна эксекверт,"развлекательное, лайфстайл",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"13–17 лет– 12,9%;, 18–24 года – 66%;, 25–34 го...","мужской – 8,1%, женский – 91,7%","россия – 80,6%."
6,аня акулич,"бьюти, лайфстайл",аня акулич,https://instagram.com/anna.utka22,"бьюти, лайфстайл",620 000,350 000,120 000,390 000,None,...,None,None,"москва – 13,5% спб – 4,7%","13–17 лет – 20,1%, 18–24 года – 62,7%, 25–34 г...","мужской – 48,2% женский – 51,7%",260 000 ₽,286 000 ₽,NaN,NaN,NaN
775,аня акулич,лайфстайл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"13–17 лет – 8%;, 18–24 года – 68%;, 25–34 года...","мужской – 19%, женский – 80%",россия – 74%.


In [433]:
# До

bufer.shape

(783, 183)

In [434]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [435]:
# После

bufer.shape

(783, 183)

In [436]:
duplicates = bufer[bufer.duplicated('блогер', keep=False)]
duplicates.sort_values(by='блогер')

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам
772,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",624 000,300 000,110 000,260 000,None,...,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%",78 500 ₽,71 500 ₽,"13–17 лет– 2,8%;, 18–24 года – 56,1%;, 25–34 г...","мужской – 35,8%, женский – 64,1%","россия – 80,5%."
679,алина олешева,"лайфстайл, музыка",алина олешева,https://www.instagram.com/oleshev,"лайфстайл, музыка",624 000,300 000,110 000,260 000,None,...,None,None,"москва - 14,6% спб - 6,6%","13-17 лет - 10,3% 18-24 года - 60,6% 25-34 год...","мужской - 34,5% женский - 65,4%",78 500 ₽,71 500 ₽,"13–17 лет– 2,8%;, 18–24 года – 56,1%;, 25–34 г...","мужской – 35,8%, женский – 64,1%","россия – 80,5%."
10,алёна эксекверт,"бьюти, лайфстайл, развлекательное",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл, развлекательное",247 000,90 000,35 000,200 000,None,...,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%",35 000 ₽,20 000 ₽,"13–17 лет– 12,9%;, 18–24 года – 66%;, 25–34 го...","мужской – 8,1%, женский – 91,7%","россия – 80,6%."
773,алёна эксекверт,"бьюти, лайфстайл, развлекательное",алёна эксекверт,https://instagram.com/exeqwert,"бьюти, лайфстайл, развлекательное",247 000,90 000,35 000,200 000,None,...,None,None,"москва – 15,6% спб – 6,1%","13–17 лет – 23% 18–24 года – 63,4% 25–34 года ...","мужской – 9,6% женский – 90,3%",35 000 ₽,20 000 ₽,"13–17 лет– 12,9%;, 18–24 года – 66%;, 25–34 го...","мужской – 8,1%, женский – 91,7%","россия – 80,6%."
6,аня акулич,"бьюти, лайфстайл",аня акулич,https://instagram.com/anna.utka22,"бьюти, лайфстайл",620 000,350 000,120 000,390 000,None,...,None,None,"москва – 13,5% спб – 4,7%","13–17 лет – 20,1%, 18–24 года – 62,7%, 25–34 г...","мужской – 48,2% женский – 51,7%",260 000 ₽,286 000 ₽,"13–17 лет – 8%;, 18–24 года – 68%;, 25–34 года...","мужской – 19%, женский – 80%",россия – 74%.
775,аня акулич,"бьюти, лайфстайл",аня акулич,https://instagram.com/anna.utka22,"бьюти, лайфстайл",620 000,350 000,120 000,390 000,None,...,None,None,"москва – 13,5% спб – 4,7%","13–17 лет – 20,1%, 18–24 года – 62,7%, 25–34 г...","мужской – 48,2% женский – 51,7%",260 000 ₽,286 000 ₽,"13–17 лет – 8%;, 18–24 года – 68%;, 25–34 года...","мужской – 19%, женский – 80%",россия – 74%.
755,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,–,None,...,None,None,"москва - 14,5% спб - 3,8%","13-17 лет - 21,8% 18-24 года - 61% 25-34 года ...","мужской - 60,1% женский - 39,2%",700 000 ₽,700 000 ₽,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%."
672,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,–,None,...,None,None,"москва - 14,5% спб - 3,8%","13-17 лет - 21,8% 18-24 года - 61% 25-34 года ...","мужской - 60,1% женский - 39,2%",700 000 ₽,700 000 ₽,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%."
780,катя бомонд,"бьюти, лайфстайл",катя бомонд,https://instagram.com/rul.bomond,"бьюти, лайфстайл",12 700,8 000,7 000,15 000,None,...,None,None,"москва – 25,1% спб – 9,5%","13–17 лет – 13,2% 18–24 года – 66,8% 25–34 го...","мужской – 25,9% женский – 74%",6 000 ₽,7 500 

> Удалим дубликаты, оставив только первое вхождение, так как чаще всего именно в них самые большие и видимо правильные значения

In [437]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

In [438]:
show_duplicates(bufer).head(6)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,RuTube стоимость преролл до 30 сек,RuTube стоимость интеграция 1 слот,Instagram ца по городам,Instagram ца по возрасту,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам


### Подготовка таблицы nextup_twitch

In [439]:
nextup_twitch.head()

6,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,средний онлайн на стриме,охват стрима за 14 дней,баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим,баннер + чат-бот + эксклюзивный стрим до 1 часа,баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц),статистика ссылка
7,NaN,П,paradeev1ch,https://twitch.tv/paradeev1ch,1 721 517,статистика,15 747,200 000,360 000 ₽,860 000 ₽,1 715 000 ₽,https://disk.yandex.ru/d/1qH15yQqi30A6g
8,NaN,П,koreshzy,https://twitch.tv/koreshzy,1 659 339,статистика,9 135,170 000,215 000 ₽,430 000 ₽,1 115 000 ₽,https://disk.yandex.ru/d/L5_h5VTloJAJ4w
9,NaN,П,stray228,https://www.twitch.tv/stray228,1 529 691,статистика,8 130,115 000,-,-,715 000 ₽,https://disk.yandex.ru/d/A6ST21aj67tKQw
10,NaN,П,jojohf,https://twitch.tv/jojohf,1 137 203,статистика,5 525,75 000,120 000 ₽,240 000 ₽,230 000 ₽,https://drive.google.com/drive/u/0/folders/1zt...
11,NaN,Э,gensyxa,https://www.twitch.tv/gensyxa,1 127 716,статистика,2 456,80 000,150 000 ₽,279 000 ₽,500 000 ₽,https://disk.yandex.ru/d/OBbr9aLYNJTpZQ


In [440]:
columns_to_drop = [0, 1]
nextup_twitch = nextup_columns_to_drop(nextup_twitch, columns_to_drop, True)

In [441]:
nextup_twitch.head(1)

6,имя блогера,ссылка,кол-во подписчиков,средний онлайн на стриме,охват стрима за 14 дней,баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим,баннер + чат-бот + эксклюзивный стрим до 1 часа,баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц),статистика ссылка
7,paradeev1ch,https://twitch.tv/paradeev1ch,1 721 517,15 747,200 000,360 000 ₽,860 000 ₽,1 715 000 ₽,https://disk.yandex.ru/d/1qH15yQqi30A6g


In [442]:
[col for col in nextup_twitch.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'средний онлайн на стриме',
 'охват стрима за 14 дней',
 'баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим',
 'баннер + чат-бот + эксклюзивный стрим до 1 часа',
 'баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц)',
 'статистика ссылка']

In [443]:
[col for col in bufer.columns if 'Twitch' in col]

['Twitch блогер',
 'Twitch ссылка',
 'Twitch тематика',
 'Twitch количество подписчиков',
 'Twitch среднее кол-во зрителей',
 'Twitch уникальные зрители (30 дней)',
 'Twitch стоимость за баннер + ссылка в описании 1 мес',
 'Twitch стоимость оверлей логотипа товара/бренда в процессе трансляции',
 'Twitch стоимость нативное голосовое упоминание товара в трансляции',
 'Twitch стоимость ссылка в чат-боте на 1 стрим',
 'Twitch стоимость демонстрация/обзор товара на трансляции',
 'Twitch статистика']

In [444]:
nextup_twitch.rename(columns = {'имя блогера':'блогер',
                                'кол-во подписчиков':'количество подписчиков',
                                'средний онлайн на стриме':'среднее кол-во зрителей',
                                'охват стрима за 14 дней':'уникальные зрители (14 дней)',
                                'баннер + чат-бот + голосовая интеграция до 3 мин. на 1 стрим':'стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим',
                                'баннер + чат-бот + эксклюзивный стрим до 1 часа':'стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа',
                                'баннер + чат-бот на 1 месяц + голосовое упоминание 1 раз в неделю (1 месяц)':'стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)',
                                'статистика ссылка':'статистика'
                               }, inplace=True)

#### Проверка на дубликаты

In [445]:
show_duplicates(nextup_twitch)

Количество дубликатов: 0


6,блогер,ссылка,количество подписчиков,среднее кол-во зрителей,уникальные зрители (14 дней),"стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",статистика


### Двадцать шестое объединение

In [446]:
bufer = merge_bufer_df(bufer, nextup_twitch, 'Twitch')

Количество дубликатов: 4


In [447]:
show_duplicates(bufer)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)"
178,finargot,,None,None,,None,None,None,None,None,...,None,19 000 ₽,20 000 ₽,None,None,None,NaN,NaN,NaN,NaN
782,finargot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20 000,27 000 ₽,30 000 ₽,75 000 ₽
756,paradeev1ch,развлекательное,None,None,развлекательное,None,None,None,None,None,...,None,None,None,"13–17 лет – 4,7%;, 18–24 года – 64,3%;, 25–34 ...","мужской – 73,3%, женский – 26,1%","россия – 74,9%.",NaN,NaN,NaN,NaN
767,paradeev1ch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,200 000,360 000 ₽,860 000 ₽,1 715 000 ₽


In [448]:
# До

bufer.shape

(788, 187)

In [449]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [450]:
# После

bufer.shape

(787, 187)

In [451]:
show_duplicates(bufer)

Количество дубликатов: 2


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram ца по полу,Telegram стоимость текстовый пост,Telegram стоимость видеокружок,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)"
178,finargot,,None,None,,None,None,None,None,None,...,None,19 000 ₽,20 000 ₽,None,None,None,20 000,27 000 ₽,30 000 ₽,75 000 ₽
782,finargot,,None,None,,None,None,None,None,None,...,None,19 000 ₽,20 000 ₽,None,None,None,20 000,27 000 ₽,30 000 ₽,75 000 ₽


> Удалим дубликаты, оставив только первое вхождение

In [452]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

### Подготовка таблицы nextup_vk

In [453]:
nextup_vk.head()

5,NaN,NaN,имя блогера,ссылка,кол-во подписчиков,статистика,ца по городам,ца по возрасту,ца по полу,охват поста за 30 дней,охват vk клип за 30 дней,стоимость поста,стоимость vk клип,статистика ссылка
6,NaN,П,илья exile,https://vk.com/exile_music,507 872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,П,paradeev1ch,https://vk.com/paradeev1ch,414 000,NaN,NaN,NaN,NaN,45 000,-,78 500 ₽,71 500 ₽,NaN
8,NaN,П,frametamer,https://vk.com/frametamer1337,262 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Э,лунаком,https://vk.com/lynacom2000,253 000,статистика,NaN,NaN,NaN,1 200 000,550 000,NaN,NaN,https://drive.google.com/drive/folders/1JEkYq2...
10,NaN,Э,bad barbie,https://vk.com/badbarbieeeeeee,167 000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [454]:
columns_to_drop = [0, 1]
nextup_vk = nextup_columns_to_drop(nextup_vk, columns_to_drop, True)

In [455]:
nextup_vk_columns_upd = ['ца по городам', 'ца по возрасту', 'ца по полу']
nextup_vk = nextup_col_upd(nextup_vk, nextup_vk_columns_upd)

In [456]:
[col for col in nextup_vk.columns]

['имя блогера',
 'ссылка',
 'кол-во подписчиков',
 'ца по городам',
 'ца по возрасту',
 'ца по полу',
 'охват поста за 30 дней',
 'охват vk клип за 30 дней',
 'стоимость поста',
 'стоимость vk клип',
 'статистика ссылка']

In [457]:
[col for col in bufer.columns if 'VK группа' in col]

['VK группа блогер',
 'VK группа ссылка',
 'VK группа тематика',
 'VK группа количество подписчиков',
 'VK группа среднее кол-во просмотров поста',
 'VK группа среднее кол-во просмотров вк клипа',
 'VK группа среднее кол-во лайков',
 'VK группа среднее кол-во комментариев',
 'VK группа стоимость фотопост',
 'VK группа стоимость видеопост',
 'VK группа стоимость вк клип',
 'VK группа статистика',
 'VK группа город',
 'VK группа стоимость передачи прав на 1 месяц']

In [458]:
nextup_vk.rename(columns = {'имя блогера':'блогер',
                            'кол-во подписчиков':'количество подписчиков',
                            'охват поста за 30 дней':'среднее кол-во просмотров поста',
                            'охват vk клип за 30 дней':'среднее кол-во просмотров вк клипа',
                            'стоимость поста':'стоимость фотопост',
                            'стоимость vk клип':'стоимость вк клип',
                            'статистика ссылка':'статистика'
                           }, inplace=True)

In [459]:
nextup_vk.head(1)

5,блогер,ссылка,количество подписчиков,ца по городам,ца по возрасту,ца по полу,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость фотопост,стоимость вк клип,статистика
6,илья exile,https://vk.com/exile_music,507 872,nan,nan,nan,NaN,NaN,NaN,NaN,NaN


#### Проверка на дубликаты

In [460]:
show_duplicates(nextup_vk)

Количество дубликатов: 0


5,блогер,ссылка,количество подписчиков,ца по городам,ца по возрасту,ца по полу,среднее кол-во просмотров поста,среднее кол-во просмотров вк клипа,стоимость фотопост,стоимость вк клип,статистика


### Двадцать седьмое объединение

In [461]:
bufer = merge_bufer_df(bufer, nextup_vk, 'VK группа')

Количество дубликатов: 14


In [462]:
show_duplicates(bufer).head(6)

Количество дубликатов: 14


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",VK группа ца по городам,VK группа ца по возрасту,VK группа ца по полу
756,paradeev1ch,развлекательное,None,None,развлекательное,None,None,None,None,None,...,"13–17 лет – 4,7%;, 18–24 года – 64,3%;, 25–34 ...","мужской – 73,3%, женский – 26,1%","россия – 74,9%.",200 000,360 000 ₽,860 000 ₽,1 715 000 ₽,NaN,NaN,NaN
787,paradeev1ch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
760,арина arlovedbyyou,"лайфстайл, развлекательное",None,None,"лайфстайл, развлекательное",None,None,None,None,None,...,"13–17 лет– 11%;, 18–24 года – 65,4%;, 25–34 го...","мужской – 5,9%, женский – 93,9%","россия – 85,3%.",None,None,None,None,NaN,NaN,NaN
792,арина arlovedbyyou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan
346,данила горилла,лайфстайл,данила горилла,https://instagram.com/danila.gorilla,лайфстайл,159 000,56 000,49 000,80 000,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
796,данила горилла,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan


In [463]:
# До

bufer.shape

(797, 190)

In [464]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [465]:
# После

bufer.shape

(795, 190)

In [466]:
show_duplicates(bufer)

Количество дубликатов: 10


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",VK группа ца по городам,VK группа ца по возрасту,VK группа ца по полу
346,данила горилла,лайфстайл,данила горилла,https://instagram.com/danila.gorilla,лайфстайл,159 000,56 000,49 000,80 000,None,...,None,None,None,None,None,None,None,nan,nan,nan
796,данила горилла,лайфстайл,данила горилла,https://instagram.com/danila.gorilla,лайфстайл,159 000,56 000,49 000,80 000,None,...,None,None,None,None,None,None,None,nan,nan,nan
622,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,–,None,...,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%.",None,None,None,None,nan,nan,nan
786,илья exile,"музыка, развлекательное",илья exile,https://instagram.com/exile_music,"музыка, развлекательное",1 020 000,95 000,170 000,–,None,...,"13–17 лет – 6%;, 18–24 года – 61%;, 25–34 года...","мужской – 76%, женский – 23%","россия – 63,1%.",None,None,None,None,nan,nan,nan
621,ира (inst_rinna),"бьюти, лайфстайл, танцы",ира (inst_rinna),https://instagram.com/inst_rinna,"бьюти, лайфстайл, танцы",1 000 000,650 000,230 000,250 000,None,...,None,None,None,None,None,None,None,"москва – 26%, спб – 11%, краснодар – 4%","13–17 лет – 17,5%, 18–24 года – 15%, 25–34 год...","мужской – 68%, женский – 32%"
793,ира (inst_rinna),"бьюти, лайфстайл, танцы",ира (inst_rinna),https://instagram.com/inst_rinna,"бьюти, лайфстайл, танцы",1 000 000,650 000,230 000,250 000,None,...,None,None,None,None,None,None,None,"москва – 26%, спб – 11%, краснодар – 4%","13–17 лет – 17,5%, 18–24 года – 15%, 25–34 год...","мужской – 68%, женский – 32%"
637,саша теслонд,"лайфстайл, развлекательное, фешн",саша теслонд,https://instagram.com/teslond,"лайфстайл, развлекательное, фешн",540 000,200 000,90 000,350 000,None,...,"13–17 лет– 8%;, 18–24 года – 67%;, 25–34 года ...","мужской – 14%, женский – 93,9%",россия – 84%.,None,None,None,None,nan,nan,nan
791,саша теслонд,"лайфстайл, развлекательное, фешн",саша теслонд,https://instagram.com/teslond,"лайфстайл, развлекательное, фешн",540 000,200 000,90 000,350 000,None,...,"13–17 лет– 8%;, 18–24 года – 67%;, 25–34 года ...","мужской – 14%, женский – 93,9%",россия – 84%.,None,None,None,None,nan,nan,nan
161,ярослав (дилблин),,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,"москва – 21%, спб – 11%, уфа – 4%","13–17 лет – 35%, 18–24 года – 17%, 25–34 года ...","мужской – 34%, женский – 66%"
794,ярослав (дилблин),,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,"москва – 21%, спб – 11%, уфа – 4%","13–17 лет – 35%, 18–24 года – 17%, 25–34 года ...","мужской – 34%, женский – 66%"


> Удалим дубликаты, оставив только первое вхождение

In [467]:
bufer = bufer.drop_duplicates(subset=['блогер'], keep='first')

In [468]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('https://t.me/Oliva_Nick', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (6%)', inplace=True)

> Посмотрим на нашу таблицу bufer

In [469]:
bufer

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,YouTube ца по возрасту,YouTube ца по полу,YouTube ца по странам,Twitch уникальные зрители (14 дней),"Twitch стоимость за баннер, чат-бот, голосовая интеграция до 3 мин. на 1 стрим","Twitch стоимость за баннер, чат-бот, эксклюзивный стрим до 1 часа","Twitch стоимость за баннер, чат-бот на 1 месяц, голосовое упоминание 1 раз в неделю (1 месяц)",VK группа ца по городам,VK группа ца по возрасту,VK группа ца по полу
0,катя клэп,"бьюти, лайфстайл",катя клэп,https://www.instagram.com/kateclapp/,"бьюти, лайфстайл",10 100 000,250 000,150 000,200 000,50 000,...,None,None,None,None,None,None,None,None,None,None
1,kari,"бьюти, лайфстайл",kari,https://www.instagram.com/k.hism/,"бьюти, лайфстайл",101 000,100 000,20 000,100 000,7 000,...,None,None,None,None,None,None,None,None,None,None
2,рязанова татьяна,"бьюти, лайфстайл",None,None,"бьюти, лайфстайл",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,эвелина лунаком,"бьюти, лайфстайл",эвелина лунаком,https://instagram.com/lyna_com2000,"бьюти, лайфстайл",1 700 000,500 000,250 000,1 000 000,None,...,None,None,None,None,None,None,None,None,None,None
4,наташа генсуха,"бьюти, лайфстайл",наташа генсуха,https://www.instagram.com/gensyxa,"бьюти, лайфстайл",625 000,680 000,80 000,350 000,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,polovinkago,,None,None,,None,None,None,None,None,...,None,None,None,300,9 000 ₽,10 000 ₽,15 000 ₽,None,None,None
788,frametamer,,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,nan,nan,nan
789,лунаком,,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,nan,nan,nan
790,bad barbie,,None,None,,None,None,None,None,None,...,None,None,None,None,None,None,None,nan,nan,nan


## Подготовка данных по блогерам от AvtorMedia

> Предварительно посмотрим на содержание таблицы

In [470]:
avtormedia_telegram.head(1)

,Блогер,Тематика,Подписчики,Материалы\nи статистика,Средние просмотры,Короткое видео,Пост,Спешл,Цена,Спешл,...,Пакетное предложение,Цена,Охват,CPV,Пакетное предложение,Цена,Охват,CPV,Ссылка,Статистика ссылка
1,Илья Варламов @varlamov_news,новости,1 200 000,Ссылка,470 000,833 333 ₽,450 000 ₽,Сторис,500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov_news,https://drive.google.com/drive/u/1/folders/165...


In [471]:
avtormedia_df = [avtormedia_youtube,
                 avtormedia_instagram,
                 avtormedia_telegram,
                 avtormedia_vk,
                 avtormedia_zen,
                 avtormedia_podcast]

In [472]:
key_word = ['CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ''  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Блогер']

In [473]:
# for df in avtormedia_df:
#     pretraining_df(df, key_word, drop_word, upd_columns)
    
    
for i in range(len(avtormedia_df)):
    df = avtormedia_df[i]  # Получаем текущий DataFrame по индексу
    
    upd_columns = []  # Пустой список столбцов с именами блогеров для приведения к единому формату

    if 'Блогер' in df.columns:
        upd_columns.append('Блогер')
    if 'Канал' in df.columns:
        upd_columns.append('Канал')

    pretraining_df(df, key_word, drop_word, upd_columns)

### Подготовка таблицы avtormedia_youtube

In [474]:
avtormedia_youtube.head()

,блогер,тематика,подписчики,"материалы, и статистика",мидролл в основные форматы,охват видео,интеграция,пре-ролл,интеграция (2 слот),спецпроект,...,цена,охват,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@HiBestMan/featured,https://drive.google.com/drive/folders/1AG9et9...
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@ItsMamix/featured,https://drive.google.com/drive/folders/12S_GJx...
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@VilyamBruno/featured,https://drive.google.com/drive/u/1/folders/1pS...
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@LUANAAA/featured,https://drive.google.com/drive/u/1/folders/1Cx...
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@OneTwoOneTwoRS/featured,https://drive.google.com/drive/folders/1HCe1B_...


In [475]:
[col for col in avtormedia_youtube.columns]

['блогер',
 'тематика',
 'подписчики',
 'материалы, и статистика',
 'мидролл в основные форматы',
 'охват видео',
 'интеграция',
 'пре-ролл',
 'интеграция (2 слот)',
 'спецпроект',
 'охват shorts',
 'shorts',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'спешл',
 'цена',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'ссылка',
 'статистика']

In [476]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'стоимость мидролл до 30 сек',
                 'среднее кол-во просмотров видео',
                 'стоимость интеграция 1 слот',
                 'стоимость преролл до 30 сек',
                 'стоимость интеграция 2 слот',
                 'стоимость спецпроект',
                 'среднее кол-во просмотров shorts',
                 'стоимость за shorts до 1 мин',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'спешл 3',
                 'цена 3',
                 'спешл 4',
                 'цена 4',
                 'спешл 5',
                 'цена 5',
                 'спешл 6',
                 'цена 6',
                 'спешл 7',
                 'цена 7',
                 'пакетное предложение 8',
                 'цена 8',
                 'охват 8',
                 'пакетное предложение 9',
                 'цена 9',
                 'охват 9',
                 'пакетное предложение 10',
                 'цена 10',
                 'охват 10',
                 'пакетное предложение 11',
                 'цена 11',
                 'охват 11',
                 'пакетное предложение 12',
                 'цена 12',
                 'охват 12',
                 'ссылка',
                 'статистика']

In [477]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала',
 'YouTube стоимость за shorts до 1 мин',
 'YouTube ца по возрасту',
 'YouTube ца по полу',
 'YouTube ца по странам']

In [478]:
avtormedia_youtube.columns = bufer_columns

In [479]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,цена 10,охват 10,пакетное предложение 11,цена 11,охват 11,пакетное предложение 12,цена 12,охват 12,ссылка,статистика
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@HiBestMan/featured,https://drive.google.com/drive/folders/1AG9et9...
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@ItsMamix/featured,https://drive.google.com/drive/folders/12S_GJx...
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@VilyamBruno/featured,https://drive.google.com/drive/u/1/folders/1pS...
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@LUANAAA/featured,https://drive.google.com/drive/u/1/folders/1Cx...
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/@OneTwoOneTwoRS/featured,https://drive.google.com/drive/folders/1HCe1B_...


In [480]:
column_list_1 = []

for column_number in range(1, 8):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_youtube = get_column_values(avtormedia_youtube,
                                           column_number,
                                           position_column, 
                                           price_column, 
                                           column_list_1)

In [481]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,Интеграция в формат «Книга жалоб» (1 слот)_4,Продакт-плейсмент_4,"Выездная интеграция в воскресный выпуск новостей ""Чё Происходит""_5",Преролл в реакцию на трейлер или разборы_5,Пост в сообществе_5,Интеграция в обзор_5,Интеграция в формат «Спешл»_5,Возможность выбрать тему ролика_6,Интеграция в подкаст «Сладкая жизнь» (2 слот)_6,Пре-ролл в подкаст «Сладкая жизнь»_7
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [482]:
sorted_column_names_1 = sorted(column_list_1)
sorted_column_names_1

['Бегущая строка_1',
 'Возможность выбрать тему ролика_6',
 'Выездная интеграция в воскресный выпуск новостей "Чё Происходит"_5',
 'Выездная интеграция_2',
 'Выездная интеграция_3',
 'Выездной спецпроект_1',
 'Интеграция (3 слот)_2',
 'Интеграция (4 слот)_3',
 'Интеграция в воскресный выпуск новостей "Чё Происходит" (2 слот)_2',
 'Интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот_4',
 'Интеграция в обзор_5',
 'Интеграция в подкаст «Сладкая жизнь» (1 слот)_2',
 'Интеграция в подкаст «Сладкая жизнь» (2 слот)_6',
 'Интеграция в разборы_4',
 'Интеграция в реакцию на трейлер_2',
 'Интеграция в рецепт_4',
 'Интеграция в ролик на тему по выбору клиента_2',
 'Интеграция в тревел-ролики_1',
 'Интеграция в формат "Криминальный ТикТок"_2',
 'Интеграция в формат «Книга жалоб» (1 слот)_4',
 'Интеграция в формат «Книга жалоб» (2 слот)_1',
 'Интеграция в формат «Свой/чужой» (1 слот)_4',
 'Интеграция в формат «Свой/чужой» (2 слот)_1',
 'Интеграция в формат «Спешл»_5',
 'Интеграция в форма

In [483]:
def get_column_values_packet(df: pd.DataFrame,
                             key: int,
                             column_with_position: str,
                             column_with_price: str,
                             column_with_coverage: str,
                             column_list: list) -> pd.DataFrame:
    
    """
    АРГУМЕНТЫ ФУНКЦИИ:
        df (pd.DataFrame) - DataFrame, в котором делаем преобразования;
        key (int) - Ключ в виде int, который нужно добавить к названию, 
            чтобы в дальнейшем объединить несколько столбцов с одинаковой позицией;
        column_with_position (str) - Столбец, в котором указаны позиции;
        column_with_price (str) - Столбец, в котором указаны цены на позиции;
        column_with_coverage (str) - Столбец, в котором указан охват позиции;
        column_list (list) - 
    
    
    
    ОПИСАНИЕ ВЫПОЛНЕНИЯ ФУНКЦИИ С ПРИМЕРОМ:
    
        Создаём список с уникальными значениями позиций,
            приведя их к единому формату, используя .strip(), .str.lower(), .str.replace:
        spisok = avtormedia_youtube_actual['Спешл'].str.strip()\
                                                   .str.lower()\
                                                   .str.replace('/\n', ', ')\
                                                   .str.replace('\n', ', ')\
                                                   .str.replace('*', '')\
                                                   .unique()

        #Большинство значений строк в столбце column_with_position - NaN, у которых нет определённых цен.
        #Мы удалим данные позиции из списка, чтобы не создавать для них столбцы и/или не получать ошибку
        spisok = spisok[~pd.isna(spisok)]


        # Цикл for проходит по списку уникальных позиций и вводит их названия:
        for i in range(len(unique_position)):

        # Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:
            df[f"{unique_position[i]}_{key}"] = df[column_with_price].\
            where(df[column_with_position] == unique_position[i], other='')


        Удаляем исходные столбцы, чтобы не было повтора:
        df.drop(columns=[column_with_price, column_with_position], axis=1, inplace= True)


        Возвращаем обновлённый DataFrame
        return df    
                
    """

    unique_position = df[column_with_position].str.strip()\
                                              .unique()
    
    unique_position = unique_position[~pd.isna(unique_position)]

# Цикл for проходит по списку уникальных позиций и вводит их названия:
    for i in range(len(unique_position)):
        
        new_column_name = f'{unique_position[i]}_{key}'
        
# Делаем уникальный столбец, название которого соотвествует уникальной позиции, заполняя его значениями соответсвующей цены:    
        df[new_column_name] = \
                df[column_with_price].where(df[column_with_position] == unique_position[i], other='')
            
        df[f'охват {new_column_name}'] = \
                df[column_with_coverage].where(df[column_with_position] == unique_position[i], other='')
        
        
        # Добавляем имема новых столбцов в список, чтобы в дальнейшем работать с ними
        column_list.append(new_column_name) 
        
        # Заменим пустую строку на NaN
        df[new_column_name] = df[new_column_name].replace('', np.nan)
            
    df.drop(columns=[column_with_price, column_with_position], axis=1, inplace=True)
    
    return df

In [484]:
column_list_2 = []

for column_number in range(8, 13):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_youtube = get_column_values_packet(avtormedia_youtube,
                                                  column_number,
                                                  position_column,
                                                  price_column,
                                                  coverage_column,
                                                  column_list_2)

In [485]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,Интеграция в рецепт + фото в Инст_9,охват Интеграция в рецепт + фото в Инст_9,Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9,охват Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9,Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,Интеграция в видео на YouTube + пост в сообществе YouTube_11,охват Интеграция в видео на YouTube + пост в сообществе YouTube_11,Интеграция на YouTube + Сторис в Instagram_12,охват Интеграция на YouTube + Сторис в Instagram_12
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,,NaN,,NaN,,NaN,,NaN,
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,,NaN,,NaN,,NaN,,NaN,


In [486]:
sorted_column_names_2 = sorted(column_list_2)
sorted_column_names_2

['3 интеграции_8',
 'Cпецпроект + фото-пост в Инст_8',
 'Интеграция + фото-пост в Инст_9',
 'Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10',
 'Интеграция в видео на YouTube + пост в сообществе YouTube_11',
 'Интеграция в видео на YouTube + фото-пост в Инст_8',
 'Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8',
 'Интеграция в обзор + фото в Инст_8',
 'Интеграция в рецепт + фото в Инст_9',
 'Интеграция на YouTube + Сторис в Instagram_12',
 'Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8',
 'Интеграция на Основном канале + Романов Говорит_8',
 'Пакет соц. сети (сторис ИГ + пост ВК + пост ТГ) (пост в формате «текст прислан», пост и фото от клиента)_8',
 'Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9',
 'Спецпроект + Анонс конкурса_8',
 'Спецпроект на YouTube + пост в Telegram_9',
 'интеграция YouTube + Дзен_8']

#### Обработаем новые столбцы

##### Начнём со списка sorted_column_names_1

In [487]:
dict_new_columns = {'стоимость усложненная интеграция':['Усложненная интеграция_2',
                                                        'Усложненная интеграция_3',
                                                        'Расширенная интеграция_1'],
                    
                    'стоимость пост в сообществе': ['Пост в сообществе_2',
                                                    'Пост в сообществе_3',
                                                    'Пост в сообществе_5'],
                    
                    'стоимость выездная интеграция':['Выездная интеграция_2',
                                                    'Выездная интеграция_3'],
                    
                   'стоимость интеграция с выбором темы видео':['Расширенная интеграция в ролик на тему по выбору клиента_1',
                                                                'Возможность выбрать тему ролика_6',
                                                                'Интеграция в ролик на тему по выбору клиента_2'],  
                    
                    'стоимость мидролл до 30 сек':['стоимость мидролл до 30 сек',
                                                   'Мидролл_1']
                    
                   }

In [488]:
def agg_columns(df: pd.DataFrame, dictation: dict) -> pd.DataFrame:
    
    """
    
    
    ПАРАМЕТРЫ:
        df: pd.DataFrame - 
        dictation: dict
 
    ВОЗВРАЩАЕМОЕ ЗНАЧЕНИЕ:
    
    ПРИМЕР ИСПОЛЬЗОВАНИЯ:
    
    """
    
    for new_column, old_column in dictation.items():

        for col in old_column:
            # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
            # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
            df[col] = df[col].fillna('').astype(str)

        df[new_column] = df[old_column].agg(''.join, axis=1)  

        #Вернём обратно NaN
        df[new_column] = df[new_column].replace('', np.nan)
        
#         # Удаляем столбцы из словаря
#         df.drop(columns=old_column, inplace=True)
        
    return df

In [489]:
# avtormedia_youtube = agg_columns(avtormedia_youtube, dict_new_columns)

In [490]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        avtormedia_youtube[col] = avtormedia_youtube[col].fillna('').astype(str)
    
    avtormedia_youtube[new_column] = avtormedia_youtube[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    avtormedia_youtube[new_column] = avtormedia_youtube[new_column].replace('', np.nan)
    avtormedia_youtube[col] = avtormedia_youtube[col].astype(str)

In [491]:
avtormedia_youtube.drop(columns=['Усложненная интеграция_2',
                                 'Усложненная интеграция_3',
                                 'Расширенная интеграция_1',
                                 'Пост в сообществе_2',
                                 'Пост в сообществе_3',
                                 'Пост в сообществе_5',
                                 'Выездная интеграция_2',
                                 'Выездная интеграция_3',
                                 'Расширенная интеграция в ролик на тему по выбору клиента_1',
                                 'Возможность выбрать тему ролика_6',
                                 'Интеграция в ролик на тему по выбору клиента_2',
                                 'Мидролл_1'], axis=1, inplace=True)

In [492]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,материалы и статистика,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,...,Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10,Интеграция в видео на YouTube + пост в сообществе YouTube_11,охват Интеграция в видео на YouTube + пост в сообществе YouTube_11,Интеграция на YouTube + Сторис в Instagram_12,охват Интеграция на YouTube + Сторис в Instagram_12,стоимость усложненная интеграция,стоимость пост в сообществе,стоимость выездная интеграция,стоимость интеграция с выбором темы видео
1,himan,челленджи\nэксперименты,19 100 000,Ссылка,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
2,it's mamix,эксперименты,15 300 000,Ссылка,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
3,вильям бруно,челленджи\nюмор,9 970 000,Ссылка,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
4,луана,анимация,6 470 000,Ссылка,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
5,onetwo,юмор\nскетчи,6 000 000,Ссылка,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN


In [493]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала',
 'YouTube стоимость за shorts до 1 мин',
 'YouTube ца по возрасту',
 'YouTube ца по полу',
 'YouTube ца по странам']

In [494]:
avtormedia_youtube.rename(columns = {'Бегущая строка_1':'стоимость бегущая строка',
                                     'Мидролл_1':'стоимость мидролл до 30 сек',
                                     'Выездная интеграция в воскресный выпуск новостей "Чё Происходит"_5':'стоимость выездная интеграция в воскресный выпуск новостей "Чё Происходит"',
                                     'Выездной спецпроект_1':'стоимость выездной спецпроект',
                                     'Интеграция (3 слот)_2':'стоимость интеграция 3 слот',
                                     'Интеграция (4 слот)_3':'стоимость интеграция 4 слот',
                                     'Интеграция в воскресный выпуск новостей "Чё Происходит" (2 слот)_2':'стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 2 слот',
                                     'Интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот_4':'стоимость интеграция в воскресный выпуск новостей "Чё Происходит" 1 слот',
                                     'Интеграция в обзор_5':'стоимость интеграция в обзор',
                                     'Интеграция в подкаст «Сладкая жизнь» (1 слот)_3':'стоимость интеграция в подкаст «Сладкая жизнь» 1 слот',
                                     'Интеграция в подкаст «Сладкая жизнь» (2 слот)_6':'стоимость интеграция в подкаст «Сладкая жизнь» 2 слот',
                                     'Интеграция в разборы_4':'стоимость интеграция в разборы',
                                     'Интеграция в реакцию на трейлер_2':'стоимость интеграция в реакцию на трейлер',
                                     'Интеграция в рецепт_4':'стоимость интеграция в рецепт',
                                     'Интеграция в ролик с Линой Диановой (1 слот)_1':'стоимость интеграция в ролик с Линой Диановой 1 слот',
                                     'Интеграция в ролик с Линой Диановой (2 слот)_2':'стоимость интеграция в ролик с Линой Диановой 2 слот',
                                     'Интеграция в тревел-ролики_1':'стоимость интеграция в тревел-ролики',
                                     'Интеграция в формат "Криминальный ТикТок"_2':'стоимость интеграция в формат "Криминальный ТикТок"',
                                     'Интеграция в формат «Книга жалоб» (1 слот)_4':'стоимость интеграция в формат «Книга жалоб» 1 слот',
                                     'Интеграция в формат «Книга жалоб» (2 слот)_1':'стоимость интеграция в формат «Книга жалоб» 2 слот',
                                     'Интеграция в формат «Спешл»_5':'стоимость интеграция в формат «Спешл»',
                                     'Интеграция в формат Стиратель на YouTube_1':'стоимость интеграция в формат Стиратель на YouTube',
                                     'Интеграция в формат новости_3':'стоимость интеграция в формат новости',
                                     'Интеграция с участием Максима Каца_1':'стоимость интеграция с участием Максима Каца',
                                     'Повторное размещение реализованной интеграции_1':'стоимость повторное размещение реализованной интеграции',
                                     'Пре-ролл  в формат "Линия выгорания"_3':'стоимость преролл в формат "Линия выгорания"',
                                     'Пре-ролл в подкаст «Сладкая жизнь»_7':'стоимость преролл в подкаст «Сладкая жизнь»',
                                     'Пре-ролл в ролик с Линой Диановой_4':'стоимость преролл в ролик с Линой Диановой',
                                     'Пре-ролл в формат «Книга жалоб»_2':'стоимость преролл в формат «Книга жалоб»',
                                     'Преролл в обзор_3':'стоимость преролл в обзор',
                                     'Преролл в реакцию на трейлер или разборы_5':'стоимость преролл в реакцию на трейлер или разборы',
                                     'Преролл в рецепт_2':'стоимость преролл в рецепт',
                                     'Преролл в формат новости_1':'стоимость преролл в формат новости',
                                     'Продакт-плейсмент_4':'стоимость продакт плейсмент',
                                     'Ссылка в закрепленном комментарии и ссылка в описании до ката_4':'стоимость псылка в закрепленном комментарии и ссылка в описании до ката',
                                    }, inplace=True)

##### Обработаем список sorted_column_names_2

In [495]:
sorted_column_names_2

['3 интеграции_8',
 'Cпецпроект + фото-пост в Инст_8',
 'Интеграция + фото-пост в Инст_9',
 'Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10',
 'Интеграция в видео на YouTube + пост в сообществе YouTube_11',
 'Интеграция в видео на YouTube + фото-пост в Инст_8',
 'Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8',
 'Интеграция в обзор + фото в Инст_8',
 'Интеграция в рецепт + фото в Инст_9',
 'Интеграция на YouTube + Сторис в Instagram_12',
 'Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8',
 'Интеграция на Основном канале + Романов Говорит_8',
 'Пакет соц. сети (сторис ИГ + пост ВК + пост ТГ) (пост в формате «текст прислан», пост и фото от клиента)_8',
 'Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9',
 'Спецпроект + Анонс конкурса_8',
 'Спецпроект на YouTube + пост в Telegram_9',
 'интеграция YouTube + Дзен_8']

In [496]:
avtormedia_youtube.rename(columns = {'3 интеграции_8':'стоимость 3 интеграции',
                                     'Cпецпроект + фото-пост в Инст_8':'стоимость Cпецпроект и фотопост в Инстаграм',
                                     'Интеграция + фото-пост в Инст_9':'стоимость интеграция и фотопост в Инстаграм',
                                     'Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10':'стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
                                     'Интеграция в видео на YouTube + пост в сообществе YouTube_11':'стоимость интеграция в видео на YouTube и пост в сообществе YouTube',
                                     'Интеграция в видео на YouTube + фото-пост в Инст_8':'стоимость интеграция в видео на YouTube и фотопост в Инстаграм',
                                     'Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8':'стоимость интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
                                     'Интеграция в обзор + фото в Инст_8':'стоимость интеграция в обзор и фотопост в Инстаграм',
                                     'Интеграция в рецепт + фото в Инст_9':'стоимость интеграция в рецепт и фотопост в Инстаграм',
                                     'Интеграция на YouTube + Сторис в Instagram_12':'стоимость интеграция на YouTube и сторис в Инстаграм',
                                     'Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8':'стоимость интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
                                     'Интеграция на Основном канале + Романов Говорит_8':'стоимость интеграция на Основном канале + Романов Говорит',
                                     'Спецпроект + Анонс конкурса_8':'стоимость спецпроект и анонс конкурса',
                                     'Спецпроект на YouTube + пост в Telegram_9':'стоимость спецпроект на YouTube и пост в Телеграм',
                                     'интеграция YouTube + Дзен_8':'стоимость интеграция YouTube и Zen'
                                    }, inplace=True)

In [497]:
[col for col in avtormedia_youtube.columns if 'охват' in col]

['охват 8',
 'охват 9',
 'охват 10',
 'охват 11',
 'охват 12',
 'охват 3 интеграции_8',
 'охват Cпецпроект + фото-пост в Инст_8',
 'охват Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8',
 'охват Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8',
 'охват Интеграция в видео на YouTube + фото-пост в Инст_8',
 'охват Спецпроект + Анонс конкурса_8',
 'охват Интеграция в обзор + фото в Инст_8',
 'охват интеграция YouTube + Дзен_8',
 'охват Интеграция на Основном канале + Романов Говорит_8',
 'охват Пакет соц. сети (сторис ИГ + пост ВК + пост ТГ) (пост в формате «текст прислан», пост и фото от клиента)_8',
 'охват Интеграция + фото-пост в Инст_9',
 'охват Спецпроект на YouTube + пост в Telegram_9',
 'охват Интеграция в рецепт + фото в Инст_9',
 'охват Пакет соц. сети авторский (сторис ИГ + пост ВК + пост ТГ) (Авторский пост: текст + фото)_9',
 'охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном коммен

> Отредактируем также название столбов с охватом пакетных предложений

In [498]:
avtormedia_youtube.rename(columns = {'охват 3 интеграции_8':'охват позиции 3 интеграции',
                                     'охват Cпецпроект + фото-пост в Инст_8':'охват позиции Cпецпроект и фотопост в Инстаграм',
                                     'охват Интеграция на YouTube канал Дюшес + сторис в Инст (серия из трех)_8':'охват позиции интеграция на YouTube канал Дюшес и за серию из 3х сторис в Инстаграм',
                                     'охват Интеграция в видео на YouTube Алиса Лисова + интеграция в видео на YouTube Lisova_8':'охват позиции интеграция в видео на YouTube Алиса Лисова и интеграцию в видео на YouTube Lisova',
                                     'охват Интеграция в видео на YouTube + фото-пост в Инст_8':'охват позиции интеграция в видео на YouTube и фотопост в Инстаграм',
                                     'охват Спецпроект + Анонс конкурса_8':'охват позиции спецпроект и анонс конкурса',
                                     'охват Интеграция в обзор + фото в Инст_8':'охват позиции интеграция в обзор и фотопост в Инстаграм',
                                     'охват интеграция YouTube + Дзен_8':'охват позиции интеграция YouTube и Zen',
                                     'охват Интеграция на Основном канале + Романов Говорит_8':'охват позиции интеграция на Основном канале + Романов Говорит',
                                     'охват Интеграция + фото-пост в Инст_9':'охват позиции интеграция и фотопост в Инстаграм',
                                     'охват Спецпроект на YouTube + пост в Telegram_9':'охват позиции спецпроект на YouTube и пост в Телеграм',
                                     'охват Интеграция в рецепт + фото в Инст_9':'охват позиции интеграция в рецепт и фотопост в Инстаграм',
                                     'охват Интеграция в видео на YouTube + пост в сообществе YouTube + ссылка в закрепленном комментарии_10':'охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии',
                                     'охват Интеграция в видео на YouTube + пост в сообществе YouTube_11':'охват позиции интеграция в видео на YouTube и пост в сообществе YouTube',
                                     'охват Интеграция на YouTube + Сторис в Instagram_12':'охват позиции интеграция на YouTube и сторис в Инстаграм'
                                    }, inplace=True)

In [499]:
avtormedia_youtube.drop(columns=['материалы и статистика',
                                 'охват 8',
                                 'охват 9',
                                 'охват 10',
                                 'охват 11',
                                 'охват 12'], inplace=True)

In [500]:
avtormedia_youtube.head()

,блогер,тематика,количество подписчиков,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,среднее кол-во просмотров shorts,...,"стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии","охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии",стоимость интеграция в видео на YouTube и пост в сообществе YouTube,охват позиции интеграция в видео на YouTube и пост в сообществе YouTube,стоимость интеграция на YouTube и сторис в Инстаграм,охват позиции интеграция на YouTube и сторис в Инстаграм,стоимость усложненная интеграция,стоимость пост в сообществе,стоимость выездная интеграция,стоимость интеграция с выбором темы видео
1,himan,челленджи\nэксперименты,19 100 000,NaN,1 200 000,1 100 000 ₽,700 000 ₽,700 000 ₽,3 310 638 ₽,2 200 000,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
2,it's mamix,эксперименты,15 300 000,NaN,8 000 000,6 000 000 ₽,2 000 000 ₽,4 500 000 ₽,NaN,NaN,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
3,вильям бруно,челленджи\nюмор,9 970 000,NaN,1 600 000,500 000 ₽,350 000 ₽,400 000 ₽,2 127 659 ₽,3 400 000,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
4,луана,анимация,6 470 000,NaN,2 350 000,850 000 ₽,600 000 ₽,400 000 ₽,1 500 000 ₽,1 500 000,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN
5,onetwo,юмор\nскетчи,6 000 000,NaN,1 400 000,1 000 000 ₽,600 000 ₽,NaN,1 656 382 ₽,NaN,...,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN


> Отчистим пустые строки и сделаем их как NaN

In [501]:
# # Получаем список столбцов, содержащих слово "охват"
# ohvat_columns = [col for col in avtormedia_youtube.columns if 'охват' in col]

# # Заменяем пустые значения в этих столбцах на NaN
# avtormedia_youtube[ohvat_columns] = avtormedia_youtube[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [502]:
show_duplicates(avtormedia_youtube)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,стоимость мидролл до 30 сек,среднее кол-во просмотров видео,стоимость интеграция 1 слот,стоимость преролл до 30 сек,стоимость интеграция 2 слот,стоимость спецпроект,среднее кол-во просмотров shorts,...,"стоимость интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии","охват позиции интеграция в видео на YouTube, пост в сообществе YouTube и ссылку в закрепленном комментарии",стоимость интеграция в видео на YouTube и пост в сообществе YouTube,охват позиции интеграция в видео на YouTube и пост в сообществе YouTube,стоимость интеграция на YouTube и сторис в Инстаграм,охват позиции интеграция на YouTube и сторис в Инстаграм,стоимость усложненная интеграция,стоимость пост в сообществе,стоимость выездная интеграция,стоимость интеграция с выбором темы видео


### Двадцать восьмое объединение

In [503]:
# # Преобразуйте все столбцы в строки (str)
# avtormedia_youtube = avtormedia_youtube.astype(str)
# bufer = bufer.astype(str)

In [504]:
bufer = merge_bufer_df(bufer, avtormedia_youtube, 'YouTube')

Количество дубликатов: 0


In [505]:
# Посмотрим на размер аудитории

bufer.shape

(838, 262)

### Подготовка таблицы avtormedia_instagram

In [506]:
avtormedia_instagram.head()

,блогер,тематика,подписчики,"материалы, и статистика",охват сториз,сториз,охват постов,фотопост,карусель,охват reels,...,цена,охват,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика
1,даша козловская,творчество\nбьюти\nлайфстайл,2 600 000,Ссылка,155 000,690 000 ₽,281 000,1 063 829 ₽,NaN,370 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/dasha__kozlovskaya/,https://drive.google.com/drive/u/1/folders/1cW...
2,илья варламов,общество\nполитика\nпутешествия,1 300 000,Ссылка,42 000,300 000 ₽,160 000,337 000 ₽,435 000 ₽,362 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/varlamov/?hl=ru,https://drive.google.com/drive/u/1/folders/165...
3,sonya neks,бьюти\nлайфстайл\nспорт,1 100 000,Ссылка,25 000,60 000 ₽,NaN,NaN,NaN,169 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/sonya_neks_,https://drive.google.com/drive/folders/1e7vH1L...
4,makeupkaty,бьюти\nлайфстайл,888 000,Ссылка,137 000,147 872 ₽,228 000,260 638 ₽,337 234 ₽,178 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/makeupkaty/,https://drive.google.com/drive/u/1/folders/1oc...
5,туся,творчество\nбьюти,776 000,Ссылка,53 000,140 000 ₽,51 000,473 404 ₽,614 893 ₽,117 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/tysya/?igshid=1loov0...,https://drive.google.com/drive/u/1/folders/1Xf...


In [507]:
[col for col in avtormedia_instagram.columns]

['блогер',
 'тематика',
 'подписчики',
 'материалы, и статистика',
 'охват сториз',
 'сториз',
 'охват постов',
 'фотопост',
 'карусель',
 'охват reels',
 'reels',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'пакетное предложение',
 'цена',
 'охват',
 'ссылка',
 'статистика']

In [508]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'Instagram город',
 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек',
 'Instagram ца по городам',
 'Instagram ца по возрасту',
 'Instagram ца по полу']

In [509]:
avtormedia_instagram

,блогер,тематика,подписчики,"материалы, и статистика",охват сториз,сториз,охват постов,фотопост,карусель,охват reels,...,цена,охват,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика
1,даша козловская,творчество\nбьюти\nлайфстайл,2 600 000,Ссылка,155 000,690 000 ₽,281 000,1 063 829 ₽,NaN,370 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/dasha__kozlovskaya/,https://drive.google.com/drive/u/1/folders/1cW...
2,илья варламов,общество\nполитика\nпутешествия,1 300 000,Ссылка,42 000,300 000 ₽,160 000,337 000 ₽,435 000 ₽,362 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/varlamov/?hl=ru,https://drive.google.com/drive/u/1/folders/165...
3,sonya neks,бьюти\nлайфстайл\nспорт,1 100 000,Ссылка,25 000,60 000 ₽,NaN,NaN,NaN,169 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/sonya_neks_,https://drive.google.com/drive/folders/1e7vH1L...
4,makeupkaty,бьюти\nлайфстайл,888 000,Ссылка,137 000,147 872 ₽,228 000,260 638 ₽,337 234 ₽,178 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/makeupkaty/,https://drive.google.com/drive/u/1/folders/1oc...
5,туся,творчество\nбьюти,776 000,Ссылка,53 000,140 000 ₽,51 000,473 404 ₽,614 893 ₽,117 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/tysya/?igshid=1loov0...,https://drive.google.com/drive/u/1/folders/1Xf...
6,эмиль зиганша,лайфстайл\nстиль,340 000,Ссылка,40 000,130 023 ₽,55 000,236 406 ₽,513 002 ₽,85 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/zigansha/,https://drive.google.com/drive/u/1/folders/1AP...
7,дмитрий романов,юмор\nлайфстайл\nобщество,307 000,Ссылка,NaN,NaN,NaN,NaN,NaN,43 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/romahadima,https://drive.google.com/drive/u/1/folders/1sm...
8,диана anaidmi,лайфстайл\nмотивация\nспорт,265 000,Ссылка,45 000,100 000 ₽,88 000,140 000 ₽,172 340 ₽,100 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/anaidmi/,https://drive.google.com/drive/u/1/folders/1RF...
9,manilounalina,лайфстайл\nобщество,263 000,Ссылка,26 000,30 000 ₽,NaN,NaN,NaN,70 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://instagram.com/maniloun,https://drive.google.com/drive/folders/1O0ySrt...
10,максим кац,урбанистика\nэкономика\nобщество,189 000,Ссылка,43 000,180 000 ₽,79 000,120 000 ₽,NaN,67 000,...,686 400 ₽,943 000,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/maxim_katz/,https://drive.google.com/drive/folders/1qnIdoG...


In [510]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров 1 сторис',
                 'стоимость 1 сторис',
                 'среднее кол-во просмотров фотопоста',
                 'стоимость фотопост',
                 'стоимость карусель',
                 'среднее кол-во просмотров reels / видеопоста',
                 'стоимость reels / видеопост',
                 'пакетное предложение 1',
                 'цена 1',
                 'охват 1',
                 'пакетное предложение 2',
                 'цена 2',
                 'охват 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'пакетное предложение 4',
                 'цена 4',
                 'охват 4',
                 'ссылка',
                 'статистика']

avtormedia_instagram.columns = bufer_columns

In [511]:
# column_list_3 = []

# for column_number in range(1, 3):
#     position_column = f'спешл {column_number}'
#     price_column = f'цена {column_number}'
#     avtormedia_instagram = get_column_values(avtormedia_instagram,
#                                              column_number,
#                                              position_column,
#                                              price_column,
#                                              column_list_3)

In [512]:
column_list_4 = []

for column_number in range(1, 5):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_instagram = get_column_values_packet(avtormedia_instagram,
                                                    column_number,
                                                    position_column,
                                                    price_column,
                                                    coverage_column,
                                                    column_list_4)

##### Обработаем список sorted_column_names_3

In [513]:
# sorted_column_names_3 = sorted(column_list_3)
# sorted_column_names_3

In [514]:
# avtormedia_instagram.rename(columns = {'Рекламный день (сториз 7-10 экранов)_2':'стоимость рекламный день (сториз 7-10 экранов)',
#                                        'Сториз (5 экранов)_1':'стоимость сториз (5 экранов)'}, inplace = True)

##### Обработаем список sorted_column_names_4

In [515]:
sorted_column_names_4 = sorted(column_list_4)
sorted_column_names_4

['Cторис (серия из трех) + Пост ТГ_4',
 'Интеграция в видео на YouTube + фото-пост в Инст_1',
 'Интеграция в обзор + фото в Инст_2',
 'Интеграция в рецепт + фото в Инст_1',
 'Интеграция на YouTube + Сторис в Instagram_2',
 'Пост в Telegram (удаление через сутки) + Пост в Instagram_1',
 'Пост в ТГ + сторис в ИГ_1',
 'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2',
 'Фото в Инст + сторис в Инст (серия из трех)_1',
 'Фото-пост в Инст + пост в Telegram_1',
 'Фото-пост в Инст + пост в Telegram_3']

In [516]:
dict_new_columns = {'стоимость фотопост в Инстаграм + пост в Телеграм': [ 'Фото-пост в Инст + пост в Telegram_1',
                                                                         'Фото-пост в Инст + пост в Telegram_3'],
                   
                   'охват позиции стоимость фотопост в Инстаграм + пост в Телеграм':['охват Фото-пост в Инст + пост в Telegram_1',
                                                                              'охват Фото-пост в Инст + пост в Telegram_3']}


# avtormedia_instagram = agg_columns(avtormedia_instagram, dict_new_columns)

In [517]:
for new_column, old_column in dict_new_columns.items():
    for col in old_column:
        # Чтобы избежать преобразования значений NaN в строки "nan" при объединении строк,
        # мы будем использовать метод .fillna('') для замены значений NaN на пустые строки перед объединением
        avtormedia_instagram[col] = avtormedia_instagram[col].fillna('').astype(str)
    
    avtormedia_instagram[new_column] = avtormedia_instagram[old_column].agg(''.join, axis=1)  
    
    #Вернём обратно NaN
    avtormedia_instagram[new_column] = avtormedia_instagram[new_column].replace('', np.nan)
    avtormedia_instagram[col] = avtormedia_instagram[col].astype(str)

In [518]:
avtormedia_instagram.rename(columns = {'Cторис (серия из трех) + Пост ТГ_4':'стоимость серия из 3х сторис и пост в Телеграм',
                                       'Интеграция в видео на YouTube + фото-пост в Инст_1':'стоимость интеграция в видео на Ютуб и фотопост в Инстаграм',
                                       'Интеграция в обзор + фото в Инст_2':'стоимость интеграция в обзор и фотопост в Инстаграм',
                                       'Интеграция в рецепт + фото в Инст_1':'стоимость интеграция в рецепт и фотопост в Инстаграм',
                                       'Интеграция на YouTube + Сторис в Instagram_2':'стоимость интеграция на Ютуб и сторис в Инстаграм',
                                       'Пост в Telegram (удаление через сутки) + Пост в Instagram_1':'стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                       'Пост в ТГ + сторис в ИГ_1':'стоимость пост в Телеграм и сторис в Инстаграм',
                                       'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2':'стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
                                       'Фото в Инст + сторис в Инст (серия из трех)_1':'стоимость фотопост в Инстаграм + серию из 3х сторис в Инстаграм'
                                      }, inplace=True)

In [519]:
avtormedia_instagram.columns

Index(['блогер', 'тематика', 'количество подписчиков',
       'материалы и статистика', 'среднее кол-во просмотров 1 сторис',
       'стоимость 1 сторис', 'среднее кол-во просмотров фотопоста',
       'стоимость фотопост', 'стоимость карусель',
       'среднее кол-во просмотров reels / видеопоста',
       'стоимость reels / видеопост', 'охват 1', 'охват 2', 'охват 3',
       'охват 4', 'ссылка', 'статистика',
       'Фото-пост в Инст + пост в Telegram_1',
       'охват Фото-пост в Инст + пост в Telegram_1',
       'стоимость пост в Телеграм и сторис в Инстаграм',
       'охват Пост в ТГ + сторис в ИГ_1',
       'стоимость интеграция в видео на Ютуб и фотопост в Инстаграм',
       'охват Интеграция в видео на YouTube + фото-пост в Инст_1',
       'стоимость фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
       'охват Фото в Инст + сторис в Инст (серия из трех)_1',
       'стоимость интеграция в рецепт и фотопост в Инстаграм',
       'охват Интеграция в рецепт + фото в Инст_1',


In [520]:
avtormedia_instagram.drop(columns=['Фото-пост в Инст + пост в Telegram_1',
                                   'Фото-пост в Инст + пост в Telegram_3',
                                   'охват Фото-пост в Инст + пост в Telegram_1',
                                   'охват Фото-пост в Инст + пост в Telegram_3',
                                      
                                   'охват 1',
                                   'охват 2',
                                   'охват 3',
                                   'охват 4'], inplace=True)

In [521]:
[col for col in avtormedia_instagram.columns if 'охват' in col]

['охват Пост в ТГ + сторис в ИГ_1',
 'охват Интеграция в видео на YouTube + фото-пост в Инст_1',
 'охват Фото в Инст + сторис в Инст (серия из трех)_1',
 'охват Интеграция в рецепт + фото в Инст_1',
 'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_1',
 'охват Интеграция на YouTube + Сторис в Instagram_2',
 'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2',
 'охват Интеграция в обзор + фото в Инст_2',
 'охват Cторис (серия из трех) + Пост ТГ_4',
 'охват позиции стоимость фотопост в Инстаграм + пост в Телеграм']

In [522]:
avtormedia_instagram.rename(columns = {'охват Пост в ТГ + сторис в ИГ_1':'охват позиции стоимость за пост в Телеграм и сторис в Инстаграм',
                                       'охват Интеграция в видео на YouTube + фото-пост в Инст_1':'охват позиции стоимость за интеграцию в видео на Ютуб и фотопост в Инстаграм',
                                       'охват Фото в Инст + сторис в Инст (серия из трех)_1':'охват позиции стоимость за фотопост в Инстаграм + серию из 3х сторис в Инстаграм',
                                       'охват Интеграция в рецепт + фото в Инст_1':'охват позиции стоимость за интеграцию в рецепт и фотопост в Инстаграм',
                                       'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_1':'охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                       'охват Интеграция на YouTube + Сторис в Instagram_2':'охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм',
                                       'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_2':'охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК',
                                       'охват Интеграция в обзор + фото в Инст_2':'охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм',
                                       'охват Cторис (серия из трех) + Пост ТГ_4':'охват позиции стоимость за серию из 3х сторис и пост в Телеграм'
                                      }, inplace = True)
                                   

> Отчистим пустые строки и сделаем их как NaN

In [523]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_instagram.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_instagram[ohvat_columns] = avtormedia_instagram[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [524]:
show_duplicates(avtormedia_instagram)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,материалы и статистика,среднее кол-во просмотров 1 сторис,стоимость 1 сторис,среднее кол-во просмотров фотопоста,стоимость фотопост,стоимость карусель,среднее кол-во просмотров reels / видеопоста,...,стоимость интеграция на Ютуб и сторис в Инстаграм,охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм,"стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК","охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК",стоимость интеграция в обзор и фотопост в Инстаграм,охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм,стоимость серия из 3х сторис и пост в Телеграм,охват позиции стоимость за серию из 3х сторис и пост в Телеграм,стоимость фотопост в Инстаграм + пост в Телеграм,охват позиции стоимость фотопост в Инстаграм + пост в Телеграм


### Двадцать девятое объединение

In [525]:
bufer = merge_bufer_df(bufer, avtormedia_instagram, 'Instagram')

Количество дубликатов: 16


In [526]:
show_duplicates(bufer).head(6)

Количество дубликатов: 16


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram стоимость интеграция на Ютуб и сторис в Инстаграм,Instagram охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм,"Instagram стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК","Instagram охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК",Instagram стоимость интеграция в обзор и фотопост в Инстаграм,Instagram охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм,Instagram стоимость серия из 3х сторис и пост в Телеграм,Instagram охват позиции стоимость за серию из 3х сторис и пост в Телеграм,Instagram стоимость фотопост в Инстаграм + пост в Телеграм,Instagram охват позиции стоимость фотопост в Инстаграм + пост в Телеграм
809,makeupkaty,бьюти\nлайфстайл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
841,makeupkaty,бьюти\nлайфстайл,makeupkaty,https://www.instagram.com/makeupkaty/,бьюти\nлайфстайл,888 000,228 000,137 000,178 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
819,manilounalina,общество,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
846,manilounalina,лайфстайл\nобщество,manilounalina,https://instagram.com/maniloun,лайфстайл\nобщество,263 000,NaN,26 000,70 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
801,qewbite,кино,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
855,qewbite,кино\nлайфстайл,qewbite,https://instagram.com/qewbite,кино\nлайфстайл,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [527]:
# До

bufer.shape

(856, 285)

In [528]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [529]:
# После

bufer.shape

(848, 285)

In [530]:
show_duplicates(bufer)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Instagram стоимость интеграция на Ютуб и сторис в Инстаграм,Instagram охват позиции стоимость за интеграцию на Ютуб и сторис в Инстаграм,"Instagram стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК","Instagram охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм и пост в ВК",Instagram стоимость интеграция в обзор и фотопост в Инстаграм,Instagram охват позиции стоимость за интеграцию в обзор и фотопост в Инстаграм,Instagram стоимость серия из 3х сторис и пост в Телеграм,Instagram охват позиции стоимость за серию из 3х сторис и пост в Телеграм,Instagram стоимость фотопост в Инстаграм + пост в Телеграм,Instagram охват позиции стоимость фотопост в Инстаграм + пост в Телеграм


### Подготовка таблицы avtormedia_telegram

In [531]:
avtormedia_telegram.head(1)

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,короткое видео,пост,спешл,цена,спешл,цена,пакетное предложение,цена,охват,пакетное предложение,цена,охват,ссылка,статистика ссылка
1,илья варламов @varlamov_news,новости,1 200 000,Ссылка,470 000,833 333 ₽,450 000 ₽,Сторис,500 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://t.me/varlamov_news,https://drive.google.com/drive/u/1/folders/165...


In [532]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров поста',
                 'стоимость видеопост',
                 'стоимость фотопост',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'пакетное предложение 4',
                 'цена 4',
                 'охват 4',
                 'ссылка',
                 'статистика']

avtormedia_telegram.columns = bufer_columns

In [533]:
column_list_5 = []

for column_number in range(1, 3):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_telegram = get_column_values(avtormedia_telegram,
                                            column_number,
                                            position_column,
                                            price_column,
                                            column_list_5)

In [534]:
column_list_6 = []

for column_number in range(3, 5):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_telegram = get_column_values_packet(avtormedia_telegram,
                                                   column_number,
                                                   position_column,
                                                   price_column,
                                                   coverage_column,
                                                   column_list_6)

##### Обработаем список sorted_column_names_5

In [535]:
sorted_column_names_5 = sorted(column_list_5)
sorted_column_names_5

['Авторский пост_1',
 'Кружок_1',
 'Кружок_2',
 'Пост 2/48_2',
 'Пост 3/72_1',
 'Пост без удаления_1',
 'Сторис_1']

In [536]:
dict_new_columns = {'стоимость кружочек до 60 сек': ['Кружок_1',
                                                     'Кружок_2']
                   }

avtormedia_telegram = agg_columns(avtormedia_telegram, dict_new_columns)

In [537]:
avtormedia_telegram.drop(columns=['Кружок_1',
                                   'Кружок_2'], inplace=True)

In [538]:
avtormedia_telegram.rename(columns = {'Авторский пост_1':'стоимость за авторский пост',
                                      'Пост без удаления_1':'стоимость за пост из удаления',
                                      'Пост 2/48_2':'стоимость за пост (2/48)',
                                      'Пост 3/72_1':'стоимость за пост (3/72)',
                                      'Сторис_1':'стоимость 1 сторис до 15 сек'}, inplace=True)

##### Обработаем список sorted_column_names_6

In [539]:
sorted_column_names_6 = sorted(column_list_6)
sorted_column_names_6

['3 поста в TG_3',
 '5 постов в Telegram (удаление через сутки)_3',
 'Cторис (серия из трех) + Пост ТГ_4',
 'Пост в Telegram (удаление через сутки) + Пост в Instagram_4',
 'Пост в ТГ + сторис в ИГ_3',
 'Рекламное сообщение во все чаты "Вместе"_3',
 'Спецпроект на YouTube + пост в Telegram_3',
 'Фото-пост в Инст + пост в Telegram_3']

In [540]:
avtormedia_telegram.rename(columns = {'3 поста в TG_3':'стоимость 3 поста в Телеграм',
                                      '5 постов в Telegram (удаление через сутки)_3':'стоимость 5 постов в Телеграм (удаление через сутки)',
                                      'Cторис (серия из трех) + Пост ТГ_4':'стоимость серия из 3х сторис в Инстаграм и пост в Телеграм',
                                      'Пост в Telegram (удаление через сутки) + Пост в Instagram_4':'стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                      'Пост в ТГ + сторис в ИГ_3':'стоимость пост в Телеграм и сториз в Инстаграм',
                                      'Рекламное сообщение во все чаты "Вместе"_3':'стоимость рекламное сообщение во все чаты "Вместе"',
                                      'Спецпроект на YouTube + пост в Telegram_3':'стоимость спецпроект на Ютуб и пост в Телеграм',
                                      'Фото-пост в Инст + пост в Telegram_3':'стоимость фотопост в Инстаграм и пост в Телеграм'
                                     }, inplace=True)

In [541]:
avtormedia_telegram.rename(columns = {'охват Фото-пост в Инст + пост в Telegram_3':'охват позиции стоимость за фотопост в Инстаграм и пост в Телеграм',
                                      'охват Спецпроект на YouTube + пост в Telegram_3':'охват позиции стоимость за спецпроект на Ютуб и пост в Телеграм',
                                      'охват Пост в ТГ + сторис в ИГ_3':'охват позиции стоимость за пост в Телеграм и сториз в Инстаграм',
                                      'охват 5 постов в Telegram (удаление через сутки)_3':'охват позиции стоимость за 5 постов в Телеграм (удаление через сутки)',
                                      'охват 3 поста в TG_3':'охват позиции стоимость за 3 поста в Телеграм',
                                      'охват Рекламное сообщение во все чаты "Вместе"_3':'охват позиции стоимость за рекламное сообщение во все чаты "Вместе"',
                                      'охват Пост в Telegram (удаление через сутки) + Пост в Instagram_4':'охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм',
                                      'охват Cторис (серия из трех) + Пост ТГ_4':'охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм'
                                     }, inplace=True)

In [542]:
avtormedia_telegram.drop(columns=['материалы и статистика',
                                  'охват 3',
                                  'охват 4'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [543]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_telegram.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_telegram[ohvat_columns] = avtormedia_telegram[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [544]:
show_duplicates(avtormedia_telegram)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,среднее кол-во просмотров поста,стоимость видеопост,стоимость фотопост,ссылка,статистика,стоимость 1 сторис до 15 сек,стоимость за авторский пост,...,охват позиции стоимость за 5 постов в Телеграм (удаление через сутки),стоимость 3 поста в Телеграм,охват позиции стоимость за 3 поста в Телеграм,"стоимость рекламное сообщение во все чаты ""Вместе""","охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм,стоимость кружочек до 60 сек


### Тридцатое объединение

In [545]:
bufer = merge_bufer_df(bufer, avtormedia_telegram, 'Telegram')

Количество дубликатов: 40


In [546]:
# duplicates = bufer[bufer.duplicated('блогер', keep=False)]
# duplicates.sort_values(by='блогер').head(6)

show_duplicates(bufer).head(6)

Количество дубликатов: 40


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Telegram стоимость 5 постов в Телеграм (удаление через сутки),Telegram охват позиции стоимость за 5 постов в Телеграм (удаление через сутки),Telegram стоимость 3 поста в Телеграм,Telegram охват позиции стоимость за 3 поста в Телеграм,"Telegram стоимость рекламное сообщение во все чаты ""Вместе""","Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм
871,demin's lounge,техно-блог,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,50 000 ₽,30 000,NaN,NaN,NaN,NaN,NaN,NaN
808,demin's lounge,техно-блог,None,None,техно-блог,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
864,droider,техно-блог,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,droider,техно-блог,None,None,техно-блог,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854,himan,челленджи\nэксперименты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791,himan,"челленджи, эксперименты",None,None,"челленджи, эксперименты",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [547]:
# До

bufer.shape

(875, 306)

In [548]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [549]:
# После

bufer.shape

(855, 306)

In [550]:
show_duplicates(bufer)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Telegram стоимость 5 постов в Телеграм (удаление через сутки),Telegram охват позиции стоимость за 5 постов в Телеграм (удаление через сутки),Telegram стоимость 3 поста в Телеграм,Telegram охват позиции стоимость за 3 поста в Телеграм,"Telegram стоимость рекламное сообщение во все чаты ""Вместе""","Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм


### Подготовка таблицы avtormedia_vk

In [551]:
avtormedia_vk.head()

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,пост,vk-клип,сторис,пакетное предложение,цена,охват,ссылка,статистика
1,илья варламов,общество\nурбанистика\nполитика,301 000,Ссылка,8 000,87 777 ₽,125 000 ₽,37 500 ₽,NaN,NaN,NaN,https://vk.com/varlamov,https://drive.google.com/drive/u/1/folders/165...
2,qewbite,кино,241 000,Ссылка,20 000,30 000 ₽,NaN,NaN,NaN,NaN,NaN,https://vk.com/qewbite,https://drive.google.com/drive/u/1/folders/1wo...
3,himan,челленджи\nэксперименты,227 000,Ссылка,40 000,60 000 ₽,NaN,NaN,NaN,NaN,NaN,https://vk.com/hibestman,https://drive.google.com/drive/folders/1AG9et9...
4,onetwo,скетчи\nюмор,221 000,Ссылка,15 000,38 297 ₽,53 191 ₽,27 659 ₽,NaN,NaN,NaN,https://vk.com/onetwo_pub,https://drive.google.com/drive/u/1/folders/1HC...
5,тим ильясов,лайфстайл\nмода,105 000,Ссылка,35 000,35 000 ₽,105 000 ₽,NaN,Фото в Инст + сторис в Инст (серия из трех) + ...,105 000 ₽,57 000,https://vk.com/tim_iliasov,https://drive.google.com/drive/u/1/folders/1Uz...


In [552]:
[col for col in bufer.columns if 'VK группа' in col]

['VK группа блогер',
 'VK группа ссылка',
 'VK группа тематика',
 'VK группа количество подписчиков',
 'VK группа среднее кол-во просмотров поста',
 'VK группа среднее кол-во просмотров вк клипа',
 'VK группа среднее кол-во лайков',
 'VK группа среднее кол-во комментариев',
 'VK группа стоимость фотопост',
 'VK группа стоимость видеопост',
 'VK группа стоимость вк клип',
 'VK группа статистика',
 'VK группа город',
 'VK группа стоимость передачи прав на 1 месяц',
 'VK группа ца по городам',
 'VK группа ца по возрасту',
 'VK группа ца по полу']

In [553]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров поста',
                 'стоимость вк клип',
                 'стоимость сторис',
                 'стоимость фотопост',
                 'пакетное предложение 1',
                 'цена 1',
                 'охват 1',
                 'ссылка',
                 'статистика']

avtormedia_vk.columns = bufer_columns

In [554]:
column_list_7 = []

for column_number in range(1, 2):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_vk = get_column_values_packet(avtormedia_vk,
                                             column_number,
                                             position_column,
                                             price_column,
                                             coverage_column,
                                             column_list_7)

##### Обработаем список sorted_column_names_7

In [555]:
sorted_column_names_7 = sorted(column_list_7)
sorted_column_names_7

['Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1',
 'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1']

In [556]:
avtormedia_vk.rename(columns = {'Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1':'стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
                                'Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1':'стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
                                'охват Фото в Инст + сторис в Инст (серия из трех) + пост в ВК_1':'охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК',
                                'охват Сайт + соцсети (публикации на Droider.ru, FB, Twitter, Вконтакте)_1':'охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)',
                               }, inplace=True)

In [557]:
[col for col in avtormedia_vk.columns]

avtormedia_vk.drop(columns=['материалы и статистика',
                            'охват 1'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [558]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_vk.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_vk[ohvat_columns] = avtormedia_vk[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [559]:
show_duplicates(avtormedia_vk)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,среднее кол-во просмотров поста,стоимость вк клип,стоимость сторис,стоимость фотопост,ссылка,статистика,"стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)"


### Тридцать первое объединение

In [560]:
bufer = merge_bufer_df(bufer, avtormedia_vk, 'VK группа')

Количество дубликатов: 18


In [561]:
show_duplicates(bufer)

Количество дубликатов: 18


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,"Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм,VK группа стоимость сторис,"VK группа стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)"
860,droider,техно-блог,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,142 553 ₽,NaN,NaN,193 262 ₽,157 500
808,droider,техно-блог,None,None,техно-блог,None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
791,himan,"челленджи, эксперименты",None,None,"челленджи, эксперименты",None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
857,himan,челленджи\nэксперименты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
801,onetwo,"скетчи, юмор",None,None,"скетчи, юмор",None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
858,onetwo,скетчи\nюмор,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,53 191 ₽,NaN,NaN,NaN,NaN
805,qewbite,"кино, лайфстайл",qewbite,https://instagram.com/qewbite,"кино, лайфстайл",,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
856,qewbite,кино,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
864,дюшес,челленджи\nэксперименты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,70 000 ₽,NaN,NaN,NaN,NaN
794,дюшес,"челленджи, эксперименты, юмор",None,None,"челленджи, эксперименты, юмор",None,None,None,None,None,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN


In [562]:
# До

bufer.shape

(865, 311)

In [563]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [564]:
# После

bufer.shape

(856, 311)

In [565]:
show_duplicates(bufer)

Количество дубликатов: 0


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,"Telegram охват позиции стоимость за рекламное сообщение во все чаты ""Вместе""",Telegram стоимость пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram охват позиции стоимость за пост в Телеграм (удаление через сутки) и пост в Инстаграм,Telegram стоимость серия из 3х сторис в Инстаграм и пост в Телеграм,Telegram охват позиции стоимость за серию из 3х сторис в Инстаграм и пост в Телеграм,VK группа стоимость сторис,"VK группа стоимость фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа охват позиции стоимость за фотопост в Инстаграм, серию из 3х сторис в Инстаграм, пост в ВК","VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)"


### Подготовка таблицы avtormedia_zen

In [566]:
avtormedia_zen.head()

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,пост,интеграция в видео,статья,видео,спешл,цена,спешл,цена,пакетное предложение,цена,охват,ссылка,статистика
1,популярная наука,научпоп,432 000,Ссылка,35 000,90 425 ₽,NaN,90 425 ₽,101 063 ₽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://dzen.ru/nauka,https://drive.google.com/drive/u/1/folders/13B...
2,андрей бунбич,кулинария,246 000,Ссылка,25 000,71 276 ₽,36 170 ₽,71 276 ₽,82 978 ₽,Модерация комментариев,29 787 ₽,Выездная интеграция в видео,47 872 ₽,Статья в Яндекс.Дзен + фото-пост в Инст,78 368 ₽,25 000,https://dzen.ru/bunbich,https://drive.google.com/drive/folders/1-ZzMVf...


In [567]:
[col for col in bufer.columns if 'Дзен' in col]

['Дзен блогер',
 'Дзен количество подписчиков',
 'Дзен среднее кол-во просмотров',
 'Дзен стоимость пост',
 'Дзен ссылка',
 'Дзен город',
 'Дзен название канала',
 'Дзен тематика',
 'Дзен стоимость видеопост']

In [568]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во просмотров',
                 'стоимость интеграция до 1 мин',
                 'стоимость пост',
                 'стоимость статья',
                 'стоимость за видеопост',
                 'спешл 1',
                 'цена 1',
                 'спешл 2',
                 'цена 2',
                 'пакетное предложение 3',
                 'цена 3',
                 'охват 3',
                 'ссылка',
                 'статистика']

avtormedia_zen.columns = bufer_columns

In [569]:
column_list_8 = []

for column_number in range(1, 3):
    position_column = f'спешл {column_number}'
    price_column = f'цена {column_number}'
    avtormedia_zen = get_column_values(avtormedia_zen,
                                       column_number,
                                       position_column,
                                       price_column,
                                       column_list_8)

In [570]:
column_list_9 = []

for column_number in range(3, 4):
    position_column = f'пакетное предложение {column_number}'
    price_column = f'цена {column_number}'
    coverage_column = f'охват {column_number}'
    avtormedia_zen = get_column_values_packet(avtormedia_zen,
                                              column_number,
                                              position_column,
                                              price_column,
                                              coverage_column,
                                              column_list_9)

##### Обработаем список sorted_column_names_8

In [571]:
sorted_column_names_8 = sorted(column_list_8)
sorted_column_names_8

['Выездная интеграция в видео_2', 'Модерация комментариев_1']

In [572]:
avtormedia_zen.rename(columns = {'Выездная интеграция в видео_2':'стоимость за выездную интеграцию в видео',
                                 'Модерация комментариев_1':'стоимость за модерацию комментариев'
                                }, inplace=True)

##### Обработаем список sorted_column_names_9

In [573]:
sorted_column_names_9 = sorted(column_list_9)
sorted_column_names_9

['Статья в Яндекс.Дзен + фото-пост в Инст_3']

In [574]:
avtormedia_zen.rename(columns = {'Статья в Яндекс.Дзен + фото-пост в Инст_3':'стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм',
                                 'охват Статья в Яндекс.Дзен + фото-пост в Инст_3':'охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм'}, inplace = True)

In [575]:
avtormedia_zen.drop(columns=['материалы и статистика',
                             'охват 3'], inplace=True)

> Отчистим пустые строки и сделаем их как NaN

In [576]:
# Получаем список столбцов, содержащих слово "охват"
ohvat_columns = [col for col in avtormedia_zen.columns if 'охват' in col]

# Заменяем пустые значения в этих столбцах на NaN
avtormedia_zen[ohvat_columns] = avtormedia_zen[ohvat_columns].replace('', np.nan)

#### Проверка на дубликаты

In [577]:
show_duplicates(avtormedia_zen)

Количество дубликатов: 0


,блогер,тематика,количество подписчиков,среднее кол-во просмотров,стоимость интеграция до 1 мин,стоимость пост,стоимость статья,стоимость за видеопост,ссылка,статистика,стоимость за модерацию комментариев,стоимость за выездную интеграцию в видео,стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм,охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм


### Тридцать второе объединение

In [578]:
bufer = merge_bufer_df(bufer, avtormedia_zen, 'Дзен')

Количество дубликатов: 4


In [579]:
show_duplicates(bufer)

Количество дубликатов: 4


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,"VK группа стоимость сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)","VK группа охват позиции стоимость за сайт и соцсети (публикации на Droider.ru, FB, Твиттер, Вконтакте)",Дзен стоимость интеграция до 1 мин,Дзен стоимость статья,Дзен стоимость за видеопост,Дзен статистика,Дзен стоимость за модерацию комментариев,Дзен стоимость за выездную интеграцию в видео,Дзен стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм,Дзен охват позиции стоимость за статью в Яндекс.Дзен и фотопост в Инстаграм
851,андрей бунбич,"кулинария, лайфстайл",андрей бунбич,https://www.instagram.com/bunbich/,"кулинария, лайфстайл",94 500,None,None,6 000,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,андрей бунбич,кулинария,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,71 276 ₽,71 276 ₽,82 978 ₽,https://drive.google.com/drive/folders/1-ZzMVf...,29 787 ₽,47 872 ₽,78 368 ₽,25 000
818,популярная наука,научпоп,None,None,научпоп,None,None,None,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
856,популярная наука,научпоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90 425 ₽,90 425 ₽,101 063 ₽,https://drive.google.com/drive/u/1/folders/13B...,NaN,NaN,NaN,NaN


In [580]:
# До

bufer.shape

(858, 319)

In [581]:
col_to_check = 'блогер'
columns_to_upd = 'тематика'
bufer = duplicates_stroka(bufer, col_to_check, columns_to_upd)

In [582]:
# После

bufer.shape

(856, 319)

### Подготовка таблицы avtormedia_podcast

In [583]:
avtormedia_podcast.head()

,блогер,тематика,подписчики,"материалы, и статистика",средние просмотры,рекламная вставка,партнерская рубрика,ссылка,статистика
1,droidercast,техно-блог,,Ссылка,NaN,66 193 ₽,106 382 ₽,https://droidercast.podster.fm/,https://drive.google.com/drive/u/1/folders/1dl...


In [584]:
[col for col in avtormedia_podcast.columns]

['блогер',
 'тематика',
 'подписчики',
 'материалы, и статистика',
 'средние просмотры',
 'рекламная вставка',
 'партнерская рубрика',
 'ссылка',
 'статистика']

In [585]:
bufer_columns = ['блогер',
                 'тематика',
                 'количество подписчиков',
                 'материалы и статистика',
                 'среднее кол-во прослушиваний',
                 'стоимость рекламная вставка',
                 'стоимость партнерская рубрика',
                 'ссылка',
                 'статистика']

avtormedia_podcast.columns = bufer_columns

In [586]:
avtormedia_podcast.drop(columns='материалы и статистика', inplace=True)

### Тридцать третье объединение

In [587]:
bufer = merge_bufer_df(bufer, avtormedia_podcast, 'Подкаст')

Количество дубликатов: 0


In [588]:
# После

bufer.shape

(857, 327)

In [589]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('reklama@avtormedia.ru', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('Стоимость указана с налогами', inplace=True)

## Подготовка данных по блогерам от PLATFORMA.COMMUNITY

In [590]:
key_word = ['Вид контракта', 'CPV']  # Список ключевых слов для поиска столбцов и их удаления
drop_word = ', до учета налогов'  # Список ключевых слов для их удаления из названия столбцов
upd_columns = ['Канал']

platformacommunity_youtube = pretraining_df(platformacommunity_youtube, key_word, drop_word, upd_columns)

### Подготовка таблицы platformacommunity_youtube

In [591]:
platformacommunity_youtube.head(10)

,канал,"проект, ведущий, формат",тематика,подписчики,средние просмотры для рекламных интеграций,статистика,"pre-roll / olv, от, руб.","mid-roll / olv, от, руб.","интеграция 1-ый слот, от, руб.","интеграция 2-ой слот, от, руб.","продакт плейсмент, от, руб.","размещение в сообществе, от, руб.",пример,ссылка,статистика ссылка
0,уральские пельмени,Официальный канал шоу «Уральские Пельмени»,Специальные сборки номеров под рекламодателя,"2 550 000,00","2 000 000,00",Статистика,"1 200 000,00",NaN,"2 500 000,00",NaN,NaN,NaN,Тинькофф,https://www.youtube.com/c/ProPelmeny/featured,https://drive.google.com/drive/folders/1vQIyBd...
1,NaN,NaN,Эксклюзивные номера,NaN,"1 100 000,00",NaN,"850 000,00",NaN,NaN,NaN,NaN,NaN,Пятерочка,NaN,NaN
2,NaN,NaN,Регулярные выкладки,NaN,"700 000,00",NaN,"600 000,00",NaN,NaN,NaN,NaN,NaN,HAVAL,NaN,NaN
3,NaN,NaN,"""ЕДЕМ С ПЕЛЬМЕНЯМИ"" travel шоу про гастроли и ...",NaN,"300 000,00",NaN,NaN,NaN,NaN,NaN,"800 000,00",NaN,Орматек,NaN,NaN
4,вписка,Коля Редькин,Интервью,"1 910 000,00","2 500 000,00",Статистика,"1 500 000,00",по запросу,по запросу,NaN,по запросу,NaN,NaN,https://www.youtube.com/@VPISKA/featured,https://drive.google.com/drive/folders/1KxM-Mc...
5,истории русской попсы,Олег Кармунин,"Музыкальный критик и автор телеграм-канала ""Ру...","149 000,00","150 000,00",Статистика,"100 000,00","80 000,00","130 000,00","100 000,00",NaN,NaN,NaN,https://www.youtube.com/@ruspopshow/videos,https://drive.google.com/drive/folders/1N2NTA5...
6,кликклак,"Руслан Усачев, Эльдар Джарахов, Юрий Музыченко...","Канал друзей, где не прекращаются трэш, угар и...","6 990 000,00","2 000 000,00",Статистика,"1 650 000,00",NaN,"3 300 000,00",NaN,NaN,NaN,NaN,https://www.youtube.com/@klikklak,https://drive.google.com/drive/folders/1II3xtx...
7,джарахов,Эльдар Джарахов,"Популярный блогер, комик, музыкант","3 760 000,00","500 000,00",Статистика,"440 000,00",NaN,"660 000,00",NaN,NaN,NaN,NaN,https://www.youtube.com/@dzharakhov,https://drive.google.com/drive/folders/14Initg...
8,брокен дэнс,Николай Редькин – российский музыкальный журна...,Видео-эссе о современной зарубежной и российск...,"254 000,00","200 000,00",Статистика,NaN,NaN,"300 000,00","150 000,00",NaN,NaN,NaN,https://www.youtube.com/@broken_dance,https://drive.google.com/drive/folders/1lXd1YR...
9,prnrp,"Даня Порнорэп - музыкальный критик, журналист ...",Анализ современной музыки на стыке искусствове...,"20 000,00","50 000,00",Статистика,NaN,NaN,"50 000,00","30 000,00",NaN,NaN,NaN,https://www.youtube.com/@trap_critic,https://drive.google.com/drive/folders/1iUO5CB...


> Заполним имеющиеся пропуски "сверху вниз"

In [592]:
platformacommunity_youtube[['канал', 'проект, ведущий, формат', 'подписчики', 'ссылка', 'статистика ссылка']] = \
            platformacommunity_youtube[['канал', 'проект, ведущий, формат', 'подписчики', 'ссылка', 'статистика ссылка']].fillna(method='ffill')

> Удалим не нужные нам столбцы

In [593]:
platformacommunity_youtube.drop(columns=['статистика', 'пример'], inplace=True)

In [594]:
platformacommunity_youtube.columns

Index(['канал', 'проект, ведущий, формат', 'тематика', 'подписчики',
       'средние просмотры для рекламных интеграций',
       'pre-roll / olv, от, руб.', 'mid-roll / olv, от, руб.',
       'интеграция 1-ый слот, от, руб.', 'интеграция 2-ой слот, от, руб.',
       'продакт плейсмент, от, руб.', 'размещение в сообществе, от, руб.',
       'ссылка', 'статистика ссылка'],
      dtype='object')

> На данный момент мы не будем объединять столбцы 'канал' и 'проект, ведущий, формат', так как тогда мы не сможем правильно объединить

In [595]:
# platformacommunity_youtube['блогер'] = platformacommunity_youtube.apply(
#     lambda row: row['канал'] + ' - ' + row['проект, ведущий, формат'] if len(row['проект, ведущий, формат']) < 40 else '',
#     axis=1
# )

In [596]:
# platformacommunity_youtube['блогер'] = \
#                 platformacommunity_youtube['канал'] + ' - ' + platformacommunity_youtube['проект, ведущий, формат']

# platformacommunity_youtube.drop(columns=['статистика аудитории',
#                                          'проект, ведущий, формат'], axis=1, inplace= True)

In [597]:
[col for col in bufer.columns if 'YouTube' in col]

['YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouTube статистика',
 'YouTube блогер',
 'YouTube стоимость интеграция 2 слот',
 'YouTube среднее кол-во просмотров',
 'YouTube стоимость мини-интеграция до 1 мин',
 'YouTube стоимость интеграция в стрим',
 'YouTube стоимость спонсорство',
 'YouTube стоимость выпуск под клиента',
 'YouTube стоимость нативная интеграция',
 'YouTube город',
 'YouTube название канала',
 'YouTube стоимость за shorts до 1 мин',
 'YouTube ца по возрасту',
 'YouTube ца по полу',
 'YouTube ца по странам',
 'YouTube стоимость спецпроект',
 'YouTube стоимость бегущая строка',
 'YouT

In [598]:
[col for col in platformacommunity_youtube.columns]

['канал',
 'проект, ведущий, формат',
 'тематика',
 'подписчики',
 'средние просмотры для рекламных интеграций',
 'pre-roll / olv, от, руб.',
 'mid-roll / olv, от, руб.',
 'интеграция 1-ый слот, от, руб.',
 'интеграция 2-ой слот, от, руб.',
 'продакт плейсмент, от, руб.',
 'размещение в сообществе, от, руб.',
 'ссылка',
 'статистика ссылка']

In [599]:
bufer_columns = ['блогер',
                 'проект, ведущий, формат',
                 'шоу',
                 'количество подписчиков',
                 'среднее кол-во просмотров',
                 'стоимость преролл до 30 сек',
                 'стоимость мидролл до 30 сек',
                 'стоимость интеграция 1 слот',
                 'стоимость интеграция 2 слот',
                 'стоимость продакт плейсмент',
                 'стоимость пост в сообществе',
                 'ссылка',
                 'статистика']

In [600]:
platformacommunity_youtube.columns = bufer_columns

### Тридцать четвёртое объединение

In [601]:
platformacommunity_youtube.loc\
    [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

platformacommunity_youtube.loc\
    [platformacommunity_youtube['блогер'] == 'стс', 'блогер']  = 'телеканал стс'

# platformacommunity_youtube.loc\
#     [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

# platformacommunity_youtube.loc\
#     [platformacommunity_youtube['блогер'] == 'ю', 'блогер']  = 'телеканал ю'

In [602]:
bufer = merge_bufer_df(bufer, platformacommunity_youtube, 'YouTube')

Количество дубликатов: 48


In [603]:
show_duplicates(bufer)

Количество дубликатов: 48


,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Подкаст блогер,Подкаст тематика,Подкаст количество подписчиков,Подкаст среднее кол-во прослушиваний,Подкаст стоимость рекламная вставка,Подкаст стоимость партнерская рубрика,Подкаст ссылка,Подкаст статистика,"YouTube проект, ведущий, формат",YouTube шоу
933,hardcore media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Epic Fighting,"Лицом к лицу сойдутся звезды интернета, популя..."
932,hardcore media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hardcore MMA,Профессиональный Российский промоушен смешанны...
931,hardcore media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hardcore Fighting Championship,Первый в России и крупнейший в мире спортивный...
916,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Разгоны\r,NaN
920,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,НеТурист,NaN
919,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Офис,NaN
918,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Что у вас случилось?,NaN
917,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Книжный клуб,NaN
915,stand-up club #1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Самый Умный Комик\r,Юмор
905,александр шебанов,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ежедневный выпуск лучших мемов,NaN


### Подготовка таблицы platformacommunity_instagram

In [604]:
platformacommunity_instagram.head()

,Персоналии,Страница,Подписчики,Тематика,Статистика и инфо,Цены,Unnamed: 6,Unnamed: 7,"ATL активации, предоставление артиста в рекламные кампании"
0,NaN,NaN,NaN,NaN,NaN,Фотопост,Сторис,Видеопост,NaN
1,NaN,NaN,NaN,NaN,NaN,От,От,От,NaN
2,LOVV66,https://instagram.com/lovv66,"634 000,00","Рэпер. Один из главных фрешменов, по мнению ТН...",NaN,"300 000,00","120 000,00","300 000,00",по запросу
3,SCALLY MILANO,https://instagram.com/scallymilano,"456 000,00",Рэпер. Дмитриев. Считается одним из первых в Р...,NaN,"400 000,00","140 000,00","400 000,00",по запросу
4,TENDERLYBAE,https://instagram.com/tenderlybae,"3 900 000,00",Стримерша и певица. В 2021 году Амина победила...,NaN,"300 000,00","150 000,00","400 000,00",по запросу


In [605]:
platformacommunity_instagram.drop(columns=['Статистика и инфо',
                                           'ATL активации, предоставление артиста в рекламные кампании'], inplace=True)

In [606]:
platformacommunity_instagram['Персоналии'] = platformacommunity_instagram['Персоналии'].str.lower()

In [607]:
platformacommunity_instagram.columns

Index(['Персоналии', 'Страница', 'Подписчики', 'Тематика', 'Цены ',
       'Unnamed: 6', 'Unnamed: 7'],
      dtype='object')

In [608]:
[col for col in bufer.columns if 'Instagram' in col]

['Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'Instagram город',
 'Instagram среднее кол-во просмотров серии сторис',
 'Instagram стоимость серия сторис до 60 сек',
 'Instagram ца по городам',
 'Instagram ца по возрасту',
 'Instagram ца по полу',
 'Instagram материалы и статистика',
 'Instagram стоимость 1 сторис',
 'Instagram стоимость карусель',
 'Instagram стоимость пост в Телеграм и сторис в Инстаграм',
 'Instagram охват позиции стоимость за пост в Телеграм и сторис в Инстаграм',
 'Instagram стоимость 

In [609]:
bufer_columns = ['блогер',
                 'ссылка',
                 'количество подписчиков',
                 'тематика',
                 'стоимость фотопост',
                 'стоимость 1 сторис',
                 'стоимость reels / видеопост']

platformacommunity_instagram.columns = bufer_columns

In [610]:
platformacommunity_instagram.head()

,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост
0,NaN,NaN,NaN,NaN,Фотопост,Сторис,Видеопост
1,NaN,NaN,NaN,NaN,От,От,От
2,lovv66,https://instagram.com/lovv66,"634 000,00","Рэпер. Один из главных фрешменов, по мнению ТН...","300 000,00","120 000,00","300 000,00"
3,scally milano,https://instagram.com/scallymilano,"456 000,00",Рэпер. Дмитриев. Считается одним из первых в Р...,"400 000,00","140 000,00","400 000,00"
4,tenderlybae,https://instagram.com/tenderlybae,"3 900 000,00",Стримерша и певица. В 2021 году Амина победила...,"300 000,00","150 000,00","400 000,00"


In [611]:
platformacommunity_instagram[platformacommunity_instagram['блогер'] == 'полина маришова']

,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост
27,полина маришова,instagram,223 000,Тренер по речи,Только комплексный вид рекламы 6-7 сторис + по...,NaN,NaN


In [612]:
platformacommunity_instagram = platformacommunity_instagram.dropna(subset=['блогер'])

In [613]:
platformacommunity_instagram.loc[platformacommunity_instagram['блогер'] == 'рита дакота', 'стоимость видеопост и серия сторис'] = 750000
platformacommunity_instagram.loc[platformacommunity_instagram['блогер'] == 'полина маришова', 'стоимость фотопост и 6-7 сторис'] = 600000

In [614]:
platformacommunity_instagram['стоимость фотопост'] = \
platformacommunity_instagram['стоимость фотопост'].replace('Видео пост + серия сторис 750 000', np.nan)

platformacommunity_instagram['стоимость фотопост'] = \
            platformacommunity_instagram['стоимость фотопост'].replace('Только комплексный вид рекламы 6-7 сторис + пост 600 000', np.nan)

In [615]:
platformacommunity_instagram

,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост,стоимость видеопост и серия сторис,стоимость фотопост и 6-7 сторис
2,lovv66,https://instagram.com/lovv66,"634 000,00","Рэпер. Один из главных фрешменов, по мнению ТН...","300 000,00","120 000,00","300 000,00",NaN,NaN
3,scally milano,https://instagram.com/scallymilano,"456 000,00",Рэпер. Дмитриев. Считается одним из первых в Р...,"400 000,00","140 000,00","400 000,00",NaN,NaN
4,tenderlybae,https://instagram.com/tenderlybae,"3 900 000,00",Стримерша и певица. В 2021 году Амина победила...,"300 000,00","150 000,00","400 000,00",NaN,NaN
5,pinq,https://instagram.com/pinq,"153 000,00",Рэп-исполнитель и дизайнер,"75 000,00","40 000,00","75 000,00",NaN,NaN
6,шайни,https://instagram.com/shynegswagga,"143 000,00",Рэп-исполнитель,"45 000,00","25 000,00","90 000,00",NaN,NaN
7,яна логвина,https://www.instagram.com/logvina.ya/,"686 000,00",Трансформационный тренер,"150 000,00","120 000,00","200 000,00",NaN,NaN
8,зоя зиновьева,https://instagram.com/zzoya_diary?igshid=NTc4M...,"65 000,00","Актриса, артист-вокалист","60 000,00","50 000,00","100 000,00",NaN,NaN
9,анастасия исаева,https://instagram.com/nnassie?igshid=MzRlODBiN...,29 000,Медийный коуч,"15 000,00","15 000,00","20 000,00",NaN,NaN
10,ольяная екатерина,https://www.instagram.com/ekaterina.olyanaya,60 000,"Таролог, мотивационный спикер","20 000,00","15 000,00","30 000,00",NaN,NaN
11,света солар,https://instagram.com/svetasollar?igshid=1aj82...,1 300 000,"Блогер лайфстайл, юмор",70 000,50 000,80 000,NaN,NaN


In [616]:
show_duplicates(platformacommunity_instagram)

Количество дубликатов: 0


,блогер,ссылка,количество подписчиков,тематика,стоимость фотопост,стоимость 1 сторис,стоимость reels / видеопост,стоимость видеопост и серия сторис,стоимость фотопост и 6-7 сторис


### Тридцать пятое объединение

In [617]:
bufer = merge_bufer_df(bufer, platformacommunity_instagram, 'Instagram')

Количество дубликатов: 48


### Подготовка таблицы avtormedia_telegram

In [618]:
platformacommunity_telegram.head()

,Канал,Проект / ведущий / идеолог,Тематика,Подписчики,"Пост (текст) с закреплением в ТОП, до учета налогов","Пост (текст) без закрепления в ТОП, до учета налогов","Пост (текст+фото) , до учета налогов","Пост (видео) , до учета налогов","Пост (кружочек) , до учета налогов",Unnamed: 9
0,Лина Дианова,Лина Дианова,Психология,35 000,"90 000,00","60 000,00",NaN,NaN,NaN,https://t.me/lina_dianova
1,KADI,Диана Кади,Интервью,6 500,"25 000,00","20 000,00",NaN,NaN,NaN,https://t.me/kadinew
2,Серотонин,"Профессиональные журналисты глянца (GQ, Vogue,...",Lifestyle,13 000,NaN,NaN,"15 000,00","15 000,00","15 000,00",https://t.me/+i5_0knOc8SE3M2Y0
3,Лиза Лазерсон,"Феминистка, блогер, ведушая ""Эхо Москвы"" Лиза ...",Социальная и общественная публицистика,34 000,"55 000,00","35 000,00",NaN,NaN,NaN,https://t.me/znachit_net
4,Plarforma. Producer hub,Platforma Agency,Инфо о рынке digital видео спонсорства,370,NaN,по теме - бесплатно,NaN,NaN,NaN,https://t.me/digitalproducerhub


In [619]:
platformacommunity_telegram['Канал'] = platformacommunity_telegram['Канал'].str.lower()

In [620]:
[col for col in bufer.columns if 'Telegram' in col]

['Telegram блогер',
 'Telegram ссылка',
 'Telegram тематика',
 'Telegram количество подписчиков',
 'Telegram среднее кол-во просмотров поста',
 'Telegram средняя вовлечённость',
 'Telegram стоимость фотопост',
 'Telegram стоимость видеопост',
 'Telegram стоимость кружочек до 60 сек',
 'Telegram стоимость текстовое сообщение без фото',
 'Telegram статистика',
 'Telegram город',
 'Telegram стоимость текстовый пост',
 'Telegram стоимость видеокружок',
 'Telegram стоимость 1 сторис до 15 сек',
 'Telegram стоимость за авторский пост',
 'Telegram стоимость за пост из удаления',
 'Telegram стоимость за пост (3/72)',
 'Telegram стоимость за пост (2/48)',
 'Telegram стоимость фотопост в Инстаграм и пост в Телеграм',
 'Telegram охват позиции стоимость за фотопост в Инстаграм и пост в Телеграм',
 'Telegram стоимость спецпроект на Ютуб и пост в Телеграм',
 'Telegram охват позиции стоимость за спецпроект на Ютуб и пост в Телеграм',
 'Telegram стоимость пост в Телеграм и сториз в Инстаграм',
 'Teleg

In [621]:
[col for col in platformacommunity_telegram.columns]

['Канал',
 'Проект / ведущий / идеолог',
 'Тематика',
 'Подписчики',
 'Пост (текст) с закреплением в ТОП, до учета налогов',
 'Пост (текст) без закрепления в ТОП, до учета налогов',
 'Пост (текст+фото) , до учета налогов',
 'Пост (видео) , до учета налогов',
 'Пост (кружочек) , до учета налогов',
 'Unnamed: 9']

In [622]:
bufer_columns = ['блогер',
                 'шоу',
                 'тематика',
                 'количество подписчиков',
                 'стоимость текстовый пост с закрепом',
                 'стоимость текстовый пост без закрепа',
                 'стоимость фотопост',
                 'стоимость видеопост',
                 'стоимость кружочек до 60 сек',
                 'ссылка']

platformacommunity_telegram.columns = bufer_columns

In [623]:
platformacommunity_telegram.head()

,блогер,шоу,тематика,количество подписчиков,стоимость текстовый пост с закрепом,стоимость текстовый пост без закрепа,стоимость фотопост,стоимость видеопост,стоимость кружочек до 60 сек,ссылка
0,лина дианова,Лина Дианова,Психология,35 000,"90 000,00","60 000,00",NaN,NaN,NaN,https://t.me/lina_dianova
1,kadi,Диана Кади,Интервью,6 500,"25 000,00","20 000,00",NaN,NaN,NaN,https://t.me/kadinew
2,серотонин,"Профессиональные журналисты глянца (GQ, Vogue,...",Lifestyle,13 000,NaN,NaN,"15 000,00","15 000,00","15 000,00",https://t.me/+i5_0knOc8SE3M2Y0
3,лиза лазерсон,"Феминистка, блогер, ведушая ""Эхо Москвы"" Лиза ...",Социальная и общественная публицистика,34 000,"55 000,00","35 000,00",NaN,NaN,NaN,https://t.me/znachit_net
4,plarforma. producer hub,Platforma Agency,Инфо о рынке digital видео спонсорства,370,NaN,по теме - бесплатно,NaN,NaN,NaN,https://t.me/digitalproducerhub


### Тридцать шестое объединение

In [624]:
bufer = merge_bufer_df(bufer, platformacommunity_telegram, 'Telegram')

Количество дубликатов: 66


In [625]:
#Заполним контактов менеджера
bufer['контакты менеджера'].fillna('Нина Оуэн @ohnonina +7 901 761 4650', inplace=True)

#Заполним значение налога
bufer['налог'].fillna('НДС (20%)', inplace=True)

In [626]:
bufer.tail()

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Подкаст стоимость партнерская рубрика,Подкаст ссылка,Подкаст статистика,"YouTube проект, ведущий, формат",YouTube шоу,Instagram стоимость видеопост и серия сторис,Instagram стоимость фотопост и 6-7 сторис,Telegram шоу,Telegram стоимость текстовый пост с закрепом,Telegram стоимость текстовый пост без закрепа
1038,не тереби дао,Lifestyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,видеоблогер и диджей Ксюша Дукалис,NaN,"325 000,00"
1039,тина канделаки,"Публицистика, LifeStyle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская телеведущая и журналистка, продюсер...",NaN,NaN
1040,super.ru,Новости шоу-бизнеса,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская журналистка, видеоблогер, репортер ...",NaN,NaN
1041,the flow,Лайфстайл о мире медиа и хип-хопа,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Медиа о рэпе и всём, что популярно у современ...",NaN,"100 000,00"
1042,паша техник live,Лайфстайл Паши техника,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Личный блог Паши техника,NaN,"30 000,00"


## Подготовка данных единого DataFrame

### Очистка данных

In [627]:
itog = bufer.replace({None: np.nan})
itog = bufer.replace('None', np.nan)

In [628]:
itog

,блогер,тематика,Instagram блогер,Instagram ссылка,Instagram тематика,Instagram количество подписчиков,Instagram среднее кол-во просмотров фотопоста,Instagram среднее кол-во просмотров 1 сторис,Instagram среднее кол-во просмотров reels / видеопоста,Instagram среднее кол-во лайков,...,Подкаст стоимость партнерская рубрика,Подкаст ссылка,Подкаст статистика,"YouTube проект, ведущий, формат",YouTube шоу,Instagram стоимость видеопост и серия сторис,Instagram стоимость фотопост и 6-7 сторис,Telegram шоу,Telegram стоимость текстовый пост с закрепом,Telegram стоимость текстовый пост без закрепа
0,катя клэп,"бьюти, лайфстайл",катя клэп,https://www.instagram.com/kateclapp/,"бьюти, лайфстайл",10 100 000,250 000,150 000,200 000,50 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,kari,"бьюти, лайфстайл",kari,https://www.instagram.com/k.hism/,"бьюти, лайфстайл",101 000,100 000,20 000,100 000,7 000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,рязанова татьяна,"бьюти, лайфстайл",NaN,NaN,"бьюти, лайфстайл",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ксюша icceeeberg,"бьюти, лайфстайл",ксюша icceeeberg,https://instagram.com/icceeeberg,"бьюти, лайфстайл",590 000,70 000,50 000,280 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,катя (ekatze),"бьюти, лайфстайл",катя (ekatze),https://www.instagram.com/ekatze007,"бьюти, лайфстайл",452 000,60 000,25 000,120 000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,не тереби дао,Lifestyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,видеоблогер и диджей Ксюша Дукалис,NaN,"325 000,00"
1039,тина канделаки,"Публицистика, LifeStyle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская телеведущая и журналистка, продюсер...",NaN,NaN
1040,super.ru,Новости шоу-бизнеса,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"российская журналистка, видеоблогер, репортер ...",NaN,NaN
1041,the flow,Лайфстайл о мире медиа и хип-хопа,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Медиа о рэпе и всём, что популярно у современ...",NaN,"100 000,00"


In [629]:
[col for col in itog.columns]

['блогер',
 'тематика',
 'Instagram блогер',
 'Instagram ссылка',
 'Instagram тематика',
 'Instagram количество подписчиков',
 'Instagram среднее кол-во просмотров фотопоста',
 'Instagram среднее кол-во просмотров 1 сторис',
 'Instagram среднее кол-во просмотров reels / видеопоста',
 'Instagram среднее кол-во лайков',
 'Instagram среднее кол-во комментариев',
 'Instagram стоимость фотопост',
 'Instagram стоимость 1 сторис до 15 сек',
 'Instagram стоимость серия из 3х сторис до 45 сек',
 'Instagram стоимость reels / видеопост',
 'Instagram статистика',
 'YouTube ссылка',
 'YouTube тематика',
 'YouTube количество подписчиков',
 'YouTube среднее кол-во просмотров видео',
 'YouTube среднее кол-во просмотров shorts',
 'YouTube среднее кол-во лайков',
 'YouTube среднее кол-во комментариев',
 'YouTube стоимость интеграция 1 слот',
 'YouTube стоимость продакт плейсмент',
 'YouTube стоимость преролл до 30 сек',
 'YouTube стоимость мидролл до 30 сек',
 'YouTube стоимость shorts до 1 мин',
 'YouT

In [630]:
column_names_list = itog.columns.tolist()

In [632]:
exclude_words = ['город',
                 'ссылка',
                 'статистика',
                 'блогер',
                 'тематика',
                 'ца по возрасту',
                 'проект, ведущий, формат',
                 'материалы и статистика',
                 'ца по полу',
                 'контакты менеджера',
                 'название канала',
                 'ца по городам',
                 'шоу',
                 'ца по странам',
                 'налог']

# Создаем список из названий столбцов, исключая те, которые содержат определённые слова
spisok_columns = [col for col in itog.columns if not any(word in col for word in exclude_words)]

In [633]:
for columns in spisok_columns:
    itog[columns] = itog[columns].astype(str)
    itog[columns] = itog[columns].str.replace('р.', '', regex=True)
    itog[columns] = itog[columns].str.replace(r'(\d+),(\d+)', r'\1.\2', regex=True)
    itog[columns] = itog[columns].str.replace('[^\d.]', '', regex=True)
    itog[columns] = itog[columns].str.replace(' ', '')
    itog[columns] = itog[columns].replace('', np.nan)
    itog[columns] = itog[columns].astype(float)